<a href="https://colab.research.google.com/github/sourcesync/kagglex_gemma/blob/gw%2Finitial/colab/georges_copy_of_05_mAIstory_925_examples_HF_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<link rel="stylesheet" href="/site-assets/css/gemma.css">
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Google+Symbols:opsz,wght,FILL,GRAD@20..48,100..700,0..1,-50..200" />

## This current set up will only work with A100 as this process require more than 25GB of RAM to finetune for 3 epochs.

####OutOfMemoryError: CUDA out of memory. Tried to allocate 42.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 13.06 MiB is free. Process 4703 has 14.73 GiB memory in use. Of the allocated memory 14.58 GiB is allocated by PyTorch, and 23.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


# Set environment variables

In [1]:
# Upgrade pip first
!pip install --upgrade pip

# Install/upgrade required packages
!pip install --upgrade transformers peft datasets accelerate
!pip install --upgrade tokenizers
!pip install --upgrade torch

!pip install langchain
!pip install --upgrade langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 115.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import json

import torch
from datasets import Dataset, load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType, PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import TextGenerationPipeline

from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

from huggingface_hub import login

import os

from google.colab import userdata
from google.colab import drive

In [3]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.environ["WANDB_DISABLED"] = "true" # George - Disable HB trainer WANDB integration
# Apparently training now requires a weights and biases api key
#import wandb
#wandb.login(key=userdata.get('WANDB_KEY'))

# My story-generation rules
1.Start with the child's input or question about a scenario. 2. Offering choices between two options and explaining them when the child asks. 3. Building on the child's choices to create a continuing adventure. 4. Including elements of magic, nature, imagination, and morality. 5. Ending with another choice that could lead to further adventure. 6. For the child's input also incorporate direct questions about words such as "what is a spirit?" or "what's a castle?" 7. Format the answers as code with each bracket ending with a comma, even the final bracket. 8. Randomly begin some input with one word, like "banana", "lying", or "dragon" and proceed to tell a relevant story. 9. Each line should be between 3 to 20 words.

## The stories

In [5]:
story_conversations = [
    # Story 1: Mei the Butterfly
    [
        {"role": "Child", "text": "What's a butterfly?"},
        {"role": "Narrator", "text": "Great question! A butterfly is a beautiful insect with colorful wings. They start as caterpillars and then change into butterflies. Would you like to hear more about butterflies or continue with Mei's story?"},
        {"role": "Child", "text": "Tell me more about butterflies!"},
        {"role": "Narrator", "text": "Butterflies have four wings covered in tiny scales. They use their long tongues, called proboscises, to drink nectar from flowers. Butterflies are important because they help pollinate plants. Now, shall we return to Mei's story?"},
        {"role": "Child", "text": "Yes, let's continue the story."},
        {"role": "Narrator", "text": "Mei flew to a flower. She saw two paths: one led to a pond, the other to a field of flowers. Where should Mei go?"},
        {"role": "Child", "text": "Let's go to the pond!"},
        {"role": "Narrator", "text": "Mei fluttered to the pond. There, she met Froggy the frog. 'Hello!' said Froggy. 'Want to play leap frog?' Should Mei play with Froggy or keep exploring?"},
        {"role": "Child", "text": "Mei should play with Froggy!"},
        {"role": "Narrator", "text": "Mei and Froggy had a wonderful time playing leap frog. They laughed and jumped all afternoon. The end."}
    ],

    # Story 2: The Magical Forest
    [
        {"role": "Child", "text": "Can trees really talk?"},
        {"role": "Narrator", "text": "In real life, trees can't talk like we do. But in our magical story, Oakley is special and can speak! Should we make Oakley have a deep voice or a squeaky voice?"},
        {"role": "Child", "text": "A deep voice!"},
        {"role": "Narrator", "text": "With a deep, rumbling voice, Oakley said, 'Welcome to the magical forest!' A rabbit hopped by. Should we talk to the rabbit or explore more of the forest?"},
        {"role": "Child", "text": "Let's explore more!"},
        {"role": "Narrator", "text": "As you explore, you find a glowing mushroom and a sparkly river. Which one should we investigate?"},
        {"role": "Child", "text": "The glowing mushroom!"},
        {"role": "Narrator", "text": "You approach the glowing mushroom. It starts to grow and change colors! What color should it turn into?"},
        {"role": "Child", "text": "Purple!"},
        {"role": "Narrator", "text": "The mushroom turns a beautiful purple color. Suddenly, it sprouts tiny wings and starts to fly! Would you like to follow it or go back to Oakley the tree?"},
        {"role": "Child", "text": "Let's follow the flying mushroom!"},
        {"role": "Narrator", "text": "You follow the flying purple mushroom to a hidden cave filled with twinkling stars. The end of today's adventure! What did you like best about our magical forest story?"}
    ],

    # Story 3: Captain Whiskers' Pirate Adventure
    [
        {"role": "Child", "text": "What's a pirate?"},
        {"role": "Narrator", "text": "Great question! Pirates are sailors who sail the seas looking for treasure and adventure. They often have ships with flags called Jolly Rogers. Would you like to learn more about pirates or continue with Captain Whiskers' story?"},
        {"role": "Child", "text": "Tell me more about pirates!"},
        {"role": "Narrator", "text": "Pirates wore eye patches, had parrots as pets, and said 'Arrr!' a lot. They used maps to find buried treasure on islands. Some famous pirates were Blackbeard and Anne Bonny. Shall we get back to Captain Whiskers now?"},
        {"role": "Child", "text": "Yes, let's continue the story."},
        {"role": "Narrator", "text": "Captain Whiskers saw two islands ahead. One had a volcano, the other had a jungle. Where should he sail?"},
        {"role": "Child", "text": "Let's go to the volcano island!"},
        {"role": "Narrator", "text": "Captain Whiskers steered towards the volcano island. As they got closer, they saw shiny gems in the rocks! Should Captain Whiskers collect the gems or look for a safer place to land?"},
        {"role": "Child", "text": "Collect the gems!"},
        {"role": "Narrator", "text": "Captain Whiskers and his crew carefully collected the colorful gems. Suddenly, the volcano started rumbling! What should they do?"},
        {"role": "Child", "text": "Quick, sail away!"},
        {"role": "Narrator", "text": "With a loud 'Meow!', Captain Whiskers ordered his crew to set sail. They escaped just as the volcano erupted, their ship loaded with treasure. The end of this swashbuckling adventure!"}
    ],

    # Story 4: Robo-Pup's Space Mission
    [
        {"role": "Child", "text": "What's a robot?"},
        {"role": "Narrator", "text": "Excellent question! A robot is a machine that can do tasks automatically. Some look like animals or humans, while others look very different. Would you like to know more about robots or continue with Robo-Pup's adventure?"},
        {"role": "Child", "text": "Let's hear more about Robo-Pup!"},
        {"role": "Narrator", "text": "Robo-Pup had light-up eyes and a tail that worked like an antenna. One day, Robo-Pup received a mission to explore a new planet. Should Robo-Pup go alone or take a space buddy?"},
        {"role": "Child", "text": "Take a space buddy!"},
        {"role": "Narrator", "text": "Robo-Pup's friend, Astro-Cat, joined the mission. They landed on the planet and saw purple trees and floating rocks. Should they explore the trees or the rocks first?"},
        {"role": "Child", "text": "Let's check out the floating rocks!"},
        {"role": "Narrator", "text": "Robo-Pup and Astro-Cat bounced from rock to rock. Suddenly, they heard a friendly 'bleep-bloop' sound. It was an alien! Should they try to communicate or head back to their spaceship?"},
        {"role": "Child", "text": "Try to communicate!"},
        {"role": "Narrator", "text": "Robo-Pup wagged its antenna-tail and made friendly beeping sounds. The alien understood and invited them to a space tea party! They all became good friends and promised to meet again. The end of this cosmic adventure!"}
    ],

    # Story 5: The Enchanted Paintbrush
    [
        {"role": "Child", "text": "Can paintbrushes really be magical?"},
        {"role": "Narrator", "text": "In our imagination, anything can be magical! In real life, paintbrushes are tools artists use to create beautiful pictures. But in our story, Bristles is special. Should Bristles paint with rainbow colors or glitter paint?"},
        {"role": "Child", "text": "Rainbow colors!"},
        {"role": "Narrator", "text": "Bristles dipped into rainbow colors and started to paint. The painting came to life! It showed a castle and a dragon. Should we go into the painting or keep watching?"},
        {"role": "Child", "text": "Let's go into the painting!"},
        {"role": "Narrator", "text": "Whoosh! You're inside the painting now. The friendly dragon offers you a ride. Do you want to fly to the castle or over the rainbow river?"},
        {"role": "Child", "text": "Let's fly over the rainbow river!"},
        {"role": "Narrator", "text": "As you fly over the rainbow river, you see fish jumping and changing colors. Suddenly, you spot a treasure chest on a small island. Should you land and open it?"},
        {"role": "Child", "text": "Yes, let's open the treasure chest!"},
        {"role": "Narrator", "text": "Inside the chest, you find magic paints that can bring anything you draw to life! You paint a door back to the art studio and step through. The end of this colorful adventure! What would you like to paint with magic paints?"}
    ],

    # Story 6: The Time-Traveling Teddy
    [
        {"role": "Child", "text": "Let's see the dinosaurs!"},
        {"role": "Narrator", "text": "Whoosh! You're in a prehistoric jungle. A friendly brachiosaurus lowers its long neck. Want to feed it or explore more?"},
        {"role": "Child", "text": "Let's feed the dinosaur!"},
        {"role": "Narrator", "text": "The brachiosaurus munches happily on leaves you offer. Suddenly, you hear a roar! Is it a T-Rex? Should we hide or investigate?"},
        {"role": "Child", "text": "Let's hide!"},
        {"role": "Narrator", "text": "You hide in a cave and discover glowing crystals. They can take you home or to another time. Which do you choose?"}
    ],

    # Story 7: The Underwater Orchestra
    [
        {"role": "Child", "text": "Let's play a happy tune!"},
        {"role": "Narrator", "text": "The happy music attracts colorful fish and playful dolphins. They want to join in! Should Melody teach them to sing or dance?"},
        {"role": "Child", "text": "Teach them to dance!"},
        {"role": "Narrator", "text": "The sea creatures twirl and swirl in a beautiful underwater ballet. A grumpy shark approaches. Will Melody play music or hide?"},
        {"role": "Child", "text": "Play music for the shark!"},
        {"role": "Narrator", "text": "The shark smiles and joins the dance! Now everyone's having fun. Should they perform for the merpeople or explore a shipwreck?"}
    ],

    # Story 8: The Cloud Castle
    [
        {"role": "Child", "text": "Let's knock!"},
        {"role": "Narrator", "text": "A friendly wind spirit opens the door. 'Welcome to Cloud Castle!' it whispers. Would you like a tour or some cloud candy?"},
        {"role": "Child", "text": "Let's try the cloud candy!"},
        {"role": "Narrator", "text": "The candy makes you float! You drift into a room full of rainbows. Want to slide down one or make a rainbow bracelet?"},
        {"role": "Child", "text": "Let's slide down a rainbow!"},
        {"role": "Narrator", "text": "Wheee! You land in a field of flowers. There's a door back to Cloud Castle or a path to Moonbeam Meadow. Where to?"}
    ],

    # Story 9: The Talking Garden
    [
        {"role": "Child", "text": "Let's ask how it talks!"},
        {"role": "Narrator", "text": "The flower giggles. 'Magic soil lets us talk! Want to chat with the veggies or the trees? They have great stories!'"},
        {"role": "Child", "text": "Let's talk to the trees!"},
        {"role": "Narrator", "text": "The old oak tells tales of forest adventures. Suddenly, dark clouds gather. Should we ask the oak for shelter or find a cave?"},
        {"role": "Child", "text": "Let's ask the oak for shelter!"},
        {"role": "Narrator", "text": "The oak's branches curl around you, keeping you dry. After the rain, a rainbow appears. Want to follow it or stay and chat?"}
    ],

    # Story 10: The Cosmic Carousel
    [
        {"role": "Child", "text": "Let's go to Mars!"},
        {"role": "Narrator", "text": "Welcome to Mars! Friendly Martians wave hello. They offer you a ride in their rover or a tour of their city. Which do you choose?"},
        {"role": "Child", "text": "Let's ride in the rover!"},
        {"role": "Narrator", "text": "The rover zooms over red hills. You spot a glittering cave and a floating asteroid. Which would you like to explore?"},
        {"role": "Child", "text": "Let's check out the glittering cave!"},
        {"role": "Narrator", "text": "The cave is full of sparkly Mars crystals. You can take one home or plant it to grow a crystal tree. What's your choice?"}
    ],

    # Story 11: The Magical Cookbook
    [
        {"role": "Child", "text": "Let's make cookies!"},
        {"role": "Narrator", "text": "As you mix the dough, it turns rainbow-colored. The smell of magic fills the air. Should we bake them or eat them raw?"},
        {"role": "Child", "text": "Let's bake them!"},
        {"role": "Narrator", "text": "The oven dings, and out pop cookies shaped like stars. When you bite one, you start to float! What should we do now?"},
        {"role": "Child", "text": "Let's fly around the house!"},
        {"role": "Narrator", "text": "You soar through rooms, giggling. Suddenly, you hear a knock. It's the Cookie Wizard! Should we invite him in or hide?"},
        {"role": "Child", "text": "Let's invite him in!"},
        {"role": "Narrator", "text": "The Wizard smiles and offers to teach you more recipes. Do you want to learn how to make flying pizza or talking cupcakes?"}
    ],

    # Story 12: The Dreamy Pillow Fort
    [
        {"role": "Child", "text": "Let's check out the tower!"},
        {"role": "Narrator", "text": "In the tower, you find a telescope that shows different worlds. Do you want to look at the candy planet or the dragon realm?"},
        {"role": "Child", "text": "Let's see the dragon realm!"},
        {"role": "Narrator", "text": "You see friendly dragons playing tag in the sky. One waves at you! Should we wave back or duck down?"},
        {"role": "Child", "text": "Let's wave back!"},
        {"role": "Narrator", "text": "The dragon flies closer and offers you a ride. Do you want to go or stay in the castle?"},
        {"role": "Child", "text": "Let's go for a ride!"},
        {"role": "Narrator", "text": "You soar through clouds on the dragon's back. Below, you see a pirate ship and a floating island. Where should we land?"},
        {"role": "Child", "text": "Let's land on the floating island!"},
        {"role": "Narrator", "text": "The island is full of bouncy mushrooms and singing flowers. A fairy appears and offers to grant one wish. What do you wish for?"},
        {"role": "Child", "text": "I wish for the ability to fly!"},
        {"role": "Narrator", "text": "Sparkles surround you, and suddenly you can fly! Will you race the dragon back to the castle or explore more of the sky?"},
        {"role": "Child", "text": "Let's race the dragon!"},
        {"role": "Narrator", "text": "You zoom through the air, laughing with the dragon. Back at the castle, you find a magic door. Should we see where it leads?"}
    ],
        # Story 13: The Dino Detective
    [
        {"role": "Child", "text": "What kind of dinosaur is it?"},
        {"role": "Narrator", "text": "It looks like a small T-Rex! Would you like to learn more about T-Rex dinosaurs or name your new friend?"},
        {"role": "Child", "text": "Let's name it Rex!"},
        {"role": "Narrator", "text": "Rex grows quickly and can now talk. He wants to find his family. Should you search in the forest or the mountains?"},
        {"role": "Child", "text": "What might we find in the mountains?"},
        {"role": "Narrator", "text": "The mountains might have caves where dinosaurs live. We could find footprints too. Shall we check the mountains or forest?"},
        {"role": "Child", "text": "Let's check the mountains!"},
        {"role": "Narrator", "text": "In the mountains, you find dinosaur footprints. They lead to a cave and a valley. Which way should you go?"},
        {"role": "Child", "text": "Tell me more about the cave."},
        {"role": "Narrator", "text": "The cave looks dark but might shelter dinosaurs. The valley is open with plants dinosaurs could eat. Cave or valley?"},
        {"role": "Child", "text": "Let's explore the cave!"},
        {"role": "Narrator", "text": "The cave glows with magic crystals. You see a time portal and Rex's family. Should Rex stay with you or go home?"}
    ],

    # Story 14: The Enchanted Storybook
    [
        {"role": "Child", "text": "What can the magic wand do?"},
        {"role": "Narrator", "text": "The wand can cast spells and create magic. Flying shoes let you soar through the sky. Wand or shoes?"},
        {"role": "Child", "text": "I'll take the magic wand!"},
        {"role": "Narrator", "text": "With a wave of your wand, flowers bloom everywhere. A frog prince asks for help. Do you kiss him or find another way?"},
        {"role": "Child", "text": "Is there another way to help?"},
        {"role": "Narrator", "text": "You could use your wand to break the spell. Or kiss him like in the story. What do you want to try?"},
        {"role": "Child", "text": "Let's use the wand!"},
        {"role": "Narrator", "text": "You turn the frog into a balloon. He floats away to a castle! Should you follow him or explore the Candy House nearby?"},
        {"role": "Child", "text": "What's in the Candy House?"},
        {"role": "Narrator", "text": "The Candy House might have treats and magic. The castle could have more fairy tale characters. Candy House or castle?"},
        {"role": "Child", "text": "Let's go to the Candy House!"},
        {"role": "Narrator", "text": "The Candy House belongs to a friendly witch. She offers to teach you a spell. What kind of spell do you want to learn?"}
    ],

    # Story 15: The Weather Wizard
    [
        {"role": "Child", "text": "What happens if we make it snow?"},
        {"role": "Narrator", "text": "Snow will cover everything in white. You could build snowmen or go sledding. Want to make it snow or rain?"},
        {"role": "Child", "text": "Let's make it snow!"},
        {"role": "Narrator", "text": "Snowflakes fall, covering everything in white. Your snowman comes to life! Should you build him a friend or go sledding?"},
        {"role": "Child", "text": "Can the snowman talk?"},
        {"role": "Narrator", "text": "Yes, the snowman can talk and move! Would you like to chat with him or build him a snowy friend?"},
        {"role": "Child", "text": "Let's build a friend!"},
        {"role": "Narrator", "text": "You build a snow dog. Suddenly, the sun remote button glows. Do you press it or keep playing in the snow?"},
        {"role": "Child", "text": "What will happen if we press the sun button?"},
        {"role": "Narrator", "text": "The sun might melt the snow and change the season. Your snow friends could melt too. Press the button or keep playing?"},
        {"role": "Child", "text": "Let's press the sun button!"},
        {"role": "Narrator", "text": "The snow melts, revealing a rainbow garden. A fairy appears, offering to grant a wish. What do you wish for?"}
    ],

    # Story 16: Super You
    [
        {"role": "Child", "text": "What can I do if I'm invisible?"},
        {"role": "Narrator", "text": "When invisible, you can sneak around unseen and surprise people. Flying lets you soar through the sky. Invisibility or flying?"},
        {"role": "Child", "text": "I want to fly!"},
        {"role": "Narrator", "text": "You soar through the clouds. Below, you see a cat stuck in a tree and a runaway hot air balloon. Which do you help?"},
        {"role": "Child", "text": "Can we help both?"},
        {"role": "Narrator", "text": "We only have time for one. The cat looks scared, and the balloon is drifting away. Which one first?"},
        {"role": "Child", "text": "Let's help the cat!"},
        {"role": "Narrator", "text": "You rescue the cat, who can talk! He tells you about a secret superhero school. Do you go there or keep exploring?"},
        {"role": "Child", "text": "What's at the superhero school?"},
        {"role": "Narrator", "text": "The school teaches new superpowers and how to be a hero. You could learn amazing things! Go to school or explore more?"},
        {"role": "Child", "text": "Let's go to superhero school!"},
        {"role": "Narrator", "text": "At the school, you can learn super strength or mind reading. Which superpower do you want to learn?"}
    ],

    # Story 17: Jungle Jam
    [
        {"role": "Child", "text": "What does the magical fruit do?"},
        {"role": "Narrator", "text": "The fruit might give you animal speaking powers. The map leads to ancient treasures. Magical fruit or lost city map?"},
        {"role": "Child", "text": "I'll take the map!"},
        {"role": "Narrator", "text": "The map leads to a river. You can ride a hippo or swing on vines to cross. How do you want to travel?"},
        {"role": "Child", "text": "Is it safe to ride a hippo?"},
        {"role": "Narrator", "text": "The hippo seems friendly but might be slow. Vines are faster but trickier. Hippo ride or vine swinging?"},
        {"role": "Child", "text": "Let's swing on vines!"},
        {"role": "Narrator", "text": "Swinging on vines, you reach an ancient temple. Inside, you find a talking parrot and a golden statue. Who do you approach first?"},
        {"role": "Child", "text": "What can the parrot tell us?"},
        {"role": "Narrator", "text": "The parrot might know secrets about the temple. The statue could be magical. Talk to the parrot or examine the statue?"},
        {"role": "Child", "text": "Let's talk to the parrot!"},
        {"role": "Narrator", "text": "The parrot knows where to find rare rainbow feathers. Do you look for the feathers or explore the temple more?"}
    ],

    # Story 18: Polar Pals
    [
        {"role": "Child", "text": "Is it dangerous to help the cub?"},
        {"role": "Narrator", "text": "The cub seems friendly, but wild animals can be unpredictable. The station is safer. Help the cub or find the station?"},
        {"role": "Child", "text": "Let's help find the cub's mom!"},
        {"role": "Narrator", "text": "You and the cub trek across the ice. You spot a fishing hole and a shimmering aurora. Where should you investigate?"},
        {"role": "Child", "text": "What's special about the aurora?"},
        {"role": "Narrator", "text": "The aurora is a beautiful light show in the sky. It might lead somewhere magical. Check the aurora or fishing hole?"},
        {"role": "Child", "text": "Let's check out the aurora!"},
        {"role": "Narrator", "text": "The aurora leads you to a magical ice palace. A friendly walrus invites you in. Do you enter or keep searching?"},
        {"role": "Child", "text": "What's inside the ice palace?"},
        {"role": "Narrator", "text": "The palace might have magic or clues about the cub's mom. But entering could be risky. Enter or continue searching?"},
        {"role": "Child", "text": "Let's go inside!"},
        {"role": "Narrator", "text": "Inside, you find the bear's mom and a map to hidden treasure. What do you want to do next?"}
    ],

    # Story 19: Desert Discovery
    [
        {"role": "Child", "text": "What might happen if we rub the lamp?"},
        {"role": "Narrator", "text": "Rubbing the lamp might release a genie who grants wishes. But it could be a trick. Rub the lamp or keep riding?"},
        {"role": "Child", "text": "Let's rub the lamp!"},
        {"role": "Narrator", "text": "A genie appears! He offers to grant a wish or show you a hidden oasis. What do you choose?"},
        {"role": "Child", "text": "What's in the hidden oasis?"},
        {"role": "Narrator", "text": "The oasis might have water, palm trees, and magic. A wish could be anything you want. Oasis or wish?"},
        {"role": "Child", "text": "Take us to the oasis!"},
        {"role": "Narrator", "text": "The oasis is beautiful, with a magical pool. Do you swim in it or drink from it?"},
        {"role": "Child", "text": "Is it safe to swim in?"},
        {"role": "Narrator", "text": "The pool sparkles magically. Swimming might transform you, drinking could give powers. Swim, drink, or neither?"},
        {"role": "Child", "text": "Let's go swimming!"},
        {"role": "Narrator", "text": "As you swim, you transform into a mermaid! You can now breathe underwater. Do you explore the deep or return to the surface?"}
    ],

    # Story 20: Tiny Town
    [
        {"role": "Child", "text": "How fast is the ladybug taxi?"},
        {"role": "Narrator", "text": "The ladybug is faster than the snail but bumpier. The snail is slow but smooth. Quick ladybug or comfy snail?"},
        {"role": "Child", "text": "Let's take the ladybug taxi!"},
        {"role": "Narrator", "text": "The ladybug takes you to a fairy village in a flower patch. Do you attend the pollen ball or join a leaf boat race?"},
        {"role": "Child", "text": "What happens at the pollen ball?"},
        {"role": "Narrator", "text": "The pollen ball has dancing and fairy food. The leaf boat race is an exciting adventure. Ball or boat race?"},
        {"role": "Child", "text": "Let's go to the pollen ball!"},
        {"role": "Narrator", "text": "At the ball, you dance with butterflies. A fairy offers to grant you butterfly wings or the ability to talk to plants. Which do you choose?"},
        {"role": "Child", "text": "What can I do if I talk to plants?"},
        {"role": "Narrator", "text": "Talking to plants lets you learn nature secrets. Butterfly wings let you fly. Plant talking or wings?"},
        {"role": "Child", "text": "I want to talk to plants!"},
        {"role": "Narrator", "text": "You learn that the plants need help. Will you stay tiny to help them or return to your normal size?"}
    ],

    # Story 21: The Friendly Dragon
    [
        {"role": "Child", "text": "Tell me a story about a dragon!"},
        {"role": "Narrator", "text": "Once, there was a dragon named Sparky who couldn't breathe fire. He wanted to make friends. Should he visit the village or the forest?"},
        {"role": "Child", "text": "What's in the village?"},
        {"role": "Narrator", "text": "The village has people who might be scared of dragons. The forest has magical creatures. Village or forest?"},
        {"role": "Child", "text": "Let's go to the forest!"},
        {"role": "Narrator", "text": "In the forest, Sparky meets a unicorn and a phoenix. Who should he talk to first?"},
        {"role": "Child", "text": "Let's talk to the phoenix!"},
        {"role": "Narrator", "text": "The phoenix offers to teach Sparky to breathe fire or fly better. Which should he learn?"}
    ],

    # Story 22: The Mermaid's Adventure
    [
        {"role": "Child", "text": "Can you tell a story about mermaids?"},
        {"role": "Narrator", "text": "Marina the mermaid discovers a sunken pirate ship. Inside, there's a treasure map and a magic pearl. Which should she take?"},
        {"role": "Child", "text": "What does the magic pearl do?"},
        {"role": "Narrator", "text": "The pearl might grant wishes or give land legs. The map leads to treasure. Pearl or map?"},
        {"role": "Child", "text": "Let's take the pearl!"},
        {"role": "Narrator", "text": "The pearl gives Marina legs! Should she explore the beach or a human town?"},
        {"role": "Child", "text": "Let's go to the town!"},
        {"role": "Narrator", "text": "In town, Marina can join a dance or visit a library. Which sounds more fun?"}
    ],

    # Story 23: Space Explorers
    [
        {"role": "Child", "text": "I want to hear a story about space!"},
        {"role": "Narrator", "text": "You're an astronaut on a mission to a new planet. Your ship can land on a purple mountain or in a crystal valley. Where do you land?"},
        {"role": "Child", "text": "What might we find in the crystal valley?"},
        {"role": "Narrator", "text": "The valley might have valuable crystals or alien life. The mountain could have a great view. Valley or mountain?"},
        {"role": "Child", "text": "Let's check out the crystal valley!"},
        {"role": "Narrator", "text": "In the valley, you meet friendly aliens. They invite you to their underwater city or flying cloud home. Which do you visit?"},
        {"role": "Child", "text": "The underwater city sounds cool!"},
        {"role": "Narrator", "text": "The underwater city is amazing! You can study alien fish or join a bubble race. What do you choose?"}
    ],

    # Story 24: The Enchanted Forest
    [
        {"role": "Child", "text": "Tell me about a magical forest!"},
        {"role": "Narrator", "text": "You enter an enchanted forest where animals can talk. A wise owl and a mischievous fox approach you. Who do you speak to?"},
        {"role": "Child", "text": "What might the owl tell us?"},
        {"role": "Narrator", "text": "The owl might share ancient wisdom. The fox could lead you on an adventure. Owl or fox?"},
        {"role": "Child", "text": "Let's talk to the owl!"},
        {"role": "Narrator", "text": "The owl tells you about a hidden treehouse or a magical pond. Which place do you want to visit?"},
        {"role": "Child", "text": "The magical pond sounds interesting!"},
        {"role": "Narrator", "text": "At the pond, you can drink the water to shrink or grow giant. Which do you try?"}
    ],

    # Story 25: Pirate's Treasure
    [
        {"role": "Child", "text": "I want a story about pirates!"},
        {"role": "Narrator", "text": "You're a pirate captain searching for treasure. Your map shows a mysterious island. Do you sail there or search the sea caves?"},
        {"role": "Child", "text": "What's on the mysterious island?"},
        {"role": "Narrator", "text": "The island might have buried treasure or dangerous creatures. Sea caves could hide shipwrecks. Island or caves?"},
        {"role": "Child", "text": "Let's go to the island!"},
        {"role": "Narrator", "text": "On the island, you find a talking parrot and an old bottle with a message. Which do you investigate?"},
        {"role": "Child", "text": "Let's check the bottle!"},
        {"role": "Narrator", "text": "The message leads to treasure, but you need a key. Do you search the beach or the jungle?"}
    ],

    # Story 26: The Time Machine
    [
        {"role": "Child", "text": "Can you tell me a story about time travel?"},
        {"role": "Narrator", "text": "You find a time machine in your attic. You can travel to the age of dinosaurs or the future. When do you go?"},
        {"role": "Child", "text": "What might we see in the future?"},
        {"role": "Narrator", "text": "The future might have flying cars and robots. Dinosaur times have giant creatures. Future or dinosaurs?"},
        {"role": "Child", "text": "Let's go to the future!"},
        {"role": "Narrator", "text": "In the future, you can ride a hover-board or visit a moon colony. What's your choice?"},
        {"role": "Child", "text": "Moon colony sounds awesome!"},
        {"role": "Narrator", "text": "At the moon colony, you can join a space race or help grow moon plants. Which activity do you pick?"}
    ],

    # Story 27: The Magical Toyshop
    [
        {"role": "Child", "text": "Tell me a story about toys coming to life!"},
        {"role": "Narrator", "text": "You're in a toyshop after midnight, and the toys come alive! A teddy bear and a robot want to play. Who do you join?"},
        {"role": "Child", "text": "What games does the robot play?"},
        {"role": "Narrator", "text": "The robot likes puzzle games. The teddy bear prefers tea parties. Robot puzzles or bear's tea party?"},
        {"role": "Child", "text": "Let's play with the robot!"},
        {"role": "Narrator", "text": "The robot's puzzle reveals a secret room. Inside are magic crayons and a flying carpet. Which do you try?"},
        {"role": "Child", "text": "Magic crayons sound fun!"},
        {"role": "Narrator", "text": "The crayons bring drawings to life! Do you draw a friendly monster or a treasure map?"}
    ],

    # Story 28: The Fairy Garden
    [
        {"role": "Child", "text": "I'd like to hear a story about fairies!"},
        {"role": "Narrator", "text": "You shrink and enter a fairy garden. Flower fairies invite you to a petal ball, while forest fairies offer an acorn race. Which do you choose?"},
        {"role": "Child", "text": "What happens at the petal ball?"},
        {"role": "Narrator", "text": "The petal ball has dancing and magic shows. The acorn race is exciting and fast. Ball or race?"},
        {"role": "Child", "text": "Let's go to the petal ball!"},
        {"role": "Narrator", "text": "At the ball, you're offered fairy dust to fly or a wand to cast spells. Which magical item do you pick?"},
        {"role": "Child", "text": "Fairy dust sounds magical!"},
        {"role": "Narrator", "text": "Flying with fairy dust, you spot a rainbow slide and a cloud castle. Where do you fly to?"}
    ],

    # Story 29: Underwater Kingdom
    [
        {"role": "Child", "text": "Can you tell a story about an underwater world?"},
        {"role": "Narrator", "text": "You discover an underwater kingdom. The Mer-King offers you a job as a coral gardener or a dolphin trainer. Which do you choose?"},
        {"role": "Child", "text": "What does a coral gardener do?"},
        {"role": "Narrator", "text": "Coral gardeners grow and protect colorful reefs. Dolphin trainers teach dolphins tricks. Gardener or trainer?"},
        {"role": "Child", "text": "Coral gardener sounds interesting!"},
        {"role": "Narrator", "text": "While gardening, you find a magic shell. It can turn you into a fish or let you breathe underwater. Which power do you want?"},
        {"role": "Child", "text": "Let's turn into a fish!"},
        {"role": "Narrator", "text": "As a fish, you can explore a sunken ship or join a tropical fish parade. What's your adventure?"}
    ],

    # Story 30: The Wizard's Apprentice
    [
        {"role": "Child", "text": "Tell me a story about becoming a wizard!"},
        {"role": "Narrator", "text": "You're a wizard's apprentice. For your first lesson, you can learn to fly on a broomstick or make potions. Which do you choose?"},
        {"role": "Child", "text": "What kinds of potions can we make?"},
        {"role": "Narrator", "text": "You could make potions for invisibility or animal speaking. Flying lets you soar through the sky. Potions or flying?"},
        {"role": "Child", "text": "Let's make potions!"},
        {"role": "Narrator", "text": "Your invisibility potion works! Do you use it to explore the wizard's secret library or magical garden?"},
        {"role": "Child", "text": "The secret library sounds mysterious!"},
        {"role": "Narrator", "text": "In the library, you find a book of dragon-taming and a spell for time travel. Which do you read?"}
    ],

    # Story 31: The Magical Zoo
    [
        {"role": "Child", "text": "Tell me a story about a magical zoo!"},
        {"role": "Narrator", "text": "You visit a zoo where animals have superpowers. A flying elephant and an invisible tiger want to play. Who do you choose?"},
        {"role": "Child", "text": "What's an elephant?"},
        {"role": "Narrator", "text": "An elephant is a large animal with a long trunk and big ears. In our story, this one can fly! Elephant or tiger?"},
        {"role": "Child", "text": "Let's play with the flying elephant!"},
        {"role": "Narrator", "text": "The elephant takes you flying! You see a rainbow waterfall and a cotton candy cloud. Where do you want to go?"},
        {"role": "Child", "text": "What's cotton candy?"},
        {"role": "Narrator", "text": "Cotton candy is a fluffy, sweet treat. This cloud is made of it! Waterfall or cotton candy cloud?"},
        {"role": "Child", "text": "Let's go to the cotton candy cloud!"},
        {"role": "Narrator", "text": "On the cloud, you meet a unicorn. It offers to grant a wish or teach you magic. What do you choose?"}
    ],

    # Story 32: The Talking Tree
    [
        {"role": "Child", "text": "Can you tell a story about a talking tree?"},
        {"role": "Narrator", "text": "In your backyard, an old oak tree starts talking! It offers to show you its underground root city or treetop village. Which do you pick?"},
        {"role": "Child", "text": "What's an oak tree?"},
        {"role": "Narrator", "text": "An oak is a strong tree with leaves and acorns. This one can talk and has secret worlds! Root city or treetop village?"},
        {"role": "Child", "text": "Let's see the treetop village!"},
        {"role": "Narrator", "text": "In the village, you meet leaf fairies and acorn elves. Who do you want to play with?"},
        {"role": "Child", "text": "What are elves?"},
        {"role": "Narrator", "text": "Elves are magical creatures, often small with pointed ears. These live in acorns! Fairies or elves?"},
        {"role": "Child", "text": "Let's play with the acorn elves!"},
        {"role": "Narrator", "text": "The elves invite you to an acorn race or to help build a treehouse. What activity do you choose?"}
    ],

    # Story 33: The Brave Knight
    [
        {"role": "Child", "text": "Tell me a story about a knight!"},
        {"role": "Narrator", "text": "You're a brave knight on a quest. You can search for a dragon's treasure or rescue a wizard from a tower. Which adventure do you pick?"},
        {"role": "Child", "text": "What's a quest?"},
        {"role": "Narrator", "text": "A quest is an exciting journey to achieve something important. Your quest could be for treasure or rescue. Which one?"},
        {"role": "Child", "text": "Let's rescue the wizard!"},
        {"role": "Narrator", "text": "At the tower, you can climb up or look for a secret entrance. How do you want to rescue the wizard?"},
        {"role": "Child", "text": "What's a secret entrance?"},
        {"role": "Narrator", "text": "A secret entrance is a hidden way to get inside somewhere. It might be safer but harder to find. Climb or search?"},
        {"role": "Child", "text": "Let's look for the secret entrance!"},
        {"role": "Narrator", "text": "You find a magical door! To open it, you need to solve a riddle or play a music puzzle. Which challenge do you try?"}
    ],

    # Story 34: The Friendly Ghost
    [
        {"role": "Child", "text": "I want to hear a story about a ghost!"},
        {"role": "Narrator", "text": "You meet a friendly ghost named Boo in an old house. Boo wants to play hide-and-seek or tell spooky stories. What do you want to do?"},
        {"role": "Child", "text": "What's a ghost?"},
        {"role": "Narrator", "text": "A ghost is a spirit that some people think stays after someone is gone. Boo is a friendly, fun ghost! Hide-and-seek or stories?"},
        {"role": "Child", "text": "Let's play hide-and-seek!"},
        {"role": "Narrator", "text": "While hiding, you find a magic mirror and a dusty old book. Which do you want to check out?"},
        {"role": "Child", "text": "What's a magic mirror?"},
        {"role": "Narrator", "text": "A magic mirror might show you different places or grant wishes. The book could have spells. Mirror or book?"},
        {"role": "Child", "text": "The magic mirror sounds cool!"},
        {"role": "Narrator", "text": "The mirror shows two worlds: a candy land and a robot city. Where do you want to go?"}
    ],

    # Story 35: The Time Traveler
    [
        {"role": "Child", "text": "Can you tell a story about time travel?"},
        {"role": "Narrator", "text": "You find a pocket watch that lets you time travel. You can visit ancient Egypt or the far future. When do you want to go?"},
        {"role": "Child", "text": "What's a pocket watch?"},
        {"role": "Narrator", "text": "A pocket watch is a small clock you can carry. This one is magic and takes you through time! Egypt or future?"},
        {"role": "Child", "text": "Let's go to ancient Egypt!"},
        {"role": "Narrator", "text": "In ancient Egypt, you can help build a pyramid or explore the Nile River. What would you like to do?"},
        {"role": "Child", "text": "What's the Nile River?"},
        {"role": "Narrator", "text": "The Nile is a long, famous river in Egypt. It was very important to ancient Egyptians. Build pyramid or explore Nile?"},
        {"role": "Child", "text": "Let's explore the Nile!"},
        {"role": "Narrator", "text": "On the Nile, you meet a friendly hippo and a talking crocodile. Who do you want to talk to?"}
    ],

    # Story 36: The Superhero School
    [
        {"role": "Child", "text": "Tell me about a school for superheroes!"},
        {"role": "Narrator", "text": "You're a new student at Superhero School. Your first class is either Flying 101 or Invisibility Basics. Which do you choose?"},
        {"role": "Child", "text": "What does Invisibility mean?"},
        {"role": "Narrator", "text": "Invisibility means you can't be seen. In this class, you'd learn to disappear! Want to try Flying or Invisibility?"},
        {"role": "Child", "text": "Flying sounds fun!"},
        {"role": "Narrator", "text": "In Flying 101, you can learn to fly with a cape or with jet boots. Which flying method do you prefer?"},
        {"role": "Child", "text": "What are jet boots?"},
        {"role": "Narrator", "text": "Jet boots are special shoes with tiny rockets that let you fly. A cape uses wind to fly. Boots or cape?"},
        {"role": "Child", "text": "Let's try the jet boots!"},
        {"role": "Narrator", "text": "With your jet boots, you can race through an obstacle course or help rescue a cat from a tall tree. What's your choice?"}
    ],

    # Story 37: The Enchanted Toy Shop
    [
        {"role": "Child", "text": "I'd like a story about a magical toy shop!"},
        {"role": "Narrator", "text": "You visit a toy shop where toys come to life at night. A teddy bear invites you to a tea party, and a toy robot wants to play chess. Who do you join?"},
        {"role": "Child", "text": "What's chess?"},
        {"role": "Narrator", "text": "Chess is a thinking game with different pieces on a board. The teddy's tea party would be cozy. Chess or tea party?"},
        {"role": "Child", "text": "The tea party sounds nice!"},
        {"role": "Narrator", "text": "At the tea party, you can drink magic tea that makes you tiny or giant. Which size do you want to be?"},
        {"role": "Child", "text": "What would happen if I'm tiny?"},
        {"role": "Narrator", "text": "If you're tiny, you could explore doll houses or ride toy cars. If giant, you could reach high shelves. Tiny or giant?"},
        {"role": "Child", "text": "Let's be tiny!"},
        {"role": "Narrator", "text": "Now tiny, you can drive a toy car in a race or explore a dollhouse mansion. Which adventure do you choose?"}
    ],

    # Story 38: The Dragon Trainer
    [
        {"role": "Child", "text": "Can you tell a story about training dragons?"},
        {"role": "Narrator", "text": "You're a dragon trainer on a magical island. You can choose to train a fire-breathing dragon or an ice dragon. Which do you pick?"},
        {"role": "Child", "text": "What's an ice dragon?"},
        {"role": "Narrator", "text": "An ice dragon breathes icy air instead of fire. It can freeze things! Do you want to train the fire or ice dragon?"},
        {"role": "Child", "text": "The ice dragon sounds cool!"},
        {"role": "Narrator", "text": "To bond with your ice dragon, you can go ice sliding or build a snow castle together. What activity do you choose?"},
        {"role": "Child", "text": "What's ice sliding?"},
        {"role": "Narrator", "text": "Ice sliding is like sledding on ice - very fast and fun! Snow castles are big forts made of snow. Slide or build?"},
        {"role": "Child", "text": "Let's go ice sliding!"},
        {"role": "Narrator", "text": "While ice sliding, you discover a frozen cave. Inside is a magic ice gem and a map. Which do you want to investigate?"}
    ],

    # Story 39: The Undersea Adventure
    [
        {"role": "Child", "text": "Tell me a story about exploring under the sea!"},
        {"role": "Narrator", "text": "You dive into the ocean and can breathe underwater! A friendly dolphin offers to show you a coral reef or a sunken pirate ship. Where do you go?"},
        {"role": "Child", "text": "What's a coral reef?"},
        {"role": "Narrator", "text": "A coral reef is like an underwater city for fish, full of colorful coral and sea life. Ship or reef?"},
        {"role": "Child", "text": "Let's see the coral reef!"},
        {"role": "Narrator", "text": "At the reef, you meet a wise sea turtle and a playful octopus. Who do you want to talk to?"},
        {"role": "Child", "text": "What's an octopus?"},
        {"role": "Narrator", "text": "An octopus is a sea animal with eight arms called tentacles. It's very smart! Turtle or octopus?"},
        {"role": "Child", "text": "The octopus sounds fun!"},
        {"role": "Narrator", "text": "The octopus shows you its hidden treasure cave. Inside, you find a magic pearl and a mermaid's mirror. Which do you choose?"}
    ],

    # Story 40: The Magical Garden
    [
        {"role": "Child", "text": "I want to hear about a magical garden!"},
        {"role": "Narrator", "text": "You discover a garden where plants can talk. A sunflower wants to share a secret, and a rosebush offers to teach you plant magic. Who do you approach?"},
        {"role": "Child", "text": "What's a sunflower?"},
        {"role": "Narrator", "text": "A sunflower is a tall yellow flower that follows the sun. Roses are pretty flowers with thorns. Sunflower or rosebush?"},
        {"role": "Child", "text": "Let's talk to the sunflower!"},
        {"role": "Narrator", "text": "The sunflower's secret is about a hidden pond that grants wishes or magic seeds that grow instantly. Which interests you more?"},
        {"role": "Child", "text": "What are magic seeds?"},
        {"role": "Narrator", "text": "Magic seeds grow into plants very quickly, maybe even while you watch! Wishing pond or magic seeds?"},
        {"role": "Child", "text": "The magic seeds sound amazing!"},
        {"role": "Narrator", "text": "You plant the magic seeds. They can grow into a beanstalk to the clouds or a rainbow flower. Which do you want to grow?"}
    ],

    # Story 41: The Cloud Kingdom
    [
        {"role": "Child", "text": "Tell me a story about living in the clouds!"},
        {"role": "Narrator", "text": "You discover a kingdom in the clouds! You can become a cloud sculptor or a rainbow painter. Which job do you choose?"},
        {"role": "Child", "text": "What's a sculptor?"},
        {"role": "Narrator", "text": "A sculptor creates art by shaping materials. Here, you'd shape clouds into different forms! Sculptor or painter?"},
        {"role": "Child", "text": "Cloud sculptor sounds fun!"},
        {"role": "Narrator", "text": "As a cloud sculptor, you can make a fluffy castle or a group of cloud animals. What do you want to create?"},
        {"role": "Child", "text": "What kind of cloud animals?"},
        {"role": "Narrator", "text": "You could make cloud elephants, lions, or even dragons! Any animal you imagine. Castle or animals?"},
        {"role": "Child", "text": "Let's make cloud animals!"},
        {"role": "Narrator", "text": "Your cloud animals come to life! They invite you to ride them or play hide-and-seek in the clouds. What do you do?"}
    ],

    # Story 42: The Enchanted Orchestra
    [
        {"role": "Child", "text": "Can you tell a story about magical music?"},
        {"role": "Narrator", "text": "You find a enchanted concert hall where instruments play themselves. A violin and a drum set invite you to join. Which do you pick?"},
        {"role": "Child", "text": "What's a violin?"},
        {"role": "Narrator", "text": "A violin is a string instrument played with a bow. It makes beautiful melodies. Violin or drums?"},
        {"role": "Child", "text": "Let's try the violin!"},
        {"role": "Narrator", "text": "The violin teaches you a song that can make flowers grow or animals dance. Which magic do you want to try?"},
        {"role": "Child", "text": "What kind of animals will dance?"},
        {"role": "Narrator", "text": "All kinds! Rabbits might hop, birds could swoop, and bears could waltz. Grow flowers or make animals dance?"},
        {"role": "Child", "text": "Let's make the animals dance!"},
        {"role": "Narrator", "text": "The animals dance joyfully! Now they want to join your orchestra. Do you add singing birds or drumming elephants?"}
    ],

    # Story 43: The Time-Traveling Train
    [
        {"role": "Child", "text": "Tell me about a magical train ride!"},
        {"role": "Narrator", "text": "You board a train that can travel through time. You can visit the age of dinosaurs or the future. Where do you want to go?"},
        {"role": "Child", "text": "What's a dinosaur?"},
        {"role": "Narrator", "text": "Dinosaurs were giant reptiles that lived long ago. Some were huge, others small. Dinosaur times or the future?"},
        {"role": "Child", "text": "Let's see the dinosaurs!"},
        {"role": "Narrator", "text": "In dinosaur times, you can ride a friendly Triceratops or watch T-Rex hunt. What do you want to do?"},
        {"role": "Child", "text": "What's a Triceratops?"},
        {"role": "Narrator", "text": "A Triceratops is a dinosaur with three horns and a big frill on its head. Ride Triceratops or watch T-Rex?"},
        {"role": "Child", "text": "Riding a Triceratops sounds cool!"},
        {"role": "Narrator", "text": "While riding, you find a nest of dinosaur eggs. Do you want to watch them hatch or continue exploring?"}
    ],

    # Story 44: The Wizard's Apprentice
    [
        {"role": "Child", "text": "I want to hear about learning magic!"},
        {"role": "Narrator", "text": "You become a wizard's apprentice! For your first lesson, you can learn to make potions or cast spells. Which do you choose?"},
        {"role": "Child", "text": "What's a potion?"},
        {"role": "Narrator", "text": "A potion is a magical liquid that can do amazing things when drunk. Potions or spells?"},
        {"role": "Child", "text": "Potions sound interesting!"},
        {"role": "Narrator", "text": "Great! You can make a potion to shrink or to speak with animals. Which potion do you want to brew?"},
        {"role": "Child", "text": "What happens if I shrink?"},
        {"role": "Narrator", "text": "If you shrink, you could explore tiny places and talk to insects. Animal-speaking lets you chat with pets. Shrink or speak to animals?"},
        {"role": "Child", "text": "Let's make the animal-speaking potion!"},
        {"role": "Narrator", "text": "Your potion works! Do you want to talk to a wise owl in the forest or playful dolphins in the sea?"}
    ],

    # Story 45: The Galactic Zoo
    [
        {"role": "Child", "text": "Can you tell a story about aliens?"},
        {"role": "Narrator", "text": "You visit a zoo filled with alien creatures from across the galaxy. You can feed star-dust to a cosmic butterfly or play with a gravity-defying space puppy. What do you choose?"},
        {"role": "Child", "text": "What's stardust?"},
        {"role": "Narrator", "text": "Stardust is tiny, sparkling particles from stars. In our story, it's food for space creatures! Feed butterfly or play with puppy?"},
        {"role": "Child", "text": "Let's play with the space puppy!"},
        {"role": "Narrator", "text": "The space puppy can float and phase through walls. Do you want to have a floating race or play hide-and-seek?"},
        {"role": "Child", "text": "What does 'phase through walls' mean?"},
        {"role": "Narrator", "text": "It means the puppy can pass through solid objects like they're not there. Cool, right? Race or hide-and-seek?"},
        {"role": "Child", "text": "Hide-and-seek sounds fun!"},
        {"role": "Narrator", "text": "During the game, you discover a secret part of the zoo. It has a shape-shifting alien or a telepathic space plant. Which do you approach?"}
    ],

    # Story 46: The Fairy Forest
    [
        {"role": "Child", "text": "Tell me a story about fairies!"},
        {"role": "Narrator", "text": "You shrink to fairy size and enter an enchanted forest. You can join a group making fairy dust or learn to grow magical flowers. What do you pick?"},
        {"role": "Child", "text": "What's fairy dust used for?"},
        {"role": "Narrator", "text": "Fairy dust can make things fly or grant wishes. It's very magical! Make fairy dust or grow flowers?"},
        {"role": "Child", "text": "Let's make fairy dust!"},
        {"role": "Narrator", "text": "While making fairy dust, you can add sparkles for flying or sweetness for good dreams. Which type do you create?"},
        {"role": "Child", "text": "What are sparkles?"},
        {"role": "Narrator", "text": "Sparkles are tiny, shiny bits that glitter and shine. They make the fairy dust extra magical! Flying dust or dream dust?"},
        {"role": "Child", "text": "Flying dust sounds exciting!"},
        {"role": "Narrator", "text": "Your flying dust works! Do you want to race with dragonflies or explore a cloud castle?"}
    ],

    # Story 47: The Underwater City
    [
        {"role": "Child", "text": "I want to hear about a city under the sea!"},
        {"role": "Narrator", "text": "You discover a magical city under the sea. You can become a coral architect or a seahorse racer. Which job interests you?"},
        {"role": "Child", "text": "What's an architect?"},
        {"role": "Narrator", "text": "An architect designs and builds structures. Here, you'd build with coral! Coral architect or seahorse racer?"},
        {"role": "Child", "text": "Coral architect sounds cool!"},
        {"role": "Narrator", "text": "As a coral architect, you can design a playground for fish or a grand palace. What do you want to build?"},
        {"role": "Child", "text": "What's a palace?"},
        {"role": "Narrator", "text": "A palace is a large, fancy building where important people live. Underwater, it'd be for merpeople! Playground or palace?"},
        {"role": "Child", "text": "Let's build a playground!"},
        {"role": "Narrator", "text": "Your playground is a hit! Now, do you want to add a seaweed maze or a bubble elevator?"}
    ],

    # Story 48: The Candy Planet
    [
        {"role": "Child", "text": "Can you tell me about a world made of candy?"},
        {"role": "Narrator", "text": "You land on a planet made entirely of sweets! You can explore the chocolate river or climb the marshmallow mountains. Where do you go?"},
        {"role": "Child", "text": "What are marshmallows?"},
        {"role": "Narrator", "text": "Marshmallows are soft, sweet treats. Imagine big, fluffy clouds you can eat! Chocolate river or marshmallow mountains?"},
        {"role": "Child", "text": "Let's go to the marshmallow mountains!"},
        {"role": "Narrator", "text": "On the marshmallow mountains, you meet a friendly gumdrop giant. He offers to share his jelly bean collection or show you the lollipop forest. What do you choose?"},
        {"role": "Child", "text": "What's a gumdrop?"},
        {"role": "Narrator", "text": "Gumdrops are chewy, sugar-coated candies. This one's a giant! Jelly bean collection or lollipop forest?"},
        {"role": "Child", "text": "The lollipop forest sounds interesting!"},
        {"role": "Narrator", "text": "In the lollipop forest, you can pick a flavor-changing lollipop or ride a candy cane slide. What's your choice?"}
    ],

    # Story 49: The Dragon's School
    [
        {"role": "Child", "text": "Tell me about a school for dragons!"},
        {"role": "Narrator", "text": "You're a young dragon at Dragon School. Your first class is either Fire Breathing 101 or Flying Techniques. Which do you attend?"},
        {"role": "Child", "text": "What does '101' mean?"},
        {"role": "Narrator", "text": "'101' means it's a beginner's class, perfect for learning the basics. Fire Breathing 101 or Flying Techniques?"},
        {"role": "Child", "text": "Fire Breathing 101 sounds exciting!"},
        {"role": "Narrator", "text": "In Fire Breathing 101, you can practice lighting campfires or creating fireworks displays. What do you want to try?"},
        {"role": "Child", "text": "What are fireworks?"},
        {"role": "Narrator", "text": "Fireworks are colorful explosions in the sky, like bright, sparkling flowers. Campfires or fireworks?"},
        {"role": "Child", "text": "Fireworks sound beautiful!"},
        {"role": "Narrator", "text": "Your fireworks are amazing! Now, do you want to join the school's fire-breathing choir or the dragon acrobatics team?"}
    ],

    # Story 50: The Enchanted Toymaker
    [
        {"role": "Child", "text": "I'd like a story about magical toys!"},
        {"role": "Narrator", "text": "You become an apprentice to an enchanted toymaker. You can learn to make toys that come to life or toys that grant wishes. Which do you choose?"},
        {"role": "Child", "text": "What's an apprentice?"},
        {"role": "Narrator", "text": "An apprentice is someone learning a skill from an expert. You're learning toy-making magic! Living toys or wishing toys?"},
        {"role": "Child", "text": "Let's make toys that come to life!"},
        {"role": "Narrator", "text": "Great choice! You can create a teddy bear that gives hugs or a toy car that transforms into different vehicles. Which toy do you make?"},
        {"role": "Child", "text": "What does 'transform' mean?"},
        {"role": "Narrator", "text": "Transform means to change form. The car could change into a boat or plane! Hugging bear or transforming car?"},
        {"role": "Child", "text": "The transforming car sounds cool!"},
        {"role": "Narrator", "text": "Your transforming car is a success! Do you want to test it in a magical race or use it to explore a miniature city?"}
    ],

    # Story 51: Sweet Lessons
    [
        {"role": "Child", "text": "Candy"},
        {"role": "Narrator", "text": "Once, there was a child named Sam who loved candy. One day, Sam found a magical candy store. Should Sam eat one piece or many?"},
        {"role": "Child", "text": "Many!"},
        {"role": "Narrator", "text": "Sam ate lots of candy and got a bad tummy ache. The store owner appeared and offered a healthy snack or more candy. What should Sam choose?"},
        {"role": "Child", "text": "Healthy snack"},
        {"role": "Narrator", "text": "Sam felt better after the healthy snack. The owner praised Sam's choice and gave a special tooth-cleaning lollipop. Brush teeth or eat more candy?"}
    ],

    # Story 52: The Honest Path
    [
        {"role": "Child", "text": "Lying"},
        {"role": "Narrator", "text": "Alex broke a vase and mom asked what happened. Should Alex tell the truth or make up a story?"},
        {"role": "Child", "text": "Make up a story"},
        {"role": "Narrator", "text": "Alex blamed the cat, but felt guilty. Mom looked sad and the cat got in trouble. Should Alex confess or keep quiet?"},
        {"role": "Child", "text": "Confess"},
        {"role": "Narrator", "text": "Alex told the truth. Mom was proud of the honesty. Should Alex help clean up or go play?"}
    ],

    # Story 53: Sharing is Caring
    [
        {"role": "Child", "text": "Toys"},
        {"role": "Narrator", "text": "Jamie had many toys but never shared. One day, a new kid had no toys. Should Jamie share or keep all the toys?"},
        {"role": "Child", "text": "Keep all toys"},
        {"role": "Narrator", "text": "Jamie played alone while the new kid looked sad. A magical toy appeared, offering to double Jamie's toys if shared. Share or keep?"},
        {"role": "Child", "text": "Share"},
        {"role": "Narrator", "text": "Jamie shared and made a new friend. They had twice the fun! Should they invite more friends or play alone?"}
    ],

    # Story 54: The Helping Hand
    [
        {"role": "Child", "text": "Kindness"},
        {"role": "Narrator", "text": "Pat saw an elderly neighbor struggling with groceries. Should Pat help or continue playing?"},
        {"role": "Child", "text": "Help"},
        {"role": "Narrator", "text": "Pat helped and the neighbor was grateful. They offered Pat a choice: a candy bar or a book about kindness. Which should Pat choose?"},
        {"role": "Child", "text": "Book"},
        {"role": "Narrator", "text": "The book taught Pat more ways to be kind. Should Pat start a kindness club or do secret good deeds?"}
    ],

    # Story 55: The Bedtime Battle
    [
        {"role": "Child", "text": "Sleep"},
        {"role": "Narrator", "text": "Casey didn't want to sleep and stayed up late playing. In the morning, Casey was tired. Go to school or pretend to be sick?"},
        {"role": "Child", "text": "Pretend to be sick"},
        {"role": "Narrator", "text": "Casey stayed home but felt bad about lying. A fairy appeared, offering early bedtime or unlimited playtime. What should Casey choose?"},
        {"role": "Child", "text": "Early bedtime"},
        {"role": "Narrator", "text": "Casey slept well and felt great the next day. Should Casey tell friends about the fairy or keep it a secret?"}
    ],

    # Story 56: The Homework Helper
    [
        {"role": "Child", "text": "Cheating"},
        {"role": "Narrator", "text": "Taylor found a magic pencil that wrote correct answers. Use it on the test or study hard?"},
        {"role": "Child", "text": "Use it"},
        {"role": "Narrator", "text": "Taylor aced the test but felt guilty. The teacher suspected cheating. Confess or deny it?"},
        {"role": "Child", "text": "Confess"},
        {"role": "Narrator", "text": "The teacher gave Taylor a chance to retake the test. Study with friends or alone?"}
    ],

    # Story 57: The Golden Rule
    [
        {"role": "Child", "text": "Bullying"},
        {"role": "Narrator", "text": "Jordan was often bullied and finally had a chance to bully someone else. Do it or be kind instead?"},
        {"role": "Child", "text": "Be kind"},
        {"role": "Narrator", "text": "Jordan's kindness made a new friend. Together, they could stand up to bullies or tell a teacher. What should they do?"},
        {"role": "Child", "text": "Tell a teacher"},
        {"role": "Narrator", "text": "The teacher started an anti-bullying program. Should Jordan lead it or support from the sidelines?"}
    ],

    # Story 58: The Screen Time Struggle
    [
        {"role": "Child", "text": "Technology"},
        {"role": "Narrator", "text": "Riley loved video games and played all day. Go outside to play or keep gaming?"},
        {"role": "Child", "text": "Keep gaming"},
        {"role": "Narrator", "text": "Riley's eyes hurt and missed the family picnic. A wise owl appeared, offering limited gaming time or new outdoor games. Choose:"},
        {"role": "Child", "text": "Outdoor games"},
        {"role": "Narrator", "text": "Riley had fun outside and felt healthier. Start a neighborhood play group or stick to solo outdoor activities?"}
    ],

    # Story 59: The Littering Lesson
    [
        {"role": "Child", "text": "Environment"},
        {"role": "Narrator", "text": "Charlie finished a snack and saw no trash can nearby. Litter or carry the wrapper home?"},
        {"role": "Child", "text": "Litter"},
        {"role": "Narrator", "text": "The wrapper turned into a sad flower. It offered Charlie a chance to clean the park or ignore the litter. What to do?"},
        {"role": "Child", "text": "Clean the park"},
        {"role": "Narrator", "text": "The park became beautiful. Should Charlie organize a weekly cleanup or remind friends not to litter?"}
    ],

    # Story 60: The Patience Puzzle
    [
        {"role": "Child", "text": "Waiting"},
        {"role": "Narrator", "text": "Quinn wanted a toy but had to wait for a birthday. Beg parents to buy it now or wait patiently?"},
        {"role": "Child", "text": "Beg parents"},
        {"role": "Narrator", "text": "Parents said no, making Quinn upset. A patience fairy appeared, offering instant gratification or a lesson in waiting. Choose:"},
        {"role": "Child", "text": "Lesson in waiting"},
        {"role": "Narrator", "text": "Quinn learned patience and appreciated the toy more. Share the lesson with friends or keep the secret?"}
    ],

    # Story 61: The Friendship Test
    [
        {"role": "Child", "text": "Loyalty"},
        {"role": "Narrator", "text": "Avery's best friend was blamed for something Avery did. Tell the truth or let the friend take the blame?"},
        {"role": "Child", "text": "Let friend take blame"},
        {"role": "Narrator", "text": "The friend was punished and Avery felt terrible. A truth spirit appeared, offering a chance to confess or forget it happened. Choose:"},
        {"role": "Child", "text": "Confess"},
        {"role": "Narrator", "text": "Avery's honesty saved the friendship. Work together to make amends or move on?"}
    ],

    # Story 62: The Responsibility Lesson
    [
        {"role": "Child", "text": "Pet"},
        {"role": "Narrator", "text": "Morgan got a new puppy but forgot to feed it. The puppy looked hungry. Feed it or play first?"},
        {"role": "Child", "text": "Play first"},
        {"role": "Narrator", "text": "The puppy became sad and weak. A wise cat appeared, offering to take care of the puppy or teach Morgan responsibility. Choose:"},
        {"role": "Child", "text": "Learn responsibility"},
        {"role": "Narrator", "text": "Morgan learned to care for the puppy properly. Create a pet care chart or join a young pet owners club?"}
    ],

    # Story 63: The Gossip Gremlin
    [
        {"role": "Child", "text": "Rumors"},
        {"role": "Narrator", "text": "Jessie heard a rumor about a classmate. Spread it or keep it to yourself?"},
        {"role": "Child", "text": "Spread it"},
        {"role": "Narrator", "text": "The rumor hurt the classmate's feelings. A truth fairy appeared, offering to undo the rumor or teach Jessie about kindness. Choose:"},
        {"role": "Child", "text": "Undo the rumor"},
        {"role": "Narrator", "text": "The rumor was gone, but Jessie learned a lesson. Apologize to the classmate or start an anti-gossip campaign?"}
    ],

    # Story 64: The Recycling Robot
    [
        {"role": "Child", "text": "Trash"},
        {"role": "Narrator", "text": "Finn found a robot that turned trash into toys. Keep it a secret or share with the community?"},
        {"role": "Child", "text": "Keep it secret"},
        {"role": "Narrator", "text": "Finn's room was full of toys, but the town was still littered. The robot offered to clean the town or make more toys. Choose:"},
        {"role": "Child", "text": "Clean the town"},
        {"role": "Narrator", "text": "The town became clean and beautiful. Start a recycling club or teach others about the robot?"}
    ],

    # Story 65: The Teamwork Challenge
    [
        {"role": "Child", "text": "Cooperation"},
        {"role": "Narrator", "text": "Rory's class had a group project. Do all the work alone or collaborate with classmates?"},
        {"role": "Child", "text": "Do it alone"},
        {"role": "Narrator", "text": "Rory struggled and the project was incomplete. A teamwork sprite appeared, offering to finish the project or teach about cooperation. Choose:"},
        {"role": "Child", "text": "Learn cooperation"},
        {"role": "Narrator", "text": "Rory learned to work with others and the project succeeded. Become a team leader or support team members?"}
    ],

    # Story 66: The Healthy Choice
    [
        {"role": "Child", "text": "Exercise"},
        {"role": "Narrator", "text": "Bailey loved video games and rarely exercised. Join a sports team or continue gaming?"},
        {"role": "Child", "text": "Continue gaming"},
        {"role": "Narrator", "text": "Bailey felt tired and unwell. A health hero appeared, offering energy for more gaming or a fun exercise routine. Choose:"},
        {"role": "Child", "text": "Fun exercise"},
        {"role": "Narrator", "text": "Bailey felt better and made new friends. Create a neighborhood sports league or stick to personal fitness?"}
    ],

    # Story 67: The Empathy Experiment
    [
        {"role": "Child", "text": "Understanding"},
        {"role": "Narrator", "text": "Skyler laughed at a classmate's mistake. The classmate cried. Apologize or walk away?"},
        {"role": "Child", "text": "Walk away"},
        {"role": "Narrator", "text": "Skyler felt bad later. An empathy elf appeared, offering to erase the memory or teach about feelings. Choose:"},
        {"role": "Child", "text": "Learn about feelings"},
        {"role": "Narrator", "text": "Skyler understood others better. Start a kindness campaign or privately practice empathy?"}
    ],

    # Story 68: The Honesty Shop
    [
        {"role": "Child", "text": "Money"},
        {"role": "Narrator", "text": "Peyton found extra change from the Honesty Shop. Keep it or return it?"},
        {"role": "Child", "text": "Keep it"},
        {"role": "Narrator", "text": "The shop's mascot, an honest hare, appeared. It offered Peyton more money to keep or a lesson in honesty. Choose:"},
        {"role": "Child", "text": "Lesson in honesty"},
        {"role": "Narrator", "text": "Peyton learned about integrity and returned the money. Volunteer at the shop or spread the word about honesty?"}
    ],

    # Story 69: The Inclusion Invitation
    [
        {"role": "Child", "text": "Friendship"},
        {"role": "Narrator", "text": "Harper saw a lonely new student. Invite them to play or stick with current friends?"},
        {"role": "Child", "text": "Stick with current friends"},
        {"role": "Narrator", "text": "The new student looked sad. A friendship fairy appeared, offering a new best friend or a chance to be inclusive. Choose:"},
        {"role": "Child", "text": "Be inclusive"},
        {"role": "Narrator", "text": "Harper made everyone feel welcome. Start a club for new students or encourage friends to be more inclusive?"}
    ],

    # Story 70: The Digital Dilemma
    [
        {"role": "Child", "text": "Internet"},
        {"role": "Narrator", "text": "River found a way to download movies for free. Do it or pay for them?"},
        {"role": "Child", "text": "Download for free"},
        {"role": "Narrator", "text": "River's computer got a virus. A digital detective appeared, offering to fix it for free or teach about internet safety. Choose:"},
        {"role": "Child", "text": "Learn internet safety"},
        {"role": "Narrator", "text": "River learned to use the internet responsibly. Create an online safety guide or teach friends about digital citizenship?"}
    ],

    # Story 71: The Truthful Tree
    [
        {"role": "Child", "text": "Lying",},
        {"role": "Narrator", "text": "You find a magical tree that can tell when people are lying. It offers you truth-telling apples or fibbing figs. Which do you pick?",},
        {"role": "Child", "text": "What's fibbing?",},
        {"role": "Narrator", "text": "Fibbing is another word for telling small lies. Truth apples make you always tell the truth, fibbing figs let you lie. Which fruit?",},
        {"role": "Child", "text": "Let's try the truth-telling apples!",},
        {"role": "Narrator", "text": "After eating the apple, you always tell the truth. Your friend asks if you like their new hat, but you don't. Do you tell the truth or stay silent?",},
        {"role": "Child", "text": "What happens if I stay silent?",},
        {"role": "Narrator", "text": "Staying silent might not hurt feelings, but it's not the whole truth. Telling the truth might be hard but honest. Truth or silence?",},
        {"role": "Child", "text": "I'll tell the truth gently.",},
        {"role": "Narrator", "text": "Your friend appreciates your honesty. The tree offers you seeds to grow your own truth tree or magic glasses to see others' true feelings. Which do you choose?",},
    ],

    # Story 72: The Enchanted Orchestra
    [
        {"role": "Child", "text": "Can you tell a story about magical music?",},
        {"role": "Narrator", "text": "You find an enchanted concert hall where instruments play themselves. A violin and a drum set invite you to join. Which do you pick?",},
        {"role": "Child", "text": "What's a violin?",},
        {"role": "Narrator", "text": "A violin is a string instrument played with a bow. It makes beautiful melodies. Violin or drums?",},
        {"role": "Child", "text": "Let's try the violin!",},
        {"role": "Narrator", "text": "The violin teaches you a song that can make flowers grow or animals dance. Which magic do you want to try?",},
        {"role": "Child", "text": "What kind of animals will dance?",},
        {"role": "Narrator", "text": "All kinds! Rabbits might hop, birds could swoop, and bears could waltz. Grow flowers or make animals dance?",},
        {"role": "Child", "text": "Let's make the animals dance!",},
        {"role": "Narrator", "text": "The animals dance joyfully! Now they want to join your orchestra. Do you add singing birds or drumming elephants?",},
    ],

    # Story 73: The Dragon's Dilemma
    [
        {"role": "Child", "text": "Dragon",},
        {"role": "Narrator", "text": "You meet a friendly dragon who can't breathe fire. It asks for your help to either learn fire-breathing or find a new talent. What do you suggest?",},
        {"role": "Child", "text": "What's a talent?",},
        {"role": "Narrator", "text": "A talent is something you're naturally good at or can learn to do well. Help with fire-breathing or find a new talent?",},
        {"role": "Child", "text": "Let's find a new talent!",},
        {"role": "Narrator", "text": "The dragon discovers it can change colors like a chameleon or create beautiful ice sculptures. Which talent should it develop?",},
        {"role": "Child", "text": "What's a chameleon?",},
        {"role": "Narrator", "text": "A chameleon is a lizard that can change its color to blend in with its surroundings. Color-changing or ice sculpting?",},
        {"role": "Child", "text": "Color-changing sounds amazing!",},
        {"role": "Narrator", "text": "The dragon becomes a master of colors! Should it use its talent to create dazzling light shows or become a camouflage teacher for other magical creatures?",},
    ],

    # Story 74: The Whispering Woods
    [
        {"role": "Child", "text": "Forest",},
        {"role": "Narrator", "text": "You enter a magical forest where trees whisper secrets. Do you want to listen to the oak tree's ancient tales or the willow's future predictions?",},
        {"role": "Child", "text": "What's a willow?",},
        {"role": "Narrator", "text": "A willow is a tree with long, drooping branches that look like curtains. Oak for past stories or willow for future ones?",},
        {"role": "Child", "text": "Let's hear the willow's predictions!",},
        {"role": "Narrator", "text": "The willow shows you two possible futures: one where you become a nature guardian or one where you explore outer space. Which interests you more?",},
        {"role": "Child", "text": "What's a guardian?",},
        {"role": "Narrator", "text": "A guardian is someone who protects and takes care of something important. Nature guardian or space explorer?",},
        {"role": "Child", "text": "I want to be a nature guardian!",},
        {"role": "Narrator", "text": "As a nature guardian, you can choose to protect endangered animals or restore damaged forests. Which mission do you accept?",},
    ],

    # Story 75: The Bubble Bazaar
    [
        {"role": "Child", "text": "Tell me about a magical market!",},
        {"role": "Narrator", "text": "You discover a floating market inside giant bubbles! You can visit a shop selling bottled emotions or one with time-travel trinkets. Where do you go?",},
        {"role": "Child", "text": "What are trinkets?",},
        {"role": "Narrator", "text": "Trinkets are small, interesting objects or ornaments. Emotion bottles or time-travel trinkets?",},
        {"role": "Child", "text": "Time-travel trinkets sound cool!",},
        {"role": "Narrator", "text": "In the time-travel shop, you find a watch that slows down time and shoes that let you jump to yesterday. Which do you try?",},
        {"role": "Child", "text": "What happens if I jump to yesterday?",},
        {"role": "Narrator", "text": "You could relive yesterday and maybe change things. The watch lets you have more time in a day. Shoes or watch?",},
        {"role": "Child", "text": "I'll try the time-slowing watch!",},
        {"role": "Narrator", "text": "With slowed time, you can learn super-fast or have a long adventure before dinner. What do you do with your extra time?",},
    ],

    # Story 76: The Melodic Mountain
    [
        {"role": "Child", "text": "Music",},
        {"role": "Narrator", "text": "You climb a mountain that plays music as you ascend. You can follow the path of lively drumbeats or soothing flute melodies. Which way do you go?",},
        {"role": "Child", "text": "What's a flute?",},
        {"role": "Narrator", "text": "A flute is a wind instrument that makes soft, high sounds. Energetic drums or calm flute?",},
        {"role": "Child", "text": "Let's follow the flute music!",},
        {"role": "Narrator", "text": "The flute path leads to a garden where flowers sing. You can join their chorus or learn to conduct their plant orchestra. What do you choose?",},
        {"role": "Child", "text": "What does conduct mean?",},
        {"role": "Narrator", "text": "To conduct means to direct a group of musicians. Will you sing with the flowers or lead their performance?",},
        {"role": "Child", "text": "I want to conduct the plant orchestra!",},
        {"role": "Narrator", "text": "As the plant conductor, you can compose a song to make it rain or one to bring eternal sunshine. Which song will you create?",},
    ],

    # Story 77: The Starlight Sanctuary
    [
        {"role": "Child", "text": "Can you tell me about a place with lots of stars?",},
        {"role": "Narrator", "text": "You find a magical observatory where stars come to rest. You can stargaze through a telescope or take a ride on a shooting star. What's your choice?",},
        {"role": "Child", "text": "What's an observatory?",},
        {"role": "Narrator", "text": "An observatory is a place used to watch the stars and planets. Telescope or shooting star ride?",},
        {"role": "Child", "text": "Riding a shooting star sounds exciting!",},
        {"role": "Narrator", "text": "On your star ride, you can visit a moon made of cheese or a planet of friendly aliens. Where do you want to go?",},
        {"role": "Child", "text": "What kind of aliens?",},
        {"role": "Narrator", "text": "These aliens are curious and kind, with color-changing skin. Cheese moon or alien planet?",},
        {"role": "Child", "text": "Let's meet the aliens!",},
        {"role": "Narrator", "text": "The aliens invite you to join their zero-gravity dance party or help them build a rainbow bridge to other planets. What do you do?",},
    ],

    # Story 78: The Enchanted Toyshop
    [
        {"role": "Child", "text": "Toys",},
        {"role": "Narrator", "text": "You enter a magical toy shop where toys come to life at night. A teddy bear invites you to a tea party, and a toy robot wants to play chess. Who do you join?",},
        {"role": "Child", "text": "What's chess?",},
        {"role": "Narrator", "text": "Chess is a thinking game with different pieces on a board. The teddy's tea party would be cozy. Chess or tea party?",},
        {"role": "Child", "text": "The tea party sounds nice!",},
        {"role": "Narrator", "text": "At the tea party, you can drink magic tea that makes you tiny or giant. Which size do you want to be?",},
        {"role": "Child", "text": "What would happen if I'm tiny?",},
        {"role": "Narrator", "text": "If you're tiny, you could explore doll houses or ride toy cars. If giant, you could reach high shelves. Tiny or giant?",},
        {"role": "Child", "text": "Let's be tiny!",},
        {"role": "Narrator", "text": "Now tiny, you can drive a toy car in a race or explore a dollhouse mansion. Which adventure do you choose?",},
    ],

    # Story 79: The Cloud Classroom
    [
        {"role": "Child", "text": "Tell me about a school in the sky!",},
        {"role": "Narrator", "text": "You discover a school floating on clouds! You can take a class in rainbow making or star catching. Which do you choose?",},
        {"role": "Child", "text": "How do you catch stars?",},
        {"role": "Narrator", "text": "In this magical school, you use special nets to catch shooting stars. Rainbow making uses color magic. Stars or rainbows?",},
        {"role": "Child", "text": "Rainbow making sounds colorful!",},
        {"role": "Narrator", "text": "In rainbow class, you learn to mix sky colors. You can create a rainbow slide or paint the clouds. What do you want to do?",},
        {"role": "Child", "text": "What's a rainbow slide?",},
        {"role": "Narrator", "text": "A rainbow slide is a fun, colorful slide made from a real rainbow. Make a slide or paint clouds?",},
        {"role": "Child", "text": "Let's make a rainbow slide!",},
        {"role": "Narrator", "text": "Your rainbow slide is beautiful! Do you want to ride it down to a flower field or to a beach paradise?",},
    ],

    # Story 80: The Moonlight Meadow
    [
        {"role": "Child", "text": "Moon",},
        {"role": "Narrator", "text": "You find a meadow that only appears under the full moon. Glowing flowers offer you emotional wisdom, while shimmering grass grants wishes. Which do you approach?",},
        {"role": "Child", "text": "What's wisdom?",},
        {"role": "Narrator", "text": "Wisdom is knowledge gained from experience, helping you make good choices. Emotional wisdom or wish-granting?",},
        {"role": "Child", "text": "I'd like to make a wish!",},
        {"role": "Narrator", "text": "The grass allows one wish. You can wish for the ability to talk to animals or the power to grow plants instantly. What's your wish?",},
        {"role": "Child", "text": "What kinds of plants?",},
        {"role": "Narrator", "text": "Any kind of plant - flowers, trees, even magical ones! Talk to animals or grow plants?",},
        {"role": "Child", "text": "Growing plants sounds fun!",},
        {"role": "Narrator", "text": "You can now grow plants magically! Will you create a treehouse village or an enchanted garden maze?",},
    ],

    # Story 81: The Undersea Library
    [
        {"role": "Child", "text": "Can you tell me about reading underwater?",},
        {"role": "Narrator", "text": "You discover a library deep in the ocean where books are written in bubble letters. You can read about mermaid magic or sunken treasure maps. Which book do you choose?",},
        {"role": "Child", "text": "What are bubble letters?",},
        {"role": "Narrator", "text": "Bubble letters are words formed by tiny bubbles in the water. Mermaid magic or treasure maps?",},
        {"role": "Child", "text": "Mermaid magic sounds interesting!",},
        {"role": "Narrator", "text": "The mermaid book teaches you to breathe underwater or speak to sea creatures. Which skill do you want to learn?",},
        {"role": "Child", "text": "Can I do both?",},
        {"role": "Narrator", "text": "In this story, you can only learn one skill at a time. Breathe underwater or talk to sea animals?",},
        {"role": "Child", "text": "I want to talk to sea animals!",},
        {"role": "Narrator", "text": "You can now speak to sea creatures! Do you want to chat with a wise octopus or play with playful dolphins?",},
    ],

    # Story 82: The Time-Twisting Train
    [
        {"role": "Child", "text": "Train",},
        {"role": "Narrator", "text": "You board a magical train that can travel through time. You can visit the age of dinosaurs or the far future. Where do you want to go?",},
        {"role": "Child", "text": "What's in the far future?",},
        {"role": "Narrator", "text": "The future might have flying cars and robot helpers. Dinosaur times have giant reptiles. Future or dinosaurs?",},
        {"role": "Child", "text": "Let's see the dinosaurs!",},
        {"role": "Narrator", "text": "In dinosaur times, you can ride a friendly Triceratops or watch T-Rex hunt. What do you want to do?",},
        {"role": "Child", "text": "What's a Triceratops?",},
        {"role": "Narrator", "text": "A Triceratops is a dinosaur with three horns and a big frill on its head. Ride Triceratops or watch T-Rex?",},
        {"role": "Child", "text": "Riding a Triceratops sounds cool!",},
        {"role": "Narrator", "text": "While riding, you find a nest of dinosaur eggs. Do you want to watch them hatch or continue exploring?",},
    ],

    # Story 83: The Kaleidoscope Kingdom
    [
        {"role": "Child", "text": "Colors",},
        {"role": "Narrator", "text": "You enter a kingdom where everything constantly changes colors. You can wear a mood-changing cloak or glasses that let you control the colors around you. Which do you choose?",},
        {"role": "Child", "text": "What's a kaleidoscope?",},
        {"role": "Narrator", "text": "A kaleidoscope is a tube you look through to see beautiful, changing patterns of colors. Mood cloak or color-control glasses?",},
        {"role": "Child", "text": "The color-control glasses sound fun!",},
        {"role": "Narrator", "text": "With your glasses, you can paint the town your favorite color or create a rainbow road. What will you do?",},
        {"role": "Child", "text": "What's my favorite color?",},
        {"role": "Narrator", "text": "In this story, you can choose any color you like best! Paint the town or make a rainbow road?",},
        {"role": "Child", "text": "Let's make a rainbow road!",},
        {"role": "Narrator", "text": "Your rainbow road is beautiful! It can lead to a crystal cave or a cloud castle. Where do you want it to go?",},
    ],

    # Story 84: The Whisper Woods
    [
        {"role": "Child", "text": "Tell me about a quiet forest.",},
        {"role": "Narrator", "text": "You enter a forest where everyone must whisper. You meet a soft-spoken fairy and a silent tree sprite. Who do you approach?",},
        {"role": "Child", "text": "What's a sprite?",},
        {"role": "Narrator", "text": "A sprite is a magical forest creature, like a tiny, nature-loving fairy. Soft-spoken fairy or silent sprite?",},
        {"role": "Child", "text": "Let's talk to the fairy!",},
        {"role": "Narrator", "text": "The fairy offers to teach you the language of flowers or how to walk silently like a forest animal. Which do you learn?",},
        {"role": "Child", "text": "How do flowers talk?",},
        {"role": "Narrator", "text": "In this magical forest, flowers communicate through their scents and colors. Learn flower language or silent walking?",},
        {"role": "Child", "text": "Flower language sounds interesting!",},
        {"role": "Narrator", "text": "You learn to understand flowers! They tell you about a hidden glade with wish-granting mushrooms or a magical stream that sings lullabies. Where do you go?",},
    ],

    # Story 85: The Bubble Bazaar
    [
        {"role": "Child", "text": "Bubbles",},
        {"role": "Narrator", "text": "You visit a market where everything is made of bubbles! You can buy a bubble pet or a bubble vehicle. Which do you want?",},
        {"role": "Child", "text": "What's a bazaar?",},
        {"role": "Narrator", "text": "A bazaar is a market with many different things to buy. Bubble pet or bubble vehicle?",},
        {"role": "Child", "text": "A bubble pet sounds cute!",},
        {"role": "Narrator", "text": "Your bubble pet can be a floating puppy or a bouncing kitten. Which one do you choose?",},
        {"role": "Child", "text": "Can the pet pop?",},
        {"role": "Narrator", "text": "These magic bubble pets don't pop! They're soft and cuddly. Floating puppy or bouncing kitten?",},
        {"role": "Child", "text": "I'll take the floating puppy!",},
        {"role": "Narrator", "text": "Your bubble puppy floats beside you! Do you want to take it to the bubble park or the bubble school?",},
    ],

    # Story 86: The Laughing Lake

    [
        {"role": "Child", "text": "Lake",},
        {"role": "Narrator", "text": "You find a lake that giggles when touched. Do you swim in it or build a boat?",},
        {"role": "Child", "text": "What's giggling?",},
        {"role": "Narrator", "text": "Giggling is light, happy laughter. Swim or build a boat?",},
        {"role": "Child", "text": "Let's swim!",},
        {"role": "Narrator", "text": "The lake's laughter makes you float. Do you race water fairies or dive for laughing pearls?",},
    ],

    # Story 87: The Clockwork Castle

    [
        {"role": "Child", "text": "Castle",},
        {"role": "Narrator", "text": "You enter a castle made of gears and springs. Do you wind up the knight or the dragon?",},
        {"role": "Child", "text": "What's clockwork?",},
        {"role": "Narrator", "text": "Clockwork means powered by wound-up springs. Knight or dragon?",},
        {"role": "Child", "text": "Dragon!",},
        {"role": "Narrator", "text": "The dragon flies you to a gear garden or a spring meadow. Where to?",},
    ],

    # Story 88: The Origami Ocean

    [
        {"role": "Child", "text": "Paper",},
        {"role": "Narrator", "text": "You sail on a paper sea. Fold a whale to ride or a boat to captain?",},
        {"role": "Child", "text": "What's origami?",},
        {"role": "Narrator", "text": "Origami is the art of paper folding. Whale or boat?",},
        {"role": "Child", "text": "Let's fold a whale!",},
        {"role": "Narrator", "text": "Your paper whale swims to an island of books or a lighthouse of scrolls. Which?",},
    ],

    # Story 89: The Melody Mountains

    [
        {"role": "Child", "text": "Music",},
        {"role": "Narrator", "text": "Mountains play music as you climb. Follow the flute path or drum trail?",},
        {"role": "Child", "text": "What's a flute?",},
        {"role": "Narrator", "text": "A flute is a wind instrument with a soft sound. Flute or drums?",},
        {"role": "Child", "text": "Flute path!",},
        {"role": "Narrator", "text": "The flute leads to singing flowers or dancing trees. Visit which?",},
    ],

    # Story 90: The Bubblegum Bridge

    [
        {"role": "Child", "text": "Candy",},
        {"role": "Narrator", "text": "A sticky bridge stretches over a chocolate river. Cross it or swim?",},
        {"role": "Child", "text": "Is it real bubblegum?",},
        {"role": "Narrator", "text": "Yes, it's magical, strong bubblegum. Cross or swim?",},
        {"role": "Child", "text": "Let's cross the bridge!",},
        {"role": "Narrator", "text": "You reach a lollipop forest or cotton candy clouds. Where to?",},
    ],

    # Story 91: The Whispering Winds

    [
        {"role": "Child", "text": "Secrets",},
        {"role": "Narrator", "text": "Winds whisper secrets. Listen to past stories or future predictions?",},
        {"role": "Child", "text": "What kind of secrets?",},
        {"role": "Narrator", "text": "Magical secrets of the world. Past or future?",},
        {"role": "Child", "text": "Future, please!",},
        {"role": "Narrator", "text": "You learn of flying cars or underwater cities. Which excites you?",},
    ],

    # Story 92: The Chameleon Carnival

    [
        {"role": "Child", "text": "Colors",},
        {"role": "Narrator", "text": "A carnival where everything changes color. Ride the mood swing or the emotion coaster?",},
        {"role": "Child", "text": "What's a chameleon?",},
        {"role": "Narrator", "text": "An animal that changes color. Mood swing or emotion coaster?",},
        {"role": "Child", "text": "Emotion coaster sounds fun!",},
        {"role": "Narrator", "text": "After the ride, join the color parade or the rainbow race?",},
    ],

    # Story 93: The Starlight Stage

    [
        {"role": "Child", "text": "Performance",},
        {"role": "Narrator", "text": "Stars invite you to perform. Sing a constellation into being or dance a meteor shower?",},
        {"role": "Child", "text": "What's a constellation?",},
        {"role": "Narrator", "text": "A group of stars forming a pattern. Sing or dance?",},
        {"role": "Child", "text": "I'll sing!",},
        {"role": "Narrator", "text": "Your song creates a bear or a dragon in the sky. Which?",},
    ],

    # Story 94: The Bouncing Bog

    [
        {"role": "Child", "text": "Jump",},
        {"role": "Narrator", "text": "You find a marshmallow-like bog. Bounce across or sink and explore underneath?",},
        {"role": "Child", "text": "Is it a real bog?",},
        {"role": "Narrator", "text": "It's a magical, soft bog. Bounce or explore underneath?",},
        {"role": "Child", "text": "Let's bounce across!",},
        {"role": "Narrator", "text": "You reach a rubbery forest or a springy mountain. Where to?",},
    ],

    # Story 95: The Telepathic Tea Party

    [
        {"role": "Child", "text": "Mind-reading",},
        {"role": "Narrator", "text": "Animals invite you to a mind-reading tea party. Chat with wise owl or playful fox?",},
        {"role": "Child", "text": "What's telepathic?",},
        {"role": "Narrator", "text": "Telepathic means communicating with thoughts. Owl or fox?",},
        {"role": "Child", "text": "Let's talk to the owl!",},
        {"role": "Narrator", "text": "Owl shares knowledge of flight or forest secrets. Which interests you?",},
    ],

    # Story 96: The Echo Cave

    [
        {"role": "Child", "text": "Cave",},
        {"role": "Narrator", "text": "Your echoes come to life in this cave. Shout a brave word or a kind one?",},
        {"role": "Child", "text": "What will happen?",},
        {"role": "Narrator", "text": "Your word will create a magical helper. Brave or kind?",},
        {"role": "Child", "text": "I choose a kind word!",},
        {"role": "Narrator", "text": "A friendly spirit appears. Explore deeper or return to teach kindness?",},
    ],

    # Story 97: The Upside-Down House

    [
        {"role": "Child", "text": "Silly",},
        {"role": "Narrator", "text": "You enter a house where everything's upside-down. Walk on the ceiling or swim in the up-flowing sink?",},
        {"role": "Child", "text": "Can we really walk on ceilings?",},
        {"role": "Narrator", "text": "In this magical house, yes! Ceiling-walk or sink-swim?",},
        {"role": "Child", "text": "Let's walk on the ceiling!",},
        {"role": "Narrator", "text": "You find an upside-down garden or a reverse library. Which to visit?",},
    ],

    # Story 98: The Puzzle Planet

    [
        {"role": "Child", "text": "Space",},
        {"role": "Narrator", "text": "You land on a planet of living puzzles. Solve the riddle mountains or the mystery mazes?",},
        {"role": "Child", "text": "What's a riddle?",},
        {"role": "Narrator", "text": "A riddle is a tricky question with a clever answer. Mountains or mazes?",},
        {"role": "Child", "text": "Riddle mountains!",},
        {"role": "Narrator", "text": "You solve it! Receive the gift of flight or ability to shrink?",},
    ],

    # Story 99: The Dream Diner

    [
        {"role": "Child", "text": "Food",},
        {"role": "Narrator", "text": "At this diner, you can eat your dreams. Order sweet dreams or adventurous ones?",},
        {"role": "Child", "text": "What are sweet dreams?",},
        {"role": "Narrator", "text": "Sweet dreams are happy, peaceful dreams. Sweet or adventurous?",},
        {"role": "Child", "text": "Adventurous dreams, please!",},
        {"role": "Narrator", "text": "Your dreams take you to a cloud race or deep-sea exploration. Which?",},
    ],

    # Story 100: The Wishing Well

    [
        {"role": "Child", "text": "Wish",},
        {"role": "Narrator", "text": "A magical well grants wishes. Wish for ability to fly or to talk to animals?",},
        {"role": "Child", "text": "What's a wishing well?",},
        {"role": "Narrator", "text": "It's a well where wishes come true. Fly or talk to animals?",},
        {"role": "Child", "text": "I wish to talk to animals!",},
        {"role": "Narrator", "text": "You can now talk to animals! Chat with forest creatures or sea life?",},
    ],

    # Story 101: The Clockwork Forest

    [
        {"role": "Child", "text": "Trees",},
        {"role": "Narrator", "text": "You stumble upon a forest where the trees are made of gears and springs. The leaves chime like tiny bells in the breeze, and the branches tick-tock in rhythm. As you walk deeper into this mechanical marvel, you come across two paths: one leads to a clearing with a giant hourglass that controls the seasons, and the other winds its way to a workshop where robotic animals are crafted. The hourglass path glimmers with golden sand, while the workshop path clanks with the sounds of friendly robotic creatures. Which path intrigues you more?",},
        {"role": "Child", "text": "What's an hourglass?",},
        {"role": "Narrator", "text": "An hourglass is a device used to measure time. It has two glass bulbs connected by a narrow neck. Sand flows from the upper bulb to the lower bulb, usually taking an hour to empty – hence the name 'hourglass'. In this magical forest, the giant hourglass doesn't just measure time, but actually controls the changing of seasons! Imagine being able to flip it and watch as spring turns to summer, or winter to fall in a matter of moments. It's a powerful and wondrous object. Now, knowing this, would you like to explore the hourglass clearing or the robotic animal workshop?",},
        {"role": "Child", "text": "Let's go see the hourglass!",},
        {"role": "Narrator", "text": "As you approach the clearing, the giant hourglass comes into view. It's as tall as a house, with shimmering sand that seems to change color as it falls. Each grain represents a day, and you can see the seasons changing in the forest around you as the sand trickles down. Suddenly, a clockwork fairy appears. Her wings are delicate gears that whir softly as she hovers before you. She offers you a choice: you can either turn the hourglass to experience a season that isn't currently happening in the real world, or you can add a single grain of sand to the hourglass, creating a day that's entirely your own where anything is possible. The fairy explains that turning the hourglass will let you explore a different season and all its wonders, while creating your own day would allow you to shape a unique 24 hours filled with whatever adventures you can imagine. Both choices sound exciting and magical. Which would you prefer: to turn the hourglass and explore a different season, or to add your own grain of sand and create a day of your dreams?",},
        {"role": "Child", "text": "I want to make my own day!",},
        {"role": "Narrator", "text": "The fairy smiles and hands you a glowing grain of sand. As you drop it into the hourglass, the world around you shimmers and changes. You now have 24 hours to fill with whatever your heart desires! The clockwork forest is yours to command. You could have the mechanical trees grow candy instead of leaves, or have the robotic animals organize a grand parade. You might even be able to make it rain stardust or have flowers bloom that sing your favorite songs. The fairy explains that you have two exciting options to start your day: you can either conduct an orchestra of musical flowers that will play any tune you imagine, or you can ride on the back of a giant, mechanical butterfly that will take you anywhere in this magical world. The flower orchestra would fill the forest with beautiful music of your creation, while the butterfly ride would let you explore every nook and cranny of this wondrous place from above. Both choices promise a magical start to your special day. Will you begin by conducting the flower orchestra or by taking a scenic flight on the mechanical butterfly?",},
    ],

    # Story 102: The Bubblegum Balloon Race

    [
        {"role": "Child", "text": "Bubbles",},
        {"role": "Narrator", "text": "Welcome to the annual Bubblegum Balloon Race! The sky is filled with colorful, giant bubbles made from magical bubblegum. Each bubble is big enough to carry a person, and racers from all over the magical world have come to compete. The bubbles float gently in the breeze, their surfaces shimmering with rainbow hues. As a special guest, you've been invited to participate in the race! The starting line is a long, sticky ribbon of stretched bubblegum, and the finish line is a cloud made of cotton candy in the distance. Two friendly racers approach you, each offering to be your partner in the race. The first is Pip, a giggling pixie with wings that sparkle like dewdrops. Pip is known for being fast but sometimes gets distracted by shiny things. The other is Grum, a grumpy but goodhearted gnome with a long, fluffy beard. Grum is slower but has years of experience and knows all the wind currents. Who would you like as your racing partner?",},
        {"role": "Child", "text": "What's a gnome?",},
        {"role": "Narrator", "text": "A gnome is a mythical creature that's often described in stories and fairy tales. They're usually portrayed as small, elderly-looking beings with big beards who live underground or in gardens. In many tales, gnomes are known for their wisdom, their connection to the earth, and their skillful craftsmanship. They're often seen as guardians of nature and keepers of secret knowledge. In our story, Grum the gnome brings all of this experience and earth-wisdom to the sky race, making him a knowledgeable, if unusual, choice for a flying competition. Despite his grumpy exterior, he's kind-hearted and very clever. Now, knowing what a gnome is, would you prefer to race with Pip the quick but distractible pixie, or Grum the experienced but slower gnome?",},
        {"role": "Child", "text": "I choose Grum!",},
        {"role": "Narrator", "text": "Excellent choice! Grum's bushy eyebrows raise in surprise, and then he gives you a warm smile from beneath his beard. 'Right then,' he says, 'let's show these whippersnappers how it's done!' He helps you into a large, sturdy-looking bubble that smells like grape bubblegum. As you settle in, Grum explains that you have two possible race strategies. The first is to take the high route, soaring above the other racers where the wind is stronger. This path is faster but requires careful navigation around sky-obstacles like ticklish clouds that cause sneezing fits and floating islands of upside-down ice cream. The second option is to take the low route, weaving through a forest of giant lollipop trees. This way is slower but filled with power-ups like fizzy lifting drinks that make your bubble zoom ahead, and bubblegum powerberries that let you blow small bubbles to distract other racers. Grum assures you that both routes have their advantages, and he's confident in navigating either one. The race is about to start! Will you choose the high route with its speed and sky obstacles, or the low route with its power-ups and weaving challenges?",},
        {"role": "Child", "text": "Let's take the high route!",},
        {"role": "Narrator", "text": "With a nod, Grum steers your grape-scented bubble high into the sky. The wind whistles past as you soar above the other racers, their bubbles looking like colorful marbles far below. You're making good time, but soon you encounter your first sky-obstacle: a giant, fluffy cloud that's giggling uncontrollably. 'That there's a tickle cloud,' Grum warns. 'If we touch it, we'll laugh so hard we might pop our bubble!' Just as you're figuring out how to navigate around it, you spot a shortcut. There's a narrow tunnel of calm air passing right through the center of the tickle cloud. It would save you a lot of time, but one wrong move and you'll end up in a fit of laughter. Grum strokes his beard thoughtfully. 'It's your call, captain,' he says. 'We can take it slow and safe around the cloud, or risk the shortcut for a chance at first place.' Your grape bubble hovers as you consider the options. Will you play it safe and go around the tickle cloud, or take the risky shortcut through its center?",},
    ],

    # Story 103: The Melody Maze

    [
        {"role": "Child", "text": "Music",},
        {"role": "Narrator", "text": "You find yourself at the entrance of an enormous, mysterious maze. But this is no ordinary labyrinth – it's the famous Melody Maze! Instead of walls, the paths are lined with giant musical instruments. Flutes form flowery arches, drum paths boom beneath your feet, and xylophone bridges tinkle as you cross them. Each section of the maze plays its own tune, filling the air with a symphony of magical music. As you stand at the entrance, trying to decide which path to take, you notice two unusual guides offering their help. To your left is Aria, a song spirit who appears as a swirl of musical notes. She dances in the air, leaving a trail of sparkling stardust behind her. Aria knows all the songs of the maze by heart and can sing in perfect harmony with any instrument. To your right is Beats, a friendly robot with speakers for ears and a boombox for a heart. Beats can analyze and recreate any rhythm instantly, and his feet leave colorful light-up footprints that show where you've already been. Both guides smile at you, eager to help you navigate the musical labyrinth. Who would you like to have as your maze companion?",},
        {"role": "Child", "text": "What's a labyrinth?",},
        {"role": "Narrator", "text": "A labyrinth is another word for a maze or a complex network of paths. It's like a giant puzzle you can walk through, with twisting corridors and sometimes dead ends. The goal is usually to find your way from the entrance to the center or the exit. Labyrinths have been around for thousands of years and appear in many stories and myths. Some are designed for fun, like corn mazes at farms, while others were built long ago and are considered mysterious or even magical. In our story, the Melody Maze is a special kind of labyrinth where music and magic combine to create a unique adventure. Every turn you take and every path you choose will create different music, making your journey through the maze a one-of-a-kind symphony! Now that you know what a labyrinth is, would you prefer Aria the song spirit or Beats the rhythm robot as your guide through this musical maze?",},
        {"role": "Child", "text": "Aria sounds magical!",},
        {"role": "Narrator", "text": "Aria twirls with joy as you choose her, leaving a trail of glittering musical notes in the air. 'Oh, how wonderful!' she sings, her voice a beautiful melody. 'We're going to compose the most magnificent adventure!' She leads you into the maze, and immediately you're surrounded by the soft strumming of harp strings and the gentle chiming of bells. Aria explains that there are two main paths you can take from here. The first is the Lullaby Lane, a soothing route filled with gentle, dreamy music. This path is easier and safer, with calm waters to cross on boats shaped like quarter notes and restful gardens where friendly creatures might share secrets about the maze. The second option is the Crescendo Climb, an exciting and challenging path where the music gets louder and more intense as you go. This route involves climbing trumpet vines, swinging across chasms on violin strings, and solving musical puzzles to progress. It's more difficult but could lead to hidden treasures and magical musical powers. Aria assures you she can guide you through either path with her knowledge of the maze's melodies. The soft music around you seems to hold its breath as you consider. Will you take the peaceful journey down Lullaby Lane, or embark on the thrilling adventure of the Crescendo Climb?",},
        {"role": "Child", "text": "Crescendo Climb sounds exciting!",},
        {"role": "Narrator", "text": "Aria's eyes sparkle with excitement as you choose the Crescendo Climb. 'Bravissimo!' she sings, her voice rising in pitch. 'Let's compose an adventure to remember!' As you begin your climb, the gentle music around you gradually becomes more dynamic. You find yourself scaling a wall made of giant piano keys, each one playing a note as you step on it. Aria guides you to play a simple melody that magically forms a staircase, allowing you to climb higher. The music swells as you ascend, filling you with energy and courage. Soon, you reach a challenging obstacle: a deep chasm with thick violin strings stretched across it. On the other side, you can see a glowing musical score that might be the key to the next part of the maze. Aria hums thoughtfully, then presents you with two options to cross. First, you could try to bounce across the chasm using the violin strings like a trampoline. It's risky, but fast and exhilarating. Your other choice is to carefully climb across the strings as if they were a rope bridge. It's slower and still a bit scary, but more secure. Aria assures you she'll be right with you either way, ready to sing a safety net into existence if needed. The crescendo of the maze's music urges you to decide. Will you take the bouncing route for a thrilling jump, or climb carefully across the musical rope bridge?",},
    ],

    # Story 104: The Talking Tree

    [
        {"role": "Child", "text": "Tree",},
        {"role": "Narrator", "text": "You find a big tree with a face. It smiles and offers you magic fruit or a ride on its branches. Which do you pick?",},
        {"role": "Child", "text": "What's magic fruit?",},
        {"role": "Narrator", "text": "Magic fruit can give you special powers when you eat it. Like flying or talking to animals. Fruit or branch ride?",},
        {"role": "Child", "text": "I want the magic fruit!",},
        {"role": "Narrator", "text": "You eat the fruit and can now talk to animals! A rabbit and a bird appear. Who do you talk to?",},
        {"role": "Child", "text": "What will the rabbit say?",},
        {"role": "Narrator", "text": "The rabbit might tell you about a carrot treasure or a hidden bunny village. Bird or rabbit?",},
        {"role": "Child", "text": "Let's talk to the rabbit!",},
        {"role": "Narrator", "text": "The rabbit tells you about a golden carrot. It's guarded by a fox. Do you help the rabbit get it or leave it alone?",},
    ],

    # Story 105: The Rainbow Bridge

    [
        {"role": "Child", "text": "Rainbow",},
        {"role": "Narrator", "text": "A magical rainbow appears. You can slide down it or walk across. Which do you choose?",},
        {"role": "Child", "text": "What's a rainbow?",},
        {"role": "Narrator", "text": "A rainbow is a colorful arc in the sky after rain. It has many pretty colors. Slide or walk?",},
        {"role": "Child", "text": "Let's slide!",},
        {"role": "Narrator", "text": "You slide down and land in a cloud castle. A unicorn and a dragon live here. Who do you want to meet?",},
        {"role": "Child", "text": "What's a unicorn?",},
        {"role": "Narrator", "text": "A unicorn is a magical horse with a horn on its head. It's white and sparkly. Unicorn or dragon?",},
        {"role": "Child", "text": "I want to meet the unicorn!",},
        {"role": "Narrator", "text": "The unicorn offers to grant a wish or give you a ride. What do you pick?",},
    ],

    # Story 106: The Whispering Woods

    [
        {"role": "Child", "text": "Forest",},
        {"role": "Narrator", "text": "You enter a forest where trees whisper secrets. Do you listen to the oak or the willow?",},
        {"role": "Child", "text": "What's a willow?",},
        {"role": "Narrator", "text": "A willow is a tree with long, droopy branches like curtains. Oak or willow?",},
        {"role": "Child", "text": "I'll listen to the willow!",},
        {"role": "Narrator", "text": "The willow tells you about a hidden fairy village or a magical pond. Where do you go?",},
        {"role": "Child", "text": "What's a fairy?",},
        {"role": "Narrator", "text": "A fairy is a tiny magical person with wings. They can do magic. Fairy village or magic pond?",},
        {"role": "Child", "text": "Let's visit the fairy village!",},
        {"role": "Narrator", "text": "The fairies invite you to a party. You can dance or learn magic. What do you do?",},
    ],

    # Story 107: The Bubble Balloon

    [
        {"role": "Child", "text": "Bubbles",},
        {"role": "Narrator", "text": "You find a giant bubble that can fly. Do you want to go to the clouds or the moon?",},
        {"role": "Child", "text": "What are clouds made of?",},
        {"role": "Narrator", "text": "Clouds are made of tiny water drops floating in the air. They look fluffy. Clouds or moon?",},
        {"role": "Child", "text": "Let's go to the clouds!",},
        {"role": "Narrator", "text": "In the clouds, you meet a rain fairy and a wind spirit. Who do you talk to?",},
        {"role": "Child", "text": "What's a spirit?",},
        {"role": "Narrator", "text": "A spirit is like a magical ghost that controls things in nature. Rain fairy or wind spirit?",},
        {"role": "Child", "text": "I'll talk to the wind spirit!",},
        {"role": "Narrator", "text": "The wind spirit offers to teach you to fly or to control the weather. Which power do you want?",},
    ],

    # Story 108: The Candy Castle

    [
        {"role": "Child", "text": "Candy",},
        {"role": "Narrator", "text": "You find a castle made of candy. The door is chocolate. Do you knock or taste it?",},
        {"role": "Child", "text": "What's chocolate?",},
        {"role": "Narrator", "text": "Chocolate is a sweet, brown candy made from cocoa beans. It's yummy! Knock or taste?",},
        {"role": "Child", "text": "I'll knock!",},
        {"role": "Narrator", "text": "A gumdrop knight answers. He invites you to the lollipop garden or jellybean lake. Where do you go?",},
        {"role": "Child", "text": "What's a gumdrop?",},
        {"role": "Narrator", "text": "A gumdrop is a chewy, sugar-coated candy. They're often fruit-flavored. Garden or lake?",},
        {"role": "Child", "text": "Let's see the lollipop garden!",},
        {"role": "Narrator", "text": "In the garden, you can pick a never-ending lollipop or plant a candy tree. What do you choose?",},
    ],

    # Story 109: The Musical Mountains

    [
        {"role": "Child", "text": "Music",},
        {"role": "Narrator", "text": "You find mountains that play music. Do you follow the flute path or drum trail?",},
        {"role": "Child", "text": "What's a flute?",},
        {"role": "Narrator", "text": "A flute is a musical instrument you blow into to make pretty sounds. Flute or drums?",},
        {"role": "Child", "text": "I like drums!",},
        {"role": "Narrator", "text": "The drum trail leads to a rock concert or a music school. Where do you want to go?",},
        {"role": "Child", "text": "What's a concert?",},
        {"role": "Narrator", "text": "A concert is when musicians play music for people to watch and listen. Concert or school?",},
        {"role": "Child", "text": "Let's go to the concert!",},
        {"role": "Narrator", "text": "At the concert, you can join the band or be in the audience. What do you pick?",},
    ],

    # Story 110: The Time-Traveling Train

    [
        {"role": "Child", "text": "Train",},
        {"role": "Narrator", "text": "A magical train can take you to the past or future. Where do you want to go?",},
        {"role": "Child", "text": "What's the future?",},
        {"role": "Narrator", "text": "The future is time that hasn't happened yet. It might have flying cars! Past or future?",},
        {"role": "Child", "text": "Let's see the future!",},
        {"role": "Narrator", "text": "In the future, you can meet robots or visit a space colony. Which do you choose?",},
        {"role": "Child", "text": "What's a colony?",},
        {"role": "Narrator", "text": "A colony is a place where people live, like a town. This one's in space! Robots or space colony?",},
        {"role": "Child", "text": "I want to see the robots!",},
        {"role": "Narrator", "text": "The robots offer to teach you to build gadgets or control computers. What do you learn?",},
    ],

    # Story 111: The Enchanted Paintbrush

    [
        {"role": "Child", "text": "Paint",},
        {"role": "Narrator", "text": "You find a paintbrush that makes drawings real. Do you paint a dragon or a castle?",},
        {"role": "Child", "text": "What's a dragon?",},
        {"role": "Narrator", "text": "A dragon is a big, magical lizard that can breathe fire and sometimes fly. Dragon or castle?",},
        {"role": "Child", "text": "I'll paint a dragon!",},
        {"role": "Narrator", "text": "Your dragon comes to life! It offers to teach you to fly or breathe fire. Which do you pick?",},
        {"role": "Child", "text": "Can I do both?",},
        {"role": "Narrator", "text": "The dragon says you can only learn one today. Flying or fire-breathing?",},
        {"role": "Child", "text": "I want to fly!",},
        {"role": "Narrator", "text": "You learn to fly! Do you want to race the dragon or explore a cloud city?",},
    ],

    # Story 112: The Talking Garden

    [
        {"role": "Child", "text": "Flowers",},
        {"role": "Narrator", "text": "You enter a garden where plants talk. A rose and a sunflower want to chat. Who do you talk to?",},
        {"role": "Child", "text": "What's a sunflower?",},
        {"role": "Narrator", "text": "A sunflower is a tall yellow flower that follows the sun. Rose or sunflower?",},
        {"role": "Child", "text": "Let's talk to the sunflower!",},
        {"role": "Narrator", "text": "The sunflower tells you about a bee dance party or a butterfly race. Which do you want to see?",},
        {"role": "Child", "text": "What's a butterfly?",},
        {"role": "Narrator", "text": "A butterfly is a pretty insect with colorful wings. They start as caterpillars. Bee party or butterfly race?",},
        {"role": "Child", "text": "Butterfly race sounds fun!",},
        {"role": "Narrator", "text": "At the race, you can be a referee or ride on a butterfly's back. What do you choose?",},
    ],

    # Story 113: The Mermaid's Lagoon

    [
        {"role": "Child", "text": "Ocean",},
        {"role": "Narrator", "text": "You find a secret lagoon with mermaids. They invite you to swim or ride a dolphin. What do you pick?",},
        {"role": "Child", "text": "What's a lagoon?",},
        {"role": "Narrator", "text": "A lagoon is a small part of the ocean near the shore, often with calm water. Swim or dolphin ride?",},
        {"role": "Child", "text": "I want to ride a dolphin!",},
        {"role": "Narrator", "text": "The dolphin takes you to a coral reef or a sunken ship. Where do you want to explore?",},
        {"role": "Child", "text": "What's coral?",},
        {"role": "Narrator", "text": "Coral is a colorful underwater animal that looks like rocks or plants. Reef or sunken ship?",},
        {"role": "Child", "text": "Let's see the coral reef!",},
        {"role": "Narrator", "text": "At the reef, you find a magic pearl. It grants wishes or lets you breathe underwater. Which power do you want?",},
    ],

    # Story 121: The Global Food Festival
    [
        {"role": "Child", "text": "Food",},
        {"role": "Narrator", "text": "You find a magical cookbook that can take you anywhere! Would you like to make Italian pasta with a chef or sushi with a dragon?",},
        {"role": "Child", "text": "What's sushi?",},
        {"role": "Narrator", "text": "Sushi is special food from Japan with rice and fish. Dragon sushi or Italian pasta?",},
        {"role": "Child", "text": "Dragon sushi sounds fun!",},
        {"role": "Narrator", "text": "The dragon teaches you to roll sushi. You can make rainbow rolls or glowing star rolls. Which one?",},
        {"role": "Child", "text": "What's a rainbow roll?",},
        {"role": "Narrator", "text": "A rainbow roll has many colorful ingredients. It looks like a rainbow! Rainbow or star rolls?",},
        {"role": "Child", "text": "Let's make rainbow rolls!",},
        {"role": "Narrator", "text": "Your rainbow rolls are magical! They can make you speak Japanese or dance like a ninja. Which power do you choose?",},
    ],

    # Story 122: The Tiny Science Adventure
    [
        {"role": "Child", "text": "Small",},
        {"role": "Narrator", "text": "A friendly scientist shrinks you to the size of an ant! Do you want to explore inside a flower or visit a water droplet?",},
        {"role": "Child", "text": "What's inside a flower?",},
        {"role": "Narrator", "text": "Flowers have tiny tubes that carry food and water, like tiny streets! Flower or water droplet?",},
        {"role": "Child", "text": "Let's go inside the flower!",},
        {"role": "Narrator", "text": "Inside the flower, you meet busy bees making honey or pollen fairies painting petals. Who do you help?",},
        {"role": "Child", "text": "What's pollen?",},
        {"role": "Narrator", "text": "Pollen is yellow powder that helps flowers make seeds. Help bees or pollen fairies?",},
        {"role": "Child", "text": "I'll help the pollen fairies!",},
        {"role": "Narrator", "text": "The fairies give you magic pollen paint. You can make flowers grow or change their colors. What do you do?",},
    ],

    # Story 123: The Feelings Forest
    [
        {"role": "Child", "text": "Feelings",},
        {"role": "Narrator", "text": "You enter a forest where trees show emotions. A happy tree glows yellow, while a sad tree rains blue leaves. Which tree do you visit?",},
        {"role": "Child", "text": "Why is the tree sad?",},
        {"role": "Narrator", "text": "The tree misses its friend who moved away. Would you like to cheer it up or visit the happy tree?",},
        {"role": "Child", "text": "Let's cheer up the sad tree!",},
        {"role": "Narrator", "text": "You can tell the tree a funny story or give it a warm hug. How do you want to help?",},
        {"role": "Child", "text": "What kind of story?",},
        {"role": "Narrator", "text": "You could tell a silly joke or a happy memory. Joke or memory?",},
        {"role": "Child", "text": "I'll tell a silly joke!",},
        {"role": "Narrator", "text": "The tree laughs and turns pink with joy! It offers to teach you tree-speaking or emotion-growing. Which do you learn?",},
    ],

    # Story 124: The Building Block Galaxy
    [
        {"role": "Child", "text": "Building",},
        {"role": "Narrator", "text": "You discover magical building blocks floating in space. You can build a rocket ship or a space station. What will you create?",},
        {"role": "Child", "text": "What's a space station?",},
        {"role": "Narrator", "text": "A space station is like a house in space where astronauts live. Rocket or space station?",},
        {"role": "Child", "text": "Space station sounds cool!",},
        {"role": "Narrator", "text": "Your space station needs a garden room or a star-watching room. Which do you build first?",},
        {"role": "Child", "text": "What grows in space?",},
        {"role": "Narrator", "text": "In magic space, you can grow star-fruit or moon flowers! Garden or star-watching room?",},
        {"role": "Child", "text": "Let's make a garden!",},
        {"role": "Narrator", "text": "Your space garden is ready! Do you want to invite alien friends or have a floating tea party?",},
    ],

    # Story 125: The 3D Art Portal
    [
        {"role": "Child", "text": "Art",},
        {"role": "Narrator", "text": "Your drawings come off the page in 3D! Will you draw a magical pet or a fun playground?",},
        {"role": "Child", "text": "What kind of pet?",},
        {"role": "Narrator", "text": "You could draw a color-changing puppy or a flying kitten! Which pet do you want?",},
        {"role": "Child", "text": "A color-changing puppy!",},
        {"role": "Narrator", "text": "Your puppy changes colors with its mood! Do you want to play fetch with rainbow balls or go on a color hunt?",},
        {"role": "Child", "text": "What's a color hunt?",},
        {"role": "Narrator", "text": "You search for special colors to help your puppy learn new tricks. Hunt or play fetch?",},
        {"role": "Child", "text": "Let's go on a color hunt!",},
        {"role": "Narrator", "text": "You found all the colors! Your puppy can now paint with its tail or create light shows. Which power do you choose?",},
    ],

    # Story 126: The Recycling Robot's Mission
    [
        {"role": "Child", "text": "Helping Earth",},
        {"role": "Narrator", "text": "You meet a robot who turns trash into treasures. Help clean the beach or the forest?",},
        {"role": "Child", "text": "What's on the beach?",},
        {"role": "Narrator", "text": "The beach has plastic bottles and old nets that need cleaning. Beach or forest?",},
        {"role": "Child", "text": "Let's clean the beach!",},
        {"role": "Narrator", "text": "The robot can turn the trash into a playground or an underwater garden. Which should we make?",},
        {"role": "Child", "text": "How does it work?",},
        {"role": "Narrator", "text": "The robot uses magic recycling powers to transform trash. Playground or garden?",},
        {"role": "Child", "text": "Let's make a playground!",},
        {"role": "Narrator", "text": "The playground is ready! Do you want to build a slide from bottles or swings from fishing nets?",},
    ],

    # Story 127: The Virtual Reality Wizard
    [
        {"role": "Child", "text": "Computer magic",},
        {"role": "Narrator", "text": "A wizard gives you magic VR goggles. Visit a dinosaur park or a robot city?",},
        {"role": "Child", "text": "What's VR?",},
        {"role": "Narrator", "text": "VR lets you see magical worlds through special goggles. Dinosaurs or robots?",},
        {"role": "Child", "text": "Dinosaur park please!",},
        {"role": "Narrator", "text": "In the park, you can ride a gentle giant or help baby dinos hatch. What do you choose?",},
        {"role": "Child", "text": "What's hatching?",},
        {"role": "Narrator", "text": "Baby dinosaurs are coming out of their eggs! Ride or help hatch?",},
        {"role": "Child", "text": "Let's help them hatch!",},
        {"role": "Narrator", "text": "The babies need a water park or a training ground. Which do we build for them?",},
    ],

    # Story 128: The Musical Medicine Show
    [
        {"role": "Child", "text": "Health",},
        {"role": "Narrator", "text": "You meet a doctor who heals with music! Help cure hiccups or sleepy sneezes?",},
        {"role": "Child", "text": "What's wrong with sneezes?",},
        {"role": "Narrator", "text": "These sneezes make you too sleepy! Hiccups or sneezes?",},
        {"role": "Child", "text": "Let's fix the hiccups!",},
        {"role": "Narrator", "text": "We can use drum beats or flute songs to cure hiccups. Which music should we try?",},
        {"role": "Child", "text": "How do drums help?",},
        {"role": "Narrator", "text": "Drum beats can shake the hiccups away! Drums or flute?",},
        {"role": "Child", "text": "Drums sound fun!",},
        {"role": "Narrator", "text": "The hiccups are gone! Now help make medicine music or teach others to heal with songs?",},
    ],

    # Story 129: The Safety Superhero
    [
        {"role": "Child", "text": "Being safe",},
        {"role": "Narrator", "text": "You get safety superpowers! Help with road safety or internet safety?",},
        {"role": "Child", "text": "What's internet safety?",},
        {"role": "Narrator", "text": "Keeping safe while using computers and phones. Road or internet safety?",},
        {"role": "Child", "text": "Internet safety!",},
        {"role": "Narrator", "text": "You can stop bad downloads or protect passwords. Which mission do you choose?",},
        {"role": "Child", "text": "What are downloads?",},
        {"role": "Narrator", "text": "Downloads are things you save to your computer. Stop downloads or protect passwords?",},
        {"role": "Child", "text": "Let's protect passwords!",},
        {"role": "Narrator", "text": "Good choice! Create a password fort or teach others about strong passwords?",},
    ],

    # Story 130: The Weather Workshop
    [
        {"role": "Child", "text": "Weather",},
        {"role": "Narrator", "text": "You find a place where weather is made! Help create rain or sunshine?",},
        {"role": "Child", "text": "How is rain made?",},
        {"role": "Narrator", "text": "Clouds fill with water drops until they're too heavy! Make rain or sunshine?",},
        {"role": "Child", "text": "Let's make rain!",},
        {"role": "Narrator", "text": "You can make musical rain or rainbow rain. Which type do you want to create?",},
        {"role": "Child", "text": "What's musical rain?",},
        {"role": "Narrator", "text": "Each raindrop makes a different musical note! Musical or rainbow rain?",},
        {"role": "Child", "text": "Musical rain please!",},
        {"role": "Narrator", "text": "The musical rain is playing! Conduct a rain symphony or dance in the puddles?",},
    ],

    # Story 131: The Body Adventure Team
    [
        {"role": "Child", "text": "Inside body",},
        {"role": "Narrator", "text": "You're shrunk to explore inside a body! Visit the heart or the brain?",},
        {"role": "Child", "text": "What's in the heart?",},
        {"role": "Narrator", "text": "The heart is like a pump that moves blood around. Heart or brain?",},
        {"role": "Child", "text": "Let's see the heart!",},
        {"role": "Narrator", "text": "In the heart, help red blood cells deliver oxygen or fight germs. Which job do you pick?",},
        {"role": "Child", "text": "What are blood cells?",},
        {"role": "Narrator", "text": "They're tiny helpers that carry oxygen through the body. Delivery or germ fighting?",},
        {"role": "Child", "text": "I'll help deliver oxygen!",},
        {"role": "Narrator", "text": "Great job delivering! Now race through blood vessels or visit the lungs?",},
    ],

    # Story 132: The Dream Garden
    [
        {"role": "Child", "text": "Dreams",},
        {"role": "Narrator", "text": "You find a garden where dreams grow! Plant happy dreams or adventure dreams?",},
        {"role": "Child", "text": "How do dreams grow?",},
        {"role": "Narrator", "text": "Like flowers, but they bloom into real dreams! Happy or adventure dreams?",},
        {"role": "Child", "text": "Adventure dreams!",},
        {"role": "Narrator", "text": "Your adventure dream grows into a jungle gym or a pirate ship. Which do you explore?",},
        {"role": "Child", "text": "What's in the jungle gym?",},
        {"role": "Narrator", "text": "It has slides through clouds and swings over rainbows! Jungle gym or pirate ship?",},
        {"role": "Child", "text": "Jungle gym please!",},
        {"role": "Narrator", "text": "The dream jungle gym is ready! Invite friends to play or add more dream equipment?",},
    ],

    # Story 133: The Language Lighthouse
    [
        {"role": "Child", "text": "Words",},
        {"role": "Narrator", "text": "A lighthouse speaks all languages! Learn animal talk or flower language?",},
        {"role": "Child", "text": "How do flowers talk?",},
        {"role": "Narrator", "text": "They use colors and scents to share messages! Animal or flower language?",},
        {"role": "Child", "text": "Animal talk please!",},
        {"role": "Narrator", "text": "You can now talk to pets or wild animals. Who do you want to chat with?",},
        {"role": "Child", "text": "What do pets say?",},
        {"role": "Narrator", "text": "They might tell you their secrets and stories! Pets or wild animals?",},
        {"role": "Child", "text": "Let's talk to pets!",},
        {"role": "Narrator", "text": "The pets share their secrets! Start a pet talk club or become an animal translator?",},
    ],

    # Story 134: The Friendship Factory
    [
        {"role": "Child", "text": "Friends",},
        {"role": "Narrator", "text": "You discover a place that makes friendship magic! Create friendship bracelets or friendship potions?",},
        {"role": "Child", "text": "What do the bracelets do?",},
        {"role": "Narrator", "text": "They light up when friends are near! Bracelets or potions?",},
        {"role": "Child", "text": "Bracelets sound cool!",},
        {"role": "Narrator", "text": "Your bracelet can share feelings or send happy thoughts. Which power do you choose?",},
        {"role": "Child", "text": "How do you share feelings?",},
        {"role": "Narrator", "text": "The bracelet glows in different colors for different feelings! Share feelings or send thoughts?",},
        {"role": "Child", "text": "Let's share feelings!",},
        {"role": "Narrator", "text": "Your feeling-sharing bracelet is ready! Give one to your best friend or make new friends?",},
    ],

        # Story 136: The Coding Wizard
    [
        {"role": "Child", "text": "Tell me a story about someone learning magic!",},
        {"role": "Narrator", "text": "You enter a school where spells are written in computer code! Learn to code flying spells or potion recipes?",},
        {"role": "Child", "text": "What's computer code?",},
        {"role": "Narrator", "text": "Code is like writing instructions for computers, but here it makes magic! Flying or potions?",},
        {"role": "Child", "text": "Flying sounds fun!",},
        {"role": "Narrator", "text": "You can program a flying carpet or create flying shoes. Which will you code?",},
        {"role": "Child", "text": "What's different about them?",},
        {"role": "Narrator", "text": "Carpets can carry more people, shoes give more control. Carpet or shoes?",},
        {"role": "Child", "text": "Let's make flying shoes!",},
        {"role": "Narrator", "text": "Your shoes work! Test them in a race course or explore the cloud kingdom?",},
    ],

    # Story 137: The Sound Seeker
    [
        {"role": "Child", "text": "Can you tell me about magical music?",},
        {"role": "Narrator", "text": "You find a special instrument that collects sounds! Gather animal songs or nature sounds?",},
        {"role": "Child", "text": "What kind of animal songs?",},
        {"role": "Narrator", "text": "You might hear whale songs or bird melodies! Animals or nature?",},
        {"role": "Child", "text": "Let's collect animal songs!",},
        {"role": "Narrator", "text": "You can mix wolf howls with dolphin songs or bird chirps with tiger purrs. Which mix?",},
        {"role": "Child", "text": "What do dolphins sound like?",},
        {"role": "Narrator", "text": "Dolphins make clicks and whistles like happy laughter! Wolves and dolphins or birds and tigers?",},
        {"role": "Child", "text": "Dolphins and wolves please!",},
        {"role": "Narrator", "text": "Your song is beautiful! Use it to talk to animals or create magical dances?",},
    ],

    # Story 138: The Time Detective
    [
        {"role": "Child", "text": "Please tell me a story about solving mysteries!",},
        {"role": "Narrator", "text": "You become a detective who solves mysteries through time! Investigate missing dinosaur eggs or lost pirate treasure?",},
        {"role": "Child", "text": "What happened to the eggs?",},
        {"role": "Narrator", "text": "Someone moved them from their nest! Eggs or treasure?",},
        {"role": "Child", "text": "Let's find the eggs!",},
        {"role": "Narrator", "text": "You can follow footprints or ask other dinosaurs for help. How will you investigate?",},
        {"role": "Child", "text": "Are the dinosaurs friendly?",},
        {"role": "Narrator", "text": "Yes, they want to help find the eggs! Follow prints or ask dinosaurs?",},
        {"role": "Child", "text": "Let's talk to the dinosaurs!",},
        {"role": "Narrator", "text": "They lead you to a cave! Sneak in quietly or make a plan first?",},
    ],

    # Story 139: The Dream Explorer
    [
        {"role": "Child", "text": "Tell me a story about exploring dreams!",},
        {"role": "Narrator", "text": "You find a magical pillow that lets you visit dreams! Explore happy dreams or adventure dreams?",},
        {"role": "Child", "text": "What's in happy dreams?",},
        {"role": "Narrator", "text": "Happy dreams have rainbow slides and cotton candy clouds! Happy or adventure dreams?",},
        {"role": "Child", "text": "Happy dreams please!",},
        {"role": "Narrator", "text": "You can ride a giggly unicorn or swim in a chocolate lake. Which do you choose?",},
        {"role": "Child", "text": "Is the lake really chocolate?",},
        {"role": "Narrator", "text": "Yes, and it's perfectly safe to swim in! Unicorn or chocolate lake?",},
        {"role": "Child", "text": "Let's ride the unicorn!",},
        {"role": "Narrator", "text": "The unicorn flies you to a birthday party castle or a candy forest. Where to?",},
    ],

    # Story 140: The Color Catcher
    [
        {"role": "Child", "text": "Can you tell me about a rainbow adventure?",},
        {"role": "Narrator", "text": "You get special glasses that let you catch and collect colors! Gather sunset colors or northern lights?",},
        {"role": "Child", "text": "What are northern lights?",},
        {"role": "Narrator", "text": "They're beautiful dancing lights in the sky! Sunset or northern lights?",},
        {"role": "Child", "text": "Northern lights sound pretty!",},
        {"role": "Narrator", "text": "You can catch green sparkles or purple waves. Which colors first?",},
        {"role": "Child", "text": "Do they do anything special?",},
        {"role": "Narrator", "text": "The colors can paint the sky or make magic art! Green or purple?",},
        {"role": "Child", "text": "Let's catch green!",},
        {"role": "Narrator", "text": "Your green lights make nature glow! Create a light show or paint the forest?",},
    ],

    # Story 141: The Pet Whisperer
    [
        {"role": "Child", "text": "Tell me a story about talking to animals!",},
        {"role": "Narrator", "text": "You get a magic necklace that lets you understand pets! Visit a cat café or a dog park?",},
        {"role": "Child", "text": "What's a cat café?",},
        {"role": "Narrator", "text": "It's a place where lots of cats live and play! Cats or dogs?",},
        {"role": "Child", "text": "Cat café sounds fun!",},
        {"role": "Narrator", "text": "The cats are telling stories or teaching yoga. What would you like to do?",},
        {"role": "Child", "text": "Can cats do yoga?",},
        {"role": "Narrator", "text": "These cats are experts at stretching and posing! Stories or yoga?",},
        {"role": "Child", "text": "I want to hear stories!",},
        {"role": "Narrator", "text": "The cats share tales of midnight adventures! Join their next adventure or write their stories down?",},
    ],

    # Story 142: The Garden Doctor
    [
        {"role": "Child", "text": "Please tell me about helping sick plants!",},
        {"role": "Narrator", "text": "You become a plant doctor with a magic watering can! Help sad flowers or tired trees?",},
        {"role": "Child", "text": "Why are the flowers sad?",},
        {"role": "Narrator", "text": "They need special rainbow water to feel better! Flowers or trees?",},
        {"role": "Child", "text": "Let's help the flowers!",},
        {"role": "Narrator", "text": "You can give them giggly water or singing water. Which medicine?",},
        {"role": "Child", "text": "What does giggly water do?",},
        {"role": "Narrator", "text": "It makes the flowers laugh and dance! Giggly or singing water?",},
        {"role": "Child", "text": "Giggly water please!",},
        {"role": "Narrator", "text": "The flowers are happy now! Start a flower hospital or teach other garden doctors?",},
    ],

    # Story 143: The Sky Chef
    [
        {"role": "Child", "text": "Tell me about cooking in the clouds!",},
        {"role": "Narrator", "text": "You discover a kitchen floating in the sky! Make cloud cookies or rainbow cakes?",},
        {"role": "Child", "text": "What are cloud cookies?",},
        {"role": "Narrator", "text": "They're fluffy cookies that make you float! Cookies or cakes?",},
        {"role": "Child", "text": "Cloud cookies sound yummy!",},
        {"role": "Narrator", "text": "Add stardust sprinkles or moonbeam frosting to your cookies. Which topping?",},
        {"role": "Child", "text": "What do stardust sprinkles do?",},
        {"role": "Narrator", "text": "They make your wishes come true! Stardust or moonbeam?",},
        {"role": "Child", "text": "Stardust sprinkles please!",},
        {"role": "Narrator", "text": "Your cookies grant wishes! Share them with friends or open a sky bakery?",},
    ],

    # Story 144: The Space Gardener
    [
        {"role": "Child", "text": "Tell me about growing plants in space!",},
        {"role": "Narrator", "text": "You're a gardener on a space station! Plant star-flowers or moon-melons?",},
        {"role": "Child", "text": "What's a moon-melon?",},
        {"role": "Narrator", "text": "It's a glowing fruit that tastes like your favorite things! Flowers or melons?",},
        {"role": "Child", "text": "Moon-melons sound cool!",},
        {"role": "Narrator", "text": "Your melons can feed astronauts or power the space station. What's their purpose?",},
        {"role": "Child", "text": "How do they make power?",},
        {"role": "Narrator", "text": "Their glow can be turned into energy! Food or power source?",},
        {"role": "Child", "text": "Let's power the station!",},
        {"role": "Narrator", "text": "The station is fully powered! Grow more space fruits or teach space farming?",},
    ],

    # Story 145: The Emotion Artist
    [
        {"role": "Child", "text": "Tell me about painting feelings!",},
        {"role": "Narrator", "text": "You get magical paints that show emotions! Paint happiness or bravery?",},
        {"role": "Child", "text": "How do you paint bravery?",},
        {"role": "Narrator", "text": "With bold colors and strong shapes! Happiness or bravery?",},
        {"role": "Child", "text": "Let's paint bravery!",},
        {"role": "Narrator", "text": "Your painting comes alive! It can give courage or teach confidence. Which power?",},
        {"role": "Child", "text": "What's confidence?",},
        {"role": "Narrator", "text": "Believing in yourself and feeling strong! Give courage or teach confidence?",},
        {"role": "Child", "text": "Let's give courage!",},
        {"role": "Narrator", "text": "Your brave painting helps others! Create more emotion art or open a feeling gallery?",},
    ],

    # Story 146: The Dance Teacher
    [
        {"role": "Child", "text": "Tell me about magic dancing!",},
        {"role": "Narrator", "text": "You find shoes that create magic when you dance! Learn ballet with butterflies or hip-hop with robots?",},
        {"role": "Child", "text": "What's ballet?",},
        {"role": "Narrator", "text": "Ballet is graceful dancing on your toes! Butterflies or robots?",},
        {"role": "Child", "text": "Robot dancing please!",},
        {"role": "Narrator", "text": "The robots teach you moves that make lights or create music. Which style?",},
        {"role": "Child", "text": "How do they make lights?",},
        {"role": "Narrator", "text": "Your dancing creates colorful light trails! Light dance or music dance?",},
        {"role": "Child", "text": "Light dancing sounds fun!",},
        {"role": "Narrator", "text": "You're glowing with dance lights! Perform in a light show or teach other dancers?",},
    ],

    # Story 147: The Ocean Cleaner
    [
        {"role": "Child", "text": "Tell me about saving the ocean!",},
        {"role": "Narrator", "text": "You get a submarine that cleans the ocean! Help sea turtles or clean coral reefs?",},
        {"role": "Child", "text": "What's wrong with the coral?",},
        {"role": "Narrator", "text": "It needs cleaning to stay healthy and colorful! Turtles or coral?",},
        {"role": "Child", "text": "Let's clean the coral!",},
        {"role": "Narrator", "text": "Use rainbow bubbles or singing starfish to clean? Choose your helpers!",},
        {"role": "Child", "text": "What do the bubbles do?",},
        {"role": "Narrator", "text": "They make the coral sparkle clean! Bubbles or starfish?",},
        {"role": "Child", "text": "Rainbow bubbles please!",},
        {"role": "Narrator", "text": "The coral is clean and happy! Start a sea cleaning team or teach fish to help?",},
    ],

    # Story 148: The Dream Builder
    [
        {"role": "Child", "text": "Tell me about building dreams!",},
        {"role": "Narrator", "text": "You find blocks that build dreams! Create a flying circus or an underwater castle?",},
        {"role": "Child", "text": "What's in the flying circus?",},
        {"role": "Narrator", "text": "Acrobat clouds and trapeze rainbows! Circus or castle?",},
        {"role": "Child", "text": "Flying circus please!",},
        {"role": "Narrator", "text": "Build a cotton candy cloud tent or star light stage first?",},
        {"role": "Child", "text": "What happens on the stage?",},
        {"role": "Narrator", "text": "Stars perform amazing light shows! Cloud tent or star stage?",},
        {"role": "Child", "text": "Star stage sounds pretty!",},
        {"role": "Narrator", "text": "Your stage is ready! Put on a star show or teach dream building?",},
    ],

    # Story 149: The Melody Maker
    [
        {"role": "Child", "text": "Tell me about making special music!",},
        {"role": "Narrator", "text": "You get a magic music box that creates songs! Make lullabies for plants or dance music for toys?",},
        {"role": "Child", "text": "Do plants like music?",},
        {"role": "Narrator", "text": "These plants grow better with special songs! Lullabies or dance music?",},
        {"role": "Child", "text": "Let's make plant lullabies!",},
        {"role": "Narrator", "text": "Use gentle rain sounds or soft wind chimes in your song?",},
        {"role": "Child", "text": "What do wind chimes do?",},
        {"role": "Narrator", "text": "They make tinkly, peaceful music! Rain or chimes?",},
        {"role": "Child", "text": "Wind chimes please!",},
        {"role": "Narrator", "text": "The plants grow huge! Start a musical garden or write more plant songs?",},
    ],

    # Story 151: The Courage Garden
    [
        {"role": "Child", "text": "Tell me a story about finding courage!",},
        {"role": "Narrator", "text": "In a magical garden where feelings grow like flowers, you meet Luna, a shy butterfly with shimmering wings who's afraid to fly. The garden keeper, wise old Owl, explains that Luna needs help gathering courage-bloom petals to overcome her fears. Would you like to help Luna or explore the garden first?",},
        {"role": "Child", "text": "What's in the rest of the garden?",},
        {"role": "Narrator", "text": "The garden has glowing confidence roses and friendship daisies. But there's also a patch of worry weeds trying to spread. The garden needs help, and Luna looks hopeful. Explore more or help Luna?",},
        {"role": "Child", "text": "Let's help Luna!",},
        {"role": "Narrator", "text": "Together with Luna, you begin collecting courage-blooms. Suddenly, a storm of doubt-clouds appears, created by the grumpy Fog Troll who doesn't like seeing others become brave. You can shelter under Owl's wisdom tree or face the clouds together with Luna. What's your choice?",},
        {"role": "Child", "text": "What happens if we face the clouds?",},
        {"role": "Narrator", "text": "Facing the clouds means working together, showing Luna that being brave means feeling scared but trying anyway. The Fog Troll might learn something too! Hide or face fears together?",},
        {"role": "Child", "text": "Let's be brave together!",},
        {"role": "Narrator", "text": "Holding hands with Luna, you step into the doubt-clouds. They swirl around you, but your friendship creates a warm glow that pushes them back. The Fog Troll, seeing your bravery, becomes curious instead of grumpy. Now you can invite him to join your courage-gathering or show Luna it's time for her first flight. Which do you choose?",},
        {"role": "Child", "text": "Can the Fog Troll help Luna fly?",},
        {"role": "Narrator", "text": "Yes! The Troll could make small clouds for Luna to practice on before flying high. What a wonderful idea! Flying practice or gather more petals?",},
        {"role": "Child", "text": "Flying practice with the Troll!",},
        {"role": "Narrator", "text": "The Fog Troll creates gentle cloud steps, and Luna begins hopping from cloud to cloud, growing more confident. Owl watches proudly as you encourage her. Finally, Luna spreads her wings and soars! The garden blooms with new courage-flowers everywhere. Should you all celebrate with a flying party or start a bravery school to help other shy creatures?",},
    ],

    # Story 152: The Time-Mending Library
    [
        {"role": "Child", "text": "Tell me an adventure story about fixing mistakes!",},
        {"role": "Narrator", "text": "You discover an ancient library where stories come alive. The head librarian, Ms. Chapters, explains that someone has been changing story endings, causing chaos in both books and reality. She shows you a magical bookmark that can transport you into stories. Would you like to investigate who's changing the stories or start fixing them right away?",},
        {"role": "Child", "text": "Who would change the stories?",},
        {"role": "Narrator", "text": "Ms. Chapters suspects either the Blank Page, a mysterious character who thinks all stories should be rewritten, or her former assistant who got lost in a story long ago. Investigate the mystery or start repairs?",},
        {"role": "Child", "text": "Let's solve the mystery!",},
        {"role": "Narrator", "text": "Using the bookmark, you enter the library's secret archives. You find torn pages with clues: smudged ink tears and lonely writings. A small paper crane comes to life and offers to guide you to either the forgotten stories section or the dreams department. Where do you want to look?",},
        {"role": "Child", "text": "What's in the forgotten stories?",},
        {"role": "Narrator", "text": "Forgotten stories hold tales that were never finished - the perfect hiding place for someone who feels left out. You might find the lost assistant there! Forgotten stories or dreams?",},
        {"role": "Child", "text": "Let's check forgotten stories!",},
        {"role": "Narrator", "text": "In the forgotten section, you find the assistant, now known as the Wordweaver, surrounded by half-finished tales. She's been changing stories to give forgotten characters happy endings. You can suggest combining stories to make new ones or help her write her own original tales. What do you propose?",},
        {"role": "Child", "text": "Can we write new stories together?",},
        {"role": "Narrator", "text": "Yes! The Wordweaver's eyes light up at this idea. She could create new stories instead of changing old ones. Write together or invite her back to help Ms. Chapters?",},
        {"role": "Child", "text": "Both! She can write AND help!",},
        {"role": "Narrator", "text": "The Wordweaver and Ms. Chapters create a new section of the library for reimagined tales and original stories. Together, you start a storytelling club where forgotten characters get new adventures. Should your first project be a grand adventure book or a collection of fairy tales?",},
    ],

    # Story 153: The Emotion Orchestra
    [
        {"role": "Child", "text": "Tell me about a magical music adventure!",},
        {"role": "Narrator", "text": "Welcome to the Heartstring Hall, where emotions become music! You meet Melody, a young conductor who can see feelings as colors but struggles to help her orchestra play them correctly. The instruments are all enchanted, each with its own personality. The grumpy old tuba, Mr. Brass, keeps playing sad notes during happy songs. Would you like to help Melody with the orchestra or talk to Mr. Brass?",},
        {"role": "Child", "text": "Why is Mr. Brass so grumpy?",},
        {"role": "Narrator", "text": "Looking closer, you notice Mr. Brass's shine has faded, and he sits alone while other instruments practice together. He might be lonely or missing his old friend, the retired trumpet. Talk to him or help Melody first?",},
        {"role": "Child", "text": "Let's talk to Mr. Brass!",},
        {"role": "Narrator", "text": "Approaching Mr. Brass, you hear him humming an old friendship song. Suddenly, the Chaos Conductor appears, offering to make all music sad like Mr. Brass prefers. You can challenge the Chaos Conductor to a music duel or try to remind Mr. Brass of happy songs. What will you do?",},
        {"role": "Child", "text": "What's a music duel?",},
        {"role": "Narrator", "text": "In a music duel, each side plays songs to show the power of different emotions. You'd work with Melody and the orchestra against the Chaos Conductor! Duel or remind Mr. Brass of happy times?",},
        {"role": "Child", "text": "Let's have a music duel!",},
        {"role": "Narrator", "text": "The duel begins! Your team plays songs of friendship and joy, while the Chaos Conductor creates storms of sadness. Mr. Brass watches, torn between the two. You notice his old friend's trumpet displayed nearby. Should you bring down the trumpet or ask Melody to conduct a special friendship symphony?",},
        {"role": "Child", "text": "Let's include the trumpet!",},
        {"role": "Narrator", "text": "The trumpet joins in, playing a harmony to Mr. Brass's melody. Their song of friendship overpowers the Chaos Conductor's sadness! Mr. Brass's shine returns, and he apologizes for being grumpy. Now you can organize a celebration concert or start a music school for emotional harmony. Which would you prefer?",},
    ],

    # Story 154: The Dream Engineer
    [
        {"role": "Child", "text": "Tell me about building dreams and fixing nightmares!",},
        {"role": "Narrator", "text": "You're chosen to be a Dream Engineer at the Night Sky Workshop, where dreams are designed and nightmares are repaired. Your mentor, Starweaver Sandy, reveals that the Nightmare Mechanic has gone missing, leaving unfinished dreams everywhere. The workshop's full of dream parts: cloud gears, starlight springs, and rainbow wires. Do you want to learn dream building or begin searching for the Mechanic?",},
        {"role": "Child", "text": "What happened to the Mechanic?",},
        {"role": "Narrator", "text": "Looking through the workshop logs, you find notes about the Mechanic feeling overwhelmed by too many nightmares needing repair. There's also a map showing either the Peaceful Plains or the Worry Woods. Where should we look?",},
        {"role": "Child", "text": "Are the Worry Woods scary?",},
        {"role": "Narrator", "text": "They're misunderstood - full of tangled fears waiting to be untangled. The Mechanic might be there helping! Woods or Plains?",},
        {"role": "Child", "text": "Let's check the Worry Woods!",},
        {"role": "Narrator", "text": "In the Worry Woods, you find the Mechanic turning nightmares into dream parts! But she's caught in a loop, afraid to return because of all the work. You can show her your new dream-building skills or suggest a better way to organize the workshop. What's your idea?",},
        {"role": "Child", "text": "How can we organize better?",},
        {"role": "Narrator", "text": "We could create a dream assembly line or build a nightmare recycling center where scary parts become something new! Which plan sounds better?",},
        {"role": "Child", "text": "A recycling center sounds good!",},
        {"role": "Narrator", "text": "Together with the Mechanic, you build the recycling center. Nightmares go in scary but come out as dream parts! Now, should you create a team of dream fixers or start a night-light patrol to catch bad dreams early?",},
    ],

    # Story 155: The Friendship Garden
    [
        {"role": "Child", "text": "Tell me about growing special friendships!",},
        {"role": "Narrator", "text": "You receive a packet of magical friendship seeds from Flora, the Garden Whisperer. She explains that friendship flowers only bloom when people work together, but lately, the garden's being affected by the Loneliness Fog rolling in from the Gray Mountains. Would you like to learn about the seeds or investigate the fog?",},
        {"role": "Child", "text": "What makes the fog lonely?",},
        {"role": "Narrator", "text": "Flora shares that the fog comes from Shadow, a mountain spirit who doesn't know how to make friends. We could invite Shadow to the garden or send them a friendship flower. What should we do?",},
        {"role": "Child", "text": "Let's invite Shadow here!",},
        {"role": "Narrator", "text": "As you and Flora prepare the garden for Shadow's visit, some plants start wilting from nervousness. You can either create a special welcome garden or organize a flower festival to help everyone feel comfortable. Which would help more?",},
        {"role": "Child", "text": "What's a flower festival like?",},
        {"role": "Narrator", "text": "It's a celebration where all the plants share their colors and scents, making everyone feel welcome! Garden or festival?",},
        {"role": "Child", "text": "Festival sounds fun!",},
        {"role": "Narrator", "text": "During the festival, Shadow arrives, bringing swirling fog. But as the flowers share their stories, the fog begins to sparkle with interest. Shadow reveals they have crystal seeds from the mountains. Should you combine them with friendship seeds or start a new garden together?",},
        {"role": "Child", "text": "What do crystal seeds grow?",},
        {"role": "Narrator", "text": "Crystal seeds grow shimmering plants that reflect everyone's true selves! Combine seeds or new garden?",},
        {"role": "Child", "text": "Let's combine them!",},
        {"role": "Narrator", "text": "The combined seeds grow into beautiful Crystal Friendship Flowers that glow with acceptance! Shadow's fog becomes a gentle mist that helps the garden flourish. Should you create a friendship school or start a garden exchange program with the mountains?",},
    ],

    # Story 156: The Jungle Symphony
    [
        {"role": "Child", "text": "Jungle!",},
        {"role": "Narrator", "text": "Deep in an enchanted rainforest, you meet Maya the Musical Macaw, who conducts the jungle's morning song. But today, something's wrong - the harmony is missing because the Silence Sloth has borrowed all the morning sounds to help him sleep. Maya looks worried as mangoes fall silently from trees. Will you help find the missing sounds or try to create new ones?",},
        {"role": "Child", "text": "What kind of sounds are missing?",},
        {"role": "Narrator", "text": "The waterfall's splash, bird calls, monkey laughs, and even the whisper of leaves - all borrowed by Silence Sloth! Search for sounds or make new ones?",},
        {"role": "Child", "text": "Let's find the sounds!",},
        {"role": "Narrator", "text": "Following Maya through hanging vines, you discover Silence Sloth's treehouse filled with glowing sound bubbles. But the Rain Spirit appears, warning that without morning music, the jungle's magic will fade. Sneak in during Sloth's nap or try to wake him gently?",},
        {"role": "Child", "text": "How can we wake him nicely?",},
        {"role": "Narrator", "text": "Maya suggests singing a special lullaby that turns into a wake-up song, or leaving a trail of honey to tempt him awake. Music or honey?",},
        {"role": "Child", "text": "Let's try the lullaby!",},
        {"role": "Narrator", "text": "Maya teaches you a magical melody. As you sing, Silence Sloth smiles in his sleep, and the sound bubbles begin to dance. But he needs help organizing the sounds before returning them. Sort them by morning, noon, and night, or create a new symphony?",},
        {"role": "Child", "text": "What's a symphony?",},
        {"role": "Narrator", "text": "A symphony is when many different sounds play together in harmony to make beautiful music. Sort sounds or make new music?",},
        {"role": "Child", "text": "Let's make a symphony!",},
        {"role": "Narrator", "text": "Working together with Maya and Silence Sloth, you create the most beautiful jungle song ever heard! The Sloth realizes sounds can be both peaceful and joyful. Should you start a jungle music school or organize daily symphony performances?",},
    ],

    # Story 157: The Rainbow Rescue
    [
        {"role": "Child", "text": "Colors!",},
        {"role": "Narrator", "text": "In the Sky Painter's Studio, you meet Iris the young rainbow apprentice. She's upset because the Color Collector, a mysterious cloud, has been gathering all the bright colors, leaving the world grey. The Head Sky Painter is away, and Iris needs help bringing color back. Would you like to track the Color Collector or learn to mix new colors?",},
        {"role": "Child", "text": "Why does the cloud take colors?",},
        {"role": "Narrator", "text": "Through her telescope, Iris spots the Color Collector admiring its collection - it seems lonely and thinks keeping colors might make others visit! Investigate or create new colors?",},
        {"role": "Child", "text": "Let's visit the cloud!",},
        {"role": "Narrator", "text": "Using Iris's paint-powered hot air balloon, you float up to the Color Collector. Inside its cloudy palace, colors swirl like dancing lights. The cloud seems surprised by visitors! Share color-mixing secrets or suggest a color-trading festival?",},
        {"role": "Child", "text": "What's color trading?",},
        {"role": "Narrator", "text": "Everyone would bring colors to share and swap, making new friends and combinations! Trading festival or mixing secrets?",},
        {"role": "Child", "text": "A festival sounds fun!",},
        {"role": "Narrator", "text": "The Color Collector brightens at the idea! But first, help choose: host the festival in the cloud castle or create a rainbow bridge to connect sky and ground?",},
        {"role": "Child", "text": "Can we do both?",},
        {"role": "Narrator", "text": "Brilliant idea! The rainbow bridge could lead to the cloud castle! Should we invite sky painters first or start with ground artists?",},
    ],

    # Story 158: The Dream Garden
    [
        {"role": "Child", "text": "Sleep!",},
        {"role": "Narrator", "text": "You discover a garden where dreams grow like flowers, tended by Luna the Star Gardener. But lately, the Nightmare Weeds, planted by the worried Shadow Sprout, are spreading. Luna explains that Shadow Sprout was once a happy dream flower until fear made it hide. Will you help Luna tend the garden or look for Shadow Sprout?",},
        {"role": "Child", "text": "What are Nightmare Weeds?",},
        {"role": "Narrator", "text": "They're dark plants that grow from worried thoughts and scary dreams. Luna says they're just confused dream-flowers. Garden work or find Shadow Sprout?",},
        {"role": "Child", "text": "Let's find Shadow Sprout!",},
        {"role": "Narrator", "text": "Following a trail of starlight, you find Shadow Sprout hiding under dream-clouds. It's using nightmare seeds to feel less alone, but the scary plants are spreading! Offer to plant friendship flowers nearby or show how to transform nightmare seeds?",},
        {"role": "Child", "text": "How do we transform seeds?",},
        {"role": "Narrator", "text": "Luna knows a special starlight dance that can turn scary seeds into sweet dreams. Try the dance or plant new flowers?",},
        {"role": "Child", "text": "Let's do the starlight dance!",},
        {"role": "Narrator", "text": "Dancing together, the nightmare seeds begin to sparkle! Shadow Sprout joins in, slowly glowing brighter. Now you can create a dream rainbow garden or start a night-light flower patch. Which would you prefer?",},
        {"role": "Child", "text": "What's in a rainbow garden?",},
        {"role": "Narrator", "text": "A rainbow garden grows all kinds of dreams - happy, brave, and peaceful ones! Rainbow garden or night-lights?",},
        {"role": "Child", "text": "Rainbow garden please!",},
        {"role": "Narrator", "text": "Together with Luna and Shadow Sprout, now glowing softly, you plant a beautiful rainbow dream garden. Should you open a dream flower school or start a sweet dreams delivery service?",},
    ],

    # Story 159: The Ocean Orchestra
    [
        {"role": "Child", "text": "Music!",},
        {"role": "Narrator", "text": "Beneath the waves, you meet Marina the Melody Mermaid, conductor of the Ocean Orchestra. The orchestra faces a challenge: Discord the grumpy sea serpent has tangled all their kelp instruments! Marina needs help before the Underwater Music Festival. Untangle the instruments or talk to Discord?",},
        {"role": "Child", "text": "What's wrong with Discord?",},
        {"role": "Narrator", "text": "Marina whispers that Discord used to sing beautiful songs until a storm scared his voice away. Now he hides in the deep. Visit Discord or help the orchestra?",},
        {"role": "Child", "text": "Let's help Discord!",},
        {"role": "Narrator", "text": "Swimming to the deep caves, you hear soft humming. Discord isn't mean - he's scared to sing after the storm! You can show him the healing power of music or invite him to play a quiet instrument in the orchestra. What's your choice?",},
        {"role": "Child", "text": "How does music heal?",},
        {"role": "Narrator", "text": "Music can help us feel better when we're scared or sad. Should we share healing songs or offer him an instrument?",},
        {"role": "Child", "text": "Share healing songs!",},
        {"role": "Narrator", "text": "Marina leads a gentle song that echoes through the cave. Discord's scales begin to shimmer as he remembers his love of music. Should he join the orchestra or start teaching young sea creatures to sing?",},
        {"role": "Child", "text": "Can he do both?",},
        {"role": "Narrator", "text": "What a wonderful idea! Discord could teach during the day and perform at night! Should his first class be for shy singers or brave beginners?",},
    ],

    # Story 160: The Time Garden
    [
        {"role": "Child", "text": "Time!",},
        {"role": "Narrator", "text": "In a mysterious garden, you meet Chronos the Time-Keeping Tortoise who watches over flowers that bloom in past, present, and future. But the Present Flowers are fading because young Max the Minute Hare is too busy rushing to tend them. Help Chronos save the flowers or chase after Max?",},
        {"role": "Child", "text": "Why is Max rushing?",},
        {"role": "Narrator", "text": "Chronos explains that Max thinks faster is always better and doesn't understand the joy of the present moment. Find Max or care for flowers?",},
        {"role": "Child", "text": "Let's find Max!",},
        {"role": "Narrator", "text": "You follow Max's trail through tomorrow's tulips and yesterday's roses. Finally, you find him racing through time! Show him the beauty of slow moments or challenge him to a special race?",},
        {"role": "Child", "text": "What's a special race?",},
        {"role": "Narrator", "text": "A race where the winner is the one who notices the most beautiful things along the way! Race or show slow moments?",},
        {"role": "Child", "text": "Let's have a noticing race!",},
        {"role": "Narrator", "text": "During the race, Max discovers amazing details he never saw before. Now he wants to help but isn't sure how. Should he become a Time Garden tour guide or learn to be a present-moment gardener?",},
        {"role": "Child", "text": "What's a present-moment gardener?",},
        {"role": "Narrator", "text": "Someone who helps others enjoy and appreciate the beauty of now! Tour guide or gardener?",},
        {"role": "Child", "text": "Present-moment gardener!",},
        {"role": "Narrator", "text": "Max learns to tend the Present Flowers, sharing their magic with visitors. Should he start a mindfulness meadow or create a garden of special moments?",},
    ],

    # Story 161: The Crystal Caves
    [
        {"role": "Child", "text": "Sparkles!",},
        {"role": "Narrator", "text": "Deep underground, you discover Crystal, a young gem fairy who creates beautiful light shows with cave crystals. But the Echo Wizard has taken all the sparkles, thinking they're too noisy. Crystal needs help bringing shimmer back to the caves. Search for the wizard or learn crystal magic?",},
        {"role": "Child", "text": "Are sparkles noisy?",},
        {"role": "Narrator", "text": "The Echo Wizard thinks their twinkling sounds disturb his cave echoes. But maybe they could make music together! Find wizard or learn magic?",},
        {"role": "Child", "text": "Let's try crystal magic!",},
        {"role": "Narrator", "text": "Crystal teaches you to create gentle light patterns. Together, you discover that sparkles can dance to echo rhythms! Should you show the Echo Wizard or prepare a crystal concert?",},
        {"role": "Child", "text": "What's a crystal concert?",},
        {"role": "Narrator", "text": "A beautiful show where lights and sounds work together! Show wizard or plan concert?",},
        {"role": "Child", "text": "Let's plan a concert!",},
        {"role": "Narrator", "text": "The Echo Wizard hears the preparation and investigates. Seeing the harmony of sparkles and sounds, he wants to join! Should he become the conductor or teach echo-crystal duets?",},
        {"role": "Child", "text": "What are duets?",},
        {"role": "Narrator", "text": "When two different things perform together! Crystal sparkles and cave echoes could be beautiful duets. Conductor or teacher?",},
        {"role": "Child", "text": "Let's have him teach!",},
        {"role": "Narrator", "text": "The Echo Wizard and Crystal start a special school where lights and sounds create harmony. Should they perform underground concerts or take their shows to the surface?",},
    ],

    # Story 162: The Cloud Weavers
    [
        {"role": "Child", "text": "Sky!",},
        {"role": "Narrator", "text": "Among the clouds, you find Aria the Wind Weaver creating cloud shapes. She's worried because Storm, the young thunder cloud, keeps accidentally making scary storms when trying to play. The other clouds have drifted away, leaving the sky empty. Help Storm control powers or gather the clouds?",},
        {"role": "Child", "text": "Why does Storm make storms?",},
        {"role": "Narrator", "text": "Storm gets excited or nervous and doesn't know how to control their energy yet. Help Storm or find other clouds?",},
        {"role": "Child", "text": "Let's help Storm!",},
        {"role": "Narrator", "text": "Approaching Storm, you notice they're crying rain-tears. Aria suggests teaching Storm cloud-weaving or showing them gentle rain dances. Which would help more?",},
        {"role": "Child", "text": "What's cloud-weaving?",},
        {"role": "Narrator", "text": "It's like painting with clouds, creating soft shapes in the sky! Weaving or dancing?",},
        {"role": "Child", "text": "Cloud-weaving sounds fun!",},
        {"role": "Narrator", "text": "Storm learns to channel energy into making beautiful cloud sculptures! The other clouds return, amazed. Should Storm open a cloud art gallery or start a weather control school?",},
        {"role": "Child", "text": "Can we see the gallery?",},
        {"role": "Narrator", "text": "Yes! Storm could display cloud art and teach others too! Gallery or school?",},
        {"role": "Child", "text": "Let's make a gallery!",},
        {"role": "Narrator", "text": "The sky becomes a beautiful gallery of cloud art! Should Storm create a sunset show or organize a cloud sculpture contest?",},
    ],

    # Story 163: The Book Explorers
    [
        {"role": "Child", "text": "Stories!",},
        {"role": "Narrator", "text": "In the Living Library, you meet Paige the Story Sprite, who helps characters travel between books. But the Plot Puzzler, a confused bookworm, has been mixing up story endings! Characters are lost in the wrong tales. Help find missing characters or fix mixed-up endings?",},
        {"role": "Child", "text": "Where are the characters?",},
        {"role": "Narrator", "text": "They've wandered into different stories - there's a knight in a space book and a robot in a fairy tale! Search or fix endings?",},
        {"role": "Child", "text": "Let's find them!",},
        {"role": "Narrator", "text": "With Paige's magical bookmark, you begin searching. The Plot Puzzler appears, explaining they just wanted to make stories more interesting! Create new story combinations or restore original tales?",},
        {"role": "Child", "text": "Can stories be combined?",},
        {"role": "Narrator", "text": "Yes, but carefully and creatively! Mix stories or fix them?",},
        {"role": "Child", "text": "Let's make new stories!",},
        {"role": "Narrator", "text": "Working with Paige and the Plot Puzzler, you create exciting new adventures! Should you open a story mixing workshop or start an inter-tale travel agency?",},
        {"role": "Child", "text": "What's inter-tale travel?",},
        {"role": "Narrator", "text": "Characters could visit other stories on holiday! Workshop or travel agency?",},
        {"role": "Child", "text": "Travel agency sounds fun!",},
        {"role": "Narrator", "text": "The Living Library becomes a hub for story-crossing adventures! Should you organize character vacation tours or start a story-mixing festival?",},
    ],

    # Story 165: The Friendship Bridge
    [
        {"role": "Child", "text": "Building!",},
        {"role": "Narrator", "text": "In the Valley of Two Villages, you meet Mira and Leo, best friends who live on opposite sides. A storm destroyed the connecting bridge, and now their communities blame each other for not maintaining it. The Divide Dragon feeds on their growing anger, getting stronger. Will you help rebuild the bridge or talk to the villages?",},
        {"role": "Child", "text": "Why are they fighting?",},
        {"role": "Narrator", "text": "Each village thinks the other should have fixed the bridge. Their anger makes the Divide Dragon stronger, and he whispers things that make them distrust each other more. Talk to villages or start building?",},
        {"role": "Child", "text": "Let's talk to them!",},
        {"role": "Narrator", "text": "You discover both villages secretly miss each other. The Divide Dragon tries to interrupt, spreading rumors! Should you organize a peace meeting or show them how they need each other?",},
        {"role": "Child", "text": "How can we stop the dragon?",},
        {"role": "Narrator", "text": "The dragon grows weaker when people work together. We could gather both villages to build together or create a friendship festival. Which might work better?",},
        {"role": "Child", "text": "Let's build together!",},
        {"role": "Narrator", "text": "As the villages work together, the dragon shrinks! But he has one last trick - offering each village magical tools to build their own bridges. Should you expose his trick or turn it into an opportunity?",},
        {"role": "Child", "text": "What kind of opportunity?",},
        {"role": "Narrator", "text": "We could suggest building two bridges to make an island park in the middle where both villages can meet! Dragon's trick or unity bridge?",},
        {"role": "Child", "text": "Island park sounds nice!",},
        {"role": "Narrator", "text": "The villages love the idea! The Divide Dragon, seeing their cooperation, transforms into a friendly bridge guardian. Should they start a bridge-building school or create a unity celebration?",},
    ],

    # Story 166: The Garden of Differences
    [
        {"role": "Child", "text": "Growing!",},
        {"role": "Narrator", "text": "You discover a magical garden where Sunny the Sunflower and Shade the Mushroom argue about who belongs. Sunny thinks the garden should be full of sun, while Shade believes shadows are better. Their argument is making other plants wilt! Will you help Sunny understand Shade or show Shade the value of sun?",},
        {"role": "Child", "text": "Can't they share the garden?",},
        {"role": "Narrator", "text": "They've never tried! The Grumpy Groundhog keeps telling them they're too different to be friends. Should we prove him wrong or find a way they can help each other?",},
        {"role": "Child", "text": "How can they help each other?",},
        {"role": "Narrator", "text": "Maybe Sunny could protect smaller plants from too much sun, while Shade could help roots grow strong! Should we show them this or challenge Grumpy Groundhog's ideas?",},
        {"role": "Child", "text": "Let's show them how to help!",},
        {"role": "Narrator", "text": "You create a small garden where sunflowers and mushrooms work together. The Grumpy Groundhog tries to sabotage it by spreading weeds! Do you confront him or invite him to help?",},
        {"role": "Child", "text": "Why is he so grumpy?",},
        {"role": "Narrator", "text": "He's lonely because he thinks he's too different to make friends. Invite him to join or solve the problem without him?",},
        {"role": "Child", "text": "Let's invite him!",},
        {"role": "Narrator", "text": "The Groundhog reveals he's good at organizing! Should he help plan a garden diversity festival or start a club for different plants to meet?",},
    ],

    # Story 167: The Cloud School Conflict
    [
        {"role": "Child", "text": "Learning!",},
        {"role": "Narrator", "text": "At the Sky Academy, Rain Cloud and Sun Cloud fight over who's more important. Their fighting creates chaotic weather, and the Rainbow Twins can't practice their colors without steady weather. The Weather Wizard seems too busy to help. Should you talk to the clouds or help the twins?",},
        {"role": "Child", "text": "What are they fighting about?",},
        {"role": "Narrator", "text": "Rain Cloud says plants need water most, while Sun Cloud argues sunshine is more important. Meanwhile, the Mischief Wind keeps carrying their angry words to each other! Help them understand or stop Mischief Wind?",},
        {"role": "Child", "text": "Can we show them working together?",},
        {"role": "Narrator", "text": "Great idea! You could create a small garden that needs both rain and sun, but Mischief Wind threatens to blow it away. Face the wind or protect the garden?",},
        {"role": "Child", "text": "How can we face the wind?",},
        {"role": "Narrator", "text": "We could show Mischief Wind how carrying kind words instead might be more fun! Talk to wind or build garden walls?",},
        {"role": "Child", "text": "Let's talk to the wind!",},
        {"role": "Narrator", "text": "The wind reveals feeling left out of weather-making! Should we include all three in a weather dance or create a new type of weather together?",},
        {"role": "Child", "text": "What's a weather dance?",},
        {"role": "Narrator", "text": "A special dance where sun, rain, and wind work together to create beautiful weather! Dance or new weather?",},
        {"role": "Child", "text": "Weather dance!",},
        {"role": "Narrator", "text": "Their cooperation creates the most beautiful rainbow ever! Start a weather team or teach other elements to work together?",},
    ],

    # Story 168: The Music Mix-Up
    [
        {"role": "Child", "text": "Sounds!",},
        {"role": "Narrator", "text": "In the Melody Forest, Classical Cat and Rock Rabbit argue so loudly about whose music is better that they've scared away the Harmony Birds. The Silent Serpent, who hates all music, encourages their fighting. Help them appreciate different styles or find the birds?",},
        {"role": "Child", "text": "How can music be different but good?",},
        {"role": "Narrator", "text": "Like different foods can all taste good, different music can all sound nice! Meanwhile, Silent Serpent whispers more doubts. Deal with Serpent or help musicians?",},
        {"role": "Child", "text": "Let's help them understand!",},
        {"role": "Narrator", "text": "You suggest mixing their styles, but Silent Serpent sends Discord Dots to tangle their instruments! Chase away the dots or create a new song?",},
        {"role": "Child", "text": "What happens if we mix music?",},
        {"role": "Narrator", "text": "We might create an exciting new style! But first, we need to stop Silent Serpent's interference. Confront Serpent or protect the musicians?",},
        {"role": "Child", "text": "Why does Serpent hate music?",},
        {"role": "Narrator", "text": "Looking closer, you notice Serpent's unable to make sounds of their own. Help Serpent make music or continue with the mix?",},
        {"role": "Child", "text": "Let's help everyone make music!",},
        {"role": "Narrator", "text": "Working together, you discover Serpent can add beautiful rhythms by swaying! Create a band with everyone or start a music appreciation club?",},
    ],

    # Story 169: The Art Room Argument
    [
        {"role": "Child", "text": "Creating!",},
        {"role": "Narrator", "text": "In the Rainbow Room, Paint Brush and Pencil argue about which makes better art. The Eraser keeps removing their work, saying if they can't agree, no one should make art! The color sprites are hiding, afraid of the conflict. Stop Eraser or help artists understand each other?",},
        {"role": "Child", "text": "Why is Eraser being mean?",},
        {"role": "Narrator", "text": "Eraser feels left out - everyone talks about creating, but never about how important erasing and starting over can be. Help Eraser feel valued or focus on Paint Brush and Pencil?",},
        {"role": "Child", "text": "Can all three work together?",},
        {"role": "Narrator", "text": "That's thoughtful! But the Mess Monster appears, threatening to mix everything into brown! Handle the monster or continue peace-making?",},
        {"role": "Child", "text": "What does the monster want?",},
        {"role": "Narrator", "text": "The Mess Monster believes art should be chaotic and loves mixing things up! Include monster's ideas or protect the art supplies?",},
        {"role": "Child", "text": "Maybe mess can be fun sometimes!",},
        {"role": "Narrator", "text": "You suggest each artist contribute their strength: Pencil sketches, Paint Brush colors, Eraser refines, and Mess Monster adds excitement! Try this team art or have a friendly contest?",},
        {"role": "Child", "text": "Let's make art together!",},
        {"role": "Narrator", "text": "Their collaboration creates amazing new art! Start an all-styles art school or open a gallery celebrating differences?",},
    ],

    # Story 170: The Playground Problem
    [
        {"role": "Child", "text": "Playing!",},
        {"role": "Narrator", "text": "At Magic Meadow Playground, Skip the Slide and Swing the Swingset argue about which is more fun. The See-Saw Sisters feel ignored, and the Grouchy Grass below stops growing, tired of their bickering. The Rusty Rain threatens to fall and close the playground! Help toys cooperate or save playground?",},
        {"role": "Child", "text": "Why is the grass grouchy?",},
        {"role": "Narrator", "text": "Children stopped playing on the grass because of the arguing toys, and now it feels unloved. Meanwhile, Rusty Rain gets closer! Help grass first or handle rain?",},
        {"role": "Child", "text": "Let's cheer up the grass!",},
        {"role": "Narrator", "text": "As you encourage play on the grass, the Rusty Rain starts falling! The toys must choose: keep arguing or work together to protect their playground. What should they do?",},
        {"role": "Child", "text": "How can they work together?",},
        {"role": "Narrator", "text": "They could create a fun course using all the equipment, or hold an umbrella relay to keep children dry. Which might work better?",},
        {"role": "Child", "text": "Fun course sounds good!",},
        {"role": "Narrator", "text": "The toys create an amazing playground adventure! Even Rusty Rain wants to join the fun. Should they make rain-proof play stations or start a playground peace club?",},
    ],

    # Story 171: The Birthday Bash
    [
        {"role": "Child", "text": "Party!",},
        {"role": "Narrator", "text": "At the Friendship Festival, Cake and Ice Cream start fighting about who should be served first at parties. The Lonely Lemon Drop watches sadly as their argument ruins the celebration. The Party Popper threatens to cancel all future parties! Help with the dessert debate or cheer up Lemon Drop?",},
        {"role": "Child", "text": "Why is Lemon Drop sad?",},
        {"role": "Narrator", "text": "Nobody notices Lemon Drop because they're too busy choosing sides in the Cake vs. Ice Cream battle. Meanwhile, Party Popper sets a deadline! Help Lemon Drop or solve dessert crisis?",},
        {"role": "Child", "text": "Can they work together?",},
        {"role": "Narrator", "text": "Maybe! But the Sugar Rush Sprite appears, making everyone too excited to listen. Calm things down or try a new approach?",},
        {"role": "Child", "text": "Let's calm down first!",},
        {"role": "Narrator", "text": "Good choice! Once calmer, Lemon Drop suggests combining desserts. Create a dessert harmony or have separate but equal serving times?",},
        {"role": "Child", "text": "What's dessert harmony?",},
        {"role": "Narrator", "text": "All desserts working together to make super-special treats! Try harmony or keep them separate?",},
        {"role": "Child", "text": "Harmony sounds delicious!",},
        {"role": "Narrator", "text": "Everyone loves the combination treats! Start a dessert friendship club or create a new party tradition?",},
    ],

    # Story 172: The Season Switch
    [
        {"role": "Child", "text": "Weather!",},
        {"role": "Narrator", "text": "The four Season Siblings can't agree whose turn it is to visit Earth! Summer won't leave, Fall feels forgotten, Winter's getting impatient, and Spring's crying rain everywhere. The Time Keeper threatens to freeze all seasons forever! Help seasons share or confront Time Keeper?",},
        {"role": "Child", "text": "Why won't Summer leave?",},
        {"role": "Narrator", "text": "Summer thinks everyone has the most fun during their season. The Chaos Clock encourages the fighting by running fast and slow. Focus on Summer or stop the clock?",},
        {"role": "Child", "text": "How can we show Summer?",},
        {"role": "Narrator", "text": "We could show Summer the special parts of other seasons, but Chaos Clock keeps changing time! Fix the clock or help Summer understand?",},
        {"role": "Child", "text": "Let's help Summer learn!",},
        {"role": "Narrator", "text": "You begin showing Summer the beauty in all seasons. The Time Keeper offers everyone their own separate world. Accept offer or stay together?",},
        {"role": "Child", "text": "Should they stay together?",},
        {"role": "Narrator", "text": "Together they create Earth's beautiful cycle! Create a season calendar or start a weather festival?",},
        {"role": "Child", "text": "Festival with all seasons!",},
        {"role": "Narrator", "text": "The Season Siblings learn to celebrate each other! The Time Keeper joins as festival timekeeper. Should they plan more celebrations or teach others about cooperation?",},
    ],

    # Story 173: The Library Labyrinth
    [
        {"role": "Child", "text": "Books!",},
        {"role": "Narrator", "text": "In the Whispering Library, Fiction and Fact books have divided into separate towers, each claiming to be more important. The Story Searcher, a young apprentice librarian, can't complete her quest because the Book Bridge between towers is broken. Meanwhile, the Knowledge Nibbler, a bookmark who eats stories, grows stronger from their division. Help unite the books or stop the Nibbler?",},
        {"role": "Child", "text": "Why are they fighting?",},
        {"role": "Narrator", "text": "Fiction books think imagination is most important, while Fact books believe only truth matters. The Nibbler whispers doubts to both sides! Unite books or chase Nibbler?",},
        {"role": "Child", "text": "Can't we need both?",},
        {"role": "Narrator", "text": "Exactly! But as you try to explain, the Nibbler sends Confusion Commas to muddle everyone's words. Should we clear the confusion or show how facts and fiction work together?",},
        {"role": "Child", "text": "How do they work together?",},
        {"role": "Narrator", "text": "Facts can make stories more interesting, and stories can make facts easier to remember! Show them this or fight the Nibbler first?",},
        {"role": "Child", "text": "Let's show them!",},
        {"role": "Narrator", "text": "You begin creating a story using both fact and fiction. The Nibbler, weakening, offers to become a bookmark that helps blend both types of books. Accept his help or find another solution?",},
        {"role": "Child", "text": "Can we trust the Nibbler?",},
        {"role": "Narrator", "text": "If we give him a chance, he might change - like how the books are learning to work together! Trust Nibbler or not?",},
        {"role": "Child", "text": "Let's give him a chance!",},
        {"role": "Narrator", "text": "The Nibbler becomes the Bridge Builder, helping combine facts and fiction into wonderful new books! Create a mixed library or start a storytelling school?",},
    ],

    # Story 174: The Color War
    [
        {"role": "Child", "text": "Paint!",},
        {"role": "Narrator", "text": "In the Artist's Palette, Red and Blue refuse to mix, each wanting to be the most important color. Their argument causes Purple to fade away! The Gray Grumbler encourages their rivalry, hoping all colors will fade to gray. Meanwhile, Yellow watches nervously. Help the colors work together or save Purple?",},
        {"role": "Child", "text": "What happened to Purple?",},
        {"role": "Narrator", "text": "Purple exists when Red and Blue cooperate, but their fighting makes Purple disappear! Help them understand or find another way to make Purple?",},
        {"role": "Child", "text": "Let's help them understand!",},
        {"role": "Narrator", "text": "As you try to teach Red and Blue about cooperation, the Gray Grumbler sends Dull Dust to make everything gray. Clean the dust or continue the lesson?",},
        {"role": "Child", "text": "Why does Gray want no colors?",},
        {"role": "Narrator", "text": "Gray feels left out of the rainbow! We could include Gray in color mixing or focus on Red and Blue. Which first?",},
        {"role": "Child", "text": "Let's include everyone!",},
        {"role": "Narrator", "text": "Gray discovers it can help colors create beautiful shades! Red and Blue see how working together makes more possibilities. Create a color-mixing club or paint a friendship mural?",},
        {"role": "Child", "text": "What's a friendship mural?",},
        {"role": "Narrator", "text": "A big painting where all colors work together! Should we make the mural or try more color mixing?",},
        {"role": "Child", "text": "Mural sounds fun!",},
        {"role": "Narrator", "text": "All the colors create a beautiful mural, with Gray adding perfect shading! Start an art school or write a color cooperation book?",},
    ],

    # Story 175: The Melody Maze
    [
        {"role": "Child", "text": "Singing!",},
        {"role": "Narrator", "text": "In the Music Garden, High Notes and Low Notes argue so much they've created the Discord Dragon! The Rhythm Keeper has run away, taking all the garden's beat with them. The Silent Sorceress threatens to remove all sound if they can't harmonize. Save the rhythm or help the notes?",},
        {"role": "Child", "text": "Where's the Rhythm Keeper?",},
        {"role": "Narrator", "text": "They're hiding in the Echo Cave, scared of the fighting! Meanwhile, the Discord Dragon grows stronger. Find Rhythm Keeper or calm the notes?",},
        {"role": "Child", "text": "Let's find them!",},
        {"role": "Narrator", "text": "In the cave, you discover Rhythm Keeper conducting a quiet orchestra of drips and echoes. The Discord Dragon follows, offering to lead a loud-only band. Help Rhythm Keeper return or join Dragon's band?",},
        {"role": "Child", "text": "Can we combine quiet and loud?",},
        {"role": "Narrator", "text": "That's creative thinking! But the Silent Sorceress appears, threatening silence. Face her or make music first?",},
        {"role": "Child", "text": "Why does she want silence?",},
        {"role": "Narrator", "text": "She's never heard harmony before - only fighting! Show her beautiful music or keep her away?",},
        {"role": "Child", "text": "Let's show her music!",},
        {"role": "Narrator", "text": "Together with High Notes, Low Notes, and Rhythm Keeper, you create a song that changes the Discord Dragon into the Harmony Helper! Start a music school or create a magical orchestra?",},
    ],

    # Story 176: The Dream Dispute
    [
        {"role": "Child", "text": "Sleep!",},
        {"role": "Narrator", "text": "In the Land of Nod, Happy Dreams and Scary Dreams fight over who should visit children. The Nightmare Knight guards their border, making sure they stay apart. But the Sleep Sprite is fading because no one's getting proper rest! Help dreams cooperate or save the sprite?",},
        {"role": "Child", "text": "Are all scary dreams bad?",},
        {"role": "Narrator", "text": "Good question! Sometimes scary dreams help us be brave. The Nightmare Knight never thought of that! Talk to Knight or help dreams understand each other?",},
        {"role": "Child", "text": "Let's talk to the Knight!",},
        {"role": "Narrator", "text": "The Knight reveals he separates dreams because he's afraid of what might happen if they mix. Meanwhile, the Shadow Shuffler tries to steal all dreams! Handle Shadow or convince Knight?",},
        {"role": "Child", "text": "What happens if dreams mix?",},
        {"role": "Narrator", "text": "They might create exciting adventures where being brave leads to happiness! Try mixing dreams or stop Shadow first?",},
        {"role": "Child", "text": "Mix the dreams!",},
        {"role": "Narrator", "text": "The mixed dreams create amazing stories of overcoming challenges! Even Shadow Shuffler wants to join. Create a dream balance school or start a nighttime story festival?",},
    ],

    # Story 177: The Garden Games
    [
        {"role": "Child", "text": "Growing!",},
        {"role": "Narrator", "text": "At the Flower Fields Olympics, Tall Flowers and Ground Flowers compete angrily for sunshine and water. The Cloud Judge threatens to cancel the rain unless they can share. Meanwhile, the Weed Whistle encourages the fighting, hoping to take over the garden! Help flowers cooperate or stop Weed?",},
        {"role": "Child", "text": "How can they share?",},
        {"role": "Narrator", "text": "Tall Flowers could provide shade while Ground Flowers protect the soil. But Weed Whistle spreads rumors! Clear rumors or try sharing plan?",},
        {"role": "Child", "text": "Why does Weed cause trouble?",},
        {"role": "Narrator", "text": "Weed Whistle was never invited to the Olympics! We could include them properly or focus on the flowers. Which first?",},
        {"role": "Child", "text": "Include the Weed!",},
        {"role": "Narrator", "text": "Weed Whistle knows how to grow anywhere! Should they teach a survival class or join the Olympic planning team?",},
        {"role": "Child", "text": "What can they teach?",},
        {"role": "Narrator", "text": "They know how to find water and grow in tough spots! Teaching or planning?",},
        {"role": "Child", "text": "Let them teach!",},
        {"role": "Narrator", "text": "The Garden Olympics become about sharing skills instead of competing! Create a garden wisdom school or start a plant cooperation festival?",},
    ],

    # Story 178: The Space Station Situation
    [
        {"role": "Child", "text": "Stars!",},
        {"role": "Narrator", "text": "On the Rainbow Space Station, the Sun Crew and Moon Team argue about whose shift is more important. The Star Keeper threatens to turn off their light because the fighting disturbs the baby stars. The Meteor Mischief-Maker encourages chaos! Help teams collaborate or protect stars?",},
        {"role": "Child", "text": "What do the teams do?",},
        {"role": "Narrator", "text": "Sun Crew brings light and warmth, Moon Team guides dreams and tides. The Meteor wants them to fight more! Talk to teams or stop Meteor?",},
        {"role": "Child", "text": "They sound important together!",},
        {"role": "Narrator", "text": "They are! But as you explain this, Meteor sends Space Sparkles to make everyone too excited to listen. Calm the sparkles or continue talking?",},
        {"role": "Child", "text": "What makes everyone so excited?",},
        {"role": "Narrator", "text": "The sparkles make everyone want to be the best! Should we show how teamwork is better or clear the sparkles?",},
        {"role": "Child", "text": "Show them teamwork!",},
        {"role": "Narrator", "text": "You organize a day-and-night sky show where both teams shine! Even Meteor wants to join. Create a space harmony team or start an astronomy school?",},
    ],

    # Story 179: The Toy Box Trouble
    [
        {"role": "Child", "text": "Playing!",},
        {"role": "Narrator", "text": "In the Magic Toy Box, Old Toys and New Toys refuse to play together. The Toy Sorter keeps them separated, believing they can't mix. The Dust Bunny grows stronger as unused toys get forgotten. Help unite the toys or defeat Dust Bunny?",},
        {"role": "Child", "text": "Why won't they play together?",},
        {"role": "Narrator", "text": "Old Toys think they're too classic for new games, while New Toys think they're too modern for old games. Dust Bunny makes it worse! Bridge the gap or clean up Dust?",},
        {"role": "Child", "text": "Can they learn from each other?",},
        {"role": "Narrator", "text": "Brilliant! But the Toy Sorter insists on separation. Meanwhile, Dust Bunny covers everything in forgetfulness! Challenge Sorter or clear the dust?",},
        {"role": "Child", "text": "Why does Sorter separate them?",},
        {"role": "Narrator", "text": "The Sorter thinks it's keeping everything organized and safe. Should we show how mixing can be fun or keep things sorted?",},
        {"role": "Child", "text": "Show them fun together!",},
        {"role": "Narrator", "text": "Old and New Toys discover they create amazing games together! Even Dust Bunny wants to play. Start a toy mixing playground or create a game invention club?",},
    ],

    # Story 180: The Weather Warriors
    [
        {"role": "Child", "text": "Storms!",},
        {"role": "Narrator", "text": "At Weather Mountain, the Thunder Drum and Rainbow Ray argue about who makes better weather. Their fighting creates the Chaos Cloud, who mixes up all weather! The Season Keeper threatens to make it winter forever to stop the fighting. Help weather-makers cooperate or fix mixed-up weather?",},
        {"role": "Child", "text": "What's wrong with mixed weather?",},
        {"role": "Narrator", "text": "It's causing snow in summer and rain on sunny days! But maybe some mixing could be good? Explore mixing or separate weather?",},
        {"role": "Child", "text": "Can good weather mix?",},
        {"role": "Narrator", "text": "Thunder and rainbows could make amazing storm-bow shows! But Chaos Cloud sends Weather Wobbles everywhere. Fix wobbles or try mixing?",},
        {"role": "Child", "text": "Let's try good mixing!",},
        {"role": "Narrator", "text": "Working together, Thunder and Rainbow create beautiful storm-bows! Even Chaos Cloud wants to help make special weather. Start a weather art show or teach weather harmony?",},
        {"role": "Child", "text": "What's weather harmony?",},
        {"role": "Narrator", "text": "Different types of weather working together to make something new! Show or teaching?",},
        {"role": "Child", "text": "Let's make a show!",},
        {"role": "Narrator", "text": "The weather show is spectacular! Create a weather concert hall or start a climate cooperation school?",},
    ],

    # Story 181: The Playground Pals
    [
        {"role": "Child", "text": "Friends!",},
        {"role": "Narrator", "text": "On the Magic Playground, the Swing Set and Slide won't share their space. They each think they're the most fun toy! The Grumpy Grass below is tired of kids running between them and starts growing too tall. The Kind Carousel watches sadly as fewer kids come to play. Should we talk to the Swing or the Slide first?",},
        {"role": "Child", "text": "Why are they being mean?",},
        {"role": "Narrator", "text": "Each one wants to be the favorite playground toy. The Grumpy Grass makes it worse by tickling kids' feet when they try to play. Talk to the toys or calm down the grass?",},
        {"role": "Child", "text": "Let's talk to them!",},
        {"role": "Narrator", "text": "You tell them how kids love both toys for different reasons. The Grumpy Grass starts to listen too! Create a fun game using both toys or make a new playground rule?",},
        {"role": "Child", "text": "What kind of game?",},
        {"role": "Narrator", "text": "We could make a fun race where kids slide down then swing high! But the grass needs to help too. Should we include the grass or just use the toys?",},
        {"role": "Child", "text": "Include everyone!",},
        {"role": "Narrator", "text": "The grass becomes soft landing spots for jumps, and everyone has more fun together! Should we make more games or teach other playgrounds how to be friends?",},
    ],

    # Story 182: The Lunch Box Mix-Up
    [
        {"role": "Child", "text": "Food!",},
        {"role": "Narrator", "text": "Inside the Magic Lunch Box, the Sweet Snacks and Healthy Foods won't sit together. The Hungry Hiccups keep bouncing everything around, making the food fight worse! The Lunch Lady says she'll close the box forever if they can't get along. Help the foods be friends or stop the hiccups?",},
        {"role": "Child", "text": "Can foods be friends?",},
        {"role": "Narrator", "text": "They can! Like how carrots and dip make a yummy team. But the Hungry Hiccups keep messing things up! Stop hiccups or help foods?",},
        {"role": "Child", "text": "Help the foods!",},
        {"role": "Narrator", "text": "You show how foods can work together to make tasty snacks. Even the Hungry Hiccups get interested! Make a food friend party or write new lunch rules?",},
        {"role": "Child", "text": "What's a food friend party?",},
        {"role": "Narrator", "text": "Where different foods pair up to make happy snacks! Should we have a party or try something else?",},
        {"role": "Child", "text": "Party time!",},
        {"role": "Narrator", "text": "The foods have so much fun trying new pairs! Start a lunch buddy system or make a friendly food cookbook?",},
    ],

    # Story 183: The Crayon Case
    [
        {"role": "Child", "text": "Colors!",},
        {"role": "Narrator", "text": "In the Art Box, the Bright Crayons and Dark Crayons won't color together. The Paper feels sad because pictures aren't as pretty without all colors. The Mean Eraser keeps rubbing out any mixed colors! Should we help the crayons be friends or stop the eraser?",},
        {"role": "Child", "text": "Why don't they like each other?",},
        {"role": "Narrator", "text": "Each group thinks their colors are better. The Mean Eraser tells them they can't mix! Help crayons understand or talk to eraser?",},
        {"role": "Child", "text": "Let's help them understand!",},
        {"role": "Narrator", "text": "You start drawing a picture using both bright and dark colors. The Eraser comes to erase, but stops to look. Should we invite Eraser to draw or keep making our picture?",},
        {"role": "Child", "text": "Can erasers draw?",},
        {"role": "Narrator", "text": "Maybe Eraser could help make soft, cloudy backgrounds! Include Eraser or finish without them?",},
        {"role": "Child", "text": "Let's all draw together!",},
        {"role": "Narrator", "text": "Everyone works together to make the best picture ever! Start an art club or make a color-sharing book?",},
    ],

    # Story 184: The Pet Palace Problem
    [
        {"role": "Child", "text": "Animals!",},
        {"role": "Narrator", "text": "At the Magic Pet Palace, the Dogs and Cats have a big fight about the best playing spot. The Worried Mouse watches from the corner as the Sleepy Sun threatens to set early, ending playtime for everyone! Help pets share or keep the sun up?",},
        {"role": "Child", "text": "Why can't they share?",},
        {"role": "Narrator", "text": "Each group wants the sunny spot by the window. Mouse knows a way they could all fit but is too shy to speak up! Talk to Mouse or help pets?",},
        {"role": "Child", "text": "Let's talk to Mouse!",},
        {"role": "Narrator", "text": "Mouse suggests making different play times or building a bigger spot. The Sun peeks in, interested! Which idea should we try?",},
        {"role": "Child", "text": "How can we make it bigger?",},
        {"role": "Narrator", "text": "We could move some toys to make more room! Make the spot bigger or take turns?",},
        {"role": "Child", "text": "Bigger spot for everyone!",},
        {"role": "Narrator", "text": "The pets love their new big play space! Start a pet sharing club or make more play spots?",},
    ],

    # Story 185: The Bedtime Bunch
    [
        {"role": "Child", "text": "Sleep!",},
        {"role": "Narrator", "text": "In the Sleepy Room, the Night Light and Dark Corner argue about bedtime. The Tired Teddy Bear can't sleep with all the noise! The Dream Door might close if everyone doesn't calm down. Help make peace or save bedtime?",},
        {"role": "Child", "text": "Why are they fighting?",},
        {"role": "Narrator", "text": "Night Light thinks rooms should be a little bright, Dark Corner wants total darkness. Teddy likes both! Help them understand or help Teddy?",},
        {"role": "Child", "text": "Can we help everyone?",},
        {"role": "Narrator", "text": "Good thinking! Maybe some spots can be bright and others dark. But the Dream Door is closing! Fix the door or help them share?",},
        {"role": "Child", "text": "How do we save dreams?",},
        {"role": "Narrator", "text": "We need everyone to work together to keep the door open! Make a bedtime team or find a new solution?",},
        {"role": "Child", "text": "Let's work together!",},
        {"role": "Narrator", "text": "They create the perfect room with both light and dark spaces! Make a bedtime helper club or write a sleepy time story?",},
    ],

    # Story 186: The Garden Gang
    [
        {"role": "Child", "text": "Plants!",},
        {"role": "Narrator", "text": "In the Happy Garden, the Tall Plants and Short Plants fight over water and sun. The Grumpy Watering Can refuses to help either side! The Sad Sunshine might go away if they don't stop fighting. Should we talk to the plants or the watering can?",},
        {"role": "Child", "text": "Are they thirsty?",},
        {"role": "Narrator", "text": "Yes! But Watering Can won't help until they stop fighting. Meanwhile, Sunshine gets sadder! Help plants share or cheer up Sunshine?",},
        {"role": "Child", "text": "Let's help them share!",},
        {"role": "Narrator", "text": "You show how tall plants can give shade while short plants keep the ground nice. Watering Can gets interested! Make a garden team or write plant sharing rules?",},
        {"role": "Child", "text": "What's a garden team?",},
        {"role": "Narrator", "text": "Where all plants work together to make a happy garden! Team or rules?",},
        {"role": "Child", "text": "Garden team sounds fun!",},
        {"role": "Narrator", "text": "The garden becomes beautiful with everyone helping! Start a plant friend program or make a garden party?",},
    ],

    # Story 187: The Game Night
    [
        {"role": "Child", "text": "Playing!",},
        {"role": "Narrator", "text": "During Magic Game Night, Board Games and Card Games won't play together. The Lonely Dice feels left out, and the Fun Fairy might cancel game night! Help the games become friends or save game night?",},
        {"role": "Child", "text": "Why won't they play together?",},
        {"role": "Narrator", "text": "Each thinks their games are more fun. Dice wants to play with both! Help Dice make friends or help games share?",},
        {"role": "Child", "text": "Let's help Dice!",},
        {"role": "Narrator", "text": "Dice suggests making a new game using cards and boards together. The Fun Fairy gets excited! Try the new game or keep them separate?",},
        {"role": "Child", "text": "New game sounds cool!",},
        {"role": "Narrator", "text": "Everyone loves the new game! Should we make more mixed games or have a game teaching party?",},
        {"role": "Child", "text": "What's a game teaching party?",},
        {"role": "Narrator", "text": "Where games teach each other how to play together! Party or new games?",},
        {"role": "Child", "text": "Party time!",},
        {"role": "Narrator", "text": "The party is super fun! Create a game mixing club or write a book of new games?",},
    ],

    # Story 188: The Weather Friends
    [
        {"role": "Child", "text": "Rain!",},
        {"role": "Narrator", "text": "In Sky Town, the Rain Cloud and Sun Beam have a big fight. The Rainbow can't appear because they won't work together! The Wind Whistle threatens to blow everyone away if they don't stop. Help make weather friends or save the rainbow?",},
        {"role": "Child", "text": "How do we make a rainbow?",},
        {"role": "Narrator", "text": "Rain and sun need to work together! But Wind Whistle keeps stirring up trouble. Stop the wind or help sun and rain?",},
        {"role": "Child", "text": "Let's help them work together!",},
        {"role": "Narrator", "text": "You show how rain and sun make beautiful rainbows. Even Wind Whistle stops to look! Make more rainbows or start a weather friend club?",},
        {"role": "Child", "text": "Can we do both?",},
        {"role": "Narrator", "text": "Great idea! The weather team could make rainbow shows! Start shows now or plan more ideas?",},
        {"role": "Child", "text": "Rainbow shows!",},
        {"role": "Narrator", "text": "Everyone loves working together to make pretty skies! Make a weather party or teach other clouds and suns?",},
    ],

    # Story 189: The Sharing Sandbox
    [
        {"role": "Child", "text": "Sand!",},
        {"role": "Narrator", "text": "In the Magic Sandbox, the Toy Trucks and Shovels won't share the space. The Sand Castle feels sad watching them fight. Should we help them learn to play together?",},
        {"role": "Child", "text": "Why won't they share?",},
        {"role": "Narrator", "text": "Each wants the best spot to play. The Sand Castle knows they could have more fun together! Talk to trucks or shovels first?",},
        {"role": "Child", "text": "Let's talk to both!",},
        {"role": "Narrator", "text": "Good idea! You show how trucks can move sand while shovels dig tunnels. Make a fun project together?",},
        {"role": "Child", "text": "Yes, let's build!",},
        {"role": "Narrator", "text": "Working together, they build the biggest sand castle ever! Start a sandbox club or make sharing rules?",},
    ],

    # Story 190: The Cookie Team
    [
        {"role": "Child", "text": "Baking!",},
        {"role": "Narrator", "text": "The Chocolate Chips and Rainbow Sprinkles argue about who makes cookies better. The Sad Cookie Jar might close forever! Help them work together?",},
        {"role": "Child", "text": "Can they mix?",},
        {"role": "Narrator", "text": "They could make super special cookies together! Try mixing or keep them apart?",},
        {"role": "Child", "text": "Mix them!",},
        {"role": "Narrator", "text": "The new cookies are yummy! Everyone loves them. Make more cookie friends or share the recipe?",},
    ],

    # Story 191: The Paint Pals
    [
        {"role": "Child", "text": "Art!",},
        {"role": "Narrator", "text": "Red Paint and Blue Paint won't mix. The Empty Canvas wants to see purple! Help the paints become friends?",},
        {"role": "Child", "text": "How do we make purple?",},
        {"role": "Narrator", "text": "Red and Blue need to dance together! Should we show them how?",},
        {"role": "Child", "text": "Yes, let's help!",},
        {"role": "Narrator", "text": "They make beautiful purple! Start a paint mixing party or make a color book?",},
    ],

    # Story 192: The Block Builders
    [
        {"role": "Child", "text": "Blocks!",},
        {"role": "Narrator", "text": "The Square Blocks and Round Blocks think they can't build together. The Tiny Tower wants to grow bigger! Help them build?",},
        {"role": "Child", "text": "Can they work together?",},
        {"role": "Narrator", "text": "All shapes can make cool buildings! Try building or play alone?",},
        {"role": "Child", "text": "Build together!",},
        {"role": "Narrator", "text": "They make an amazing castle! Make more buildings or teach other blocks?",},
    ],

    # Story 193: The Puzzle Friends
    [
        {"role": "Child", "text": "Puzzles!",},
        {"role": "Narrator", "text": "The Edge Pieces and Middle Pieces won't work together. The Picture can't be finished! Should we help?",},
        {"role": "Child", "text": "How do puzzles work?",},
        {"role": "Narrator", "text": "All pieces need each other to make the picture whole! Try fitting together?",},
        {"role": "Child", "text": "Let's help them fit!",},
        {"role": "Narrator", "text": "The puzzle looks beautiful when complete! Make more puzzles or share with friends?",},
    ],

    # Story 194: The Dance Party
    [
        {"role": "Child", "text": "Dancing!",},
        {"role": "Narrator", "text": "Fast Dance and Slow Dance won't share the music. The Lonely Beat feels sad! Dance together?",},
        {"role": "Child", "text": "Can they mix dances?",},
        {"role": "Narrator", "text": "They could make a new dance together! Try mixing or stay apart?",},
        {"role": "Child", "text": "Mix the dances!",},
        {"role": "Narrator", "text": "Their new dance is super fun! Have a dance party or make a dance club?",},
    ],

    # Story 195: The Tree House Tale
    [
        {"role": "Child", "text": "Climbing!",},
        {"role": "Narrator", "text": "The Upper Branch and Lower Branch won't let kids play on both. The Tree House feels lonely! Help them share?",},
        {"role": "Child", "text": "Why can't kids use both?",},
        {"role": "Narrator", "text": "The branches think they're too different! Show them how to be friends?",},
        {"role": "Child", "text": "Yes, let's help!",},
        {"role": "Narrator", "text": "Now kids can play everywhere! Make more tree houses or have a tree party?",},
    ],

    # Story 196: The Ball Game
    [
        {"role": "Child", "text": "Sports!",},
        {"role": "Narrator", "text": "The Big Ball and Small Ball won't play together. The Lonely Net waits for a game! Play together?",},
        {"role": "Child", "text": "What games can they play?",},
        {"role": "Narrator", "text": "They could make a fun new game! Try playing or stay separate?",},
        {"role": "Child", "text": "Let's play!",},
        {"role": "Narrator", "text": "Their new game is the best ever! Make a sports club or teach others?",},
    ],

    # Story 197: The Music Makers
    [
        {"role": "Child", "text": "Songs!",},
        {"role": "Narrator", "text": "The Loud Drum and Quiet Flute think they can't play together. The Song feels incomplete! Make music?",},
        {"role": "Child", "text": "Can they make nice music?",},
        {"role": "Narrator", "text": "Different instruments make better music together! Try playing?",},
        {"role": "Child", "text": "Let's make music!",},
        {"role": "Narrator", "text": "Their song is beautiful! Have a concert or write more songs?",},
    ],

    # Story 198: The Snack Stack
    [
        {"role": "Child", "text": "Snacks!",},
        {"role": "Narrator", "text": "The Salty Snacks and Sweet Snacks won't sit together. The Hungry Tummy is sad! Help them mix?",},
        {"role": "Child", "text": "What happens if they mix?",},
        {"role": "Narrator", "text": "They might make awesome new flavors! Try mixing or keep apart?",},
        {"role": "Child", "text": "Mix them up!",},
        {"role": "Narrator", "text": "The new snack mix is delicious! Make more snacks or share with friends?",},
    ],

    # Story 199: The Story Stars
    [
        {"role": "Child", "text": "Books!",},
        {"role": "Narrator", "text": "The Happy Tales and Scary Stories won't share their shelf. The Reading Light wants both! Read together?",},
        {"role": "Child", "text": "Are scary stories bad?",},
        {"role": "Narrator", "text": "All stories can be good! Some teach us to be brave. Try both kinds?",},
        {"role": "Child", "text": "Let's read both!",},
        {"role": "Narrator", "text": "The mixed stories are exciting! Make a story club or write new tales?",},
    ],

    # Story 200: The Helper Heroes
    [
        {"role": "Child", "text": "Helping!",},
        {"role": "Narrator", "text": "The Big Helpers and Little Helpers think they can't work together. The Big Job needs everyone! Work together?",},
        {"role": "Child", "text": "Can everyone help?",},
        {"role": "Narrator", "text": "All helpers are important! Try helping together?",},
        {"role": "Child", "text": "Yes, let's all help!",},
        {"role": "Narrator", "text": "The job is done fast with everyone helping! Make a helper club or find more jobs?",},
    ],

    # Story 201: The Broken Rainbow Machine
    [
        {"role": "Child", "text": "Fix it!",},
        {"role": "Narrator", "text": "The Rainbow Machine stopped working! It only makes gray clouds now. The Color Collector suggests using art supplies, while the Weather Wizard wants to use rain drops. What should we try first?",},
        {"role": "Child", "text": "What's wrong with it?",},
        {"role": "Narrator", "text": "Looking closer, you see it needs color power! We could mix paints, collect flower colors, or gather sunset light. Which idea sounds good?",},
        {"role": "Child", "text": "Let's try all of them!",},
        {"role": "Narrator", "text": "Smart thinking! Paint makes red, flowers make yellow, and sunset makes orange. But we still need blue and purple! Where could we find these colors?",},
        {"role": "Child", "text": "Where does blue come from?",},
        {"role": "Narrator", "text": "Maybe from the ocean or the sky! And purple might be in morning mist. Should we collect sky-blue or ocean-blue?",},
        {"role": "Child", "text": "Let's try the ocean!",},
        {"role": "Narrator", "text": "The rainbow is almost fixed! Now it needs one more thing - happiness to power it. Should we throw a color party or make people laugh?",},
    ],

    # Story 202: The Lost Song Bird
    [
        {"role": "Child", "text": "Music!",},
        {"role": "Narrator", "text": "A little bird lost its song! We can help it make a new one. The Music Maker has instruments, and the Forest Friends know animal sounds. How should we help?",},
        {"role": "Child", "text": "Can birds learn new songs?",},
        {"role": "Narrator", "text": "Yes! We could mix wind chimes with cricket chirps, or blend river sounds with bell rings. Which sounds better?",},
        {"role": "Child", "text": "How do we mix sounds?",},
        {"role": "Narrator", "text": "We could ask the Echo Cave to bounce sounds together! Or the Mixing Bowl could stir them up. Which helper should we use?",},
        {"role": "Child", "text": "Let's use the cave!",},
        {"role": "Narrator", "text": "Good choice! Now should the bird practice one big song or learn small song pieces?",},
    ],

    # Story 203: The Garden Puzzle
    [
        {"role": "Child", "text": "Growing!",},
        {"role": "Narrator", "text": "The garden won't grow because the soil is too hard! The Tool Team wants to dig it up, but the Bug Brigade has another idea - make tiny tunnels! What should we try?",},
        {"role": "Child", "text": "What else could help?",},
        {"role": "Narrator", "text": "We could ask the Rain Dancers to soften the soil, or the Worm Family to wiggle through it. Or maybe try both? What's your plan?",},
        {"role": "Child", "text": "Can everyone work together?",},
        {"role": "Narrator", "text": "Great idea! The bugs make tunnels, worms wiggle through, rain softens everything, and tools help plant! Should we add anything else?",},
        {"role": "Child", "text": "What about music?",},
        {"role": "Narrator", "text": "The plants might grow better with happy songs! Have a garden concert or make a growing drum circle?",},
    ],

    # Story 204: The Shy Sunshine
    [
        {"role": "Child", "text": "Light!",},
        {"role": "Narrator", "text": "The Sun is too shy to shine bright today! The Moon offers to stay and help, while the Stars want to throw a confidence party. How can we help Sun feel brave?",},
        {"role": "Child", "text": "Why is Sun shy?",},
        {"role": "Narrator", "text": "Sun thinks its light might be too bright! We could make friendly shadows, dance with light beams, or create a light show. What's the best idea?",},
        {"role": "Child", "text": "Let's make a show!",},
        {"role": "Narrator", "text": "For the show, we need clouds for spotlight shapes and raindrops for sparkles. What else could make Sun feel special?",},
        {"role": "Child", "text": "Can we add colors?",},
        {"role": "Narrator", "text": "Yes! Should we make rainbows or paint the clouds in sunny colors?",},
    ],

    # Story 205: The Flying Book
    [
        {"role": "Child", "text": "Reading!",},
        {"role": "Narrator", "text": "A magic book can't stop flying around the library! The Bookmark Brothers want to catch it with nets, but the Story Sprites think we should tell it a story. What's your idea?",},
        {"role": "Child", "text": "What makes it fly?",},
        {"role": "Narrator", "text": "It's full of exciting adventure stories! We could give it a quiet story, make it a comfy reading nest, or find it a friend. Which should we try?",},
        {"role": "Child", "text": "Let's make a nest!",},
        {"role": "Narrator", "text": "What should we put in the nest? Soft bookmarks, story pillows, or both?",},
        {"role": "Child", "text": "Both please!",},
        {"role": "Narrator", "text": "The book loves its cozy nest! Should we make more nests or start a flying book club?",},
    ],

    # Story 206: The Mixed-Up Map
    [
        {"role": "Child", "text": "Lost!",},
        {"role": "Narrator", "text": "The treasure map got scrambled in the washing machine! Its X marks spots everywhere now. The Direction Dragon and Compass Cat both want to help fix it. Who should we ask?",},
        {"role": "Child", "text": "Can we follow all the X's?",},
        {"role": "Narrator", "text": "That's a new idea! We could make it a multi-treasure hunt, follow the soapy trail, or draw a new map. What sounds fun?",},
        {"role": "Child", "text": "Multi-treasure hunt!",},
        {"role": "Narrator", "text": "Should we hide different treasures at each X or make clues that connect them all?",},
        {"role": "Child", "text": "Connect them all!",},
        {"role": "Narrator", "text": "Now we have a super fun treasure game! Make more mixed-up maps or invite friends to play?",},
    ],

    # Story 207: The Bubble Trouble
    [
        {"role": "Child", "text": "Bubbles!",},
        {"role": "Narrator", "text": "The bubble machine is making square bubbles! Round Bubble wants to fix them, but Creative Crab thinks square bubbles might be cool. What do you think?",},
        {"role": "Child", "text": "Are square bubbles bad?",},
        {"role": "Narrator", "text": "Maybe they're just different! We could make bubble buildings, box forts, or try new shapes. Which sounds best?",},
        {"role": "Child", "text": "What other shapes?",},
        {"role": "Narrator", "text": "We could try star bubbles or heart bubbles! Which shape first?",},
        {"role": "Child", "text": "Star bubbles!",},
        {"role": "Narrator", "text": "The sky fills with star bubbles! Make a shape bubble show or write a bubble shape book?",},
    ],

    # Story 208: The Sleepy Moon
    [
        {"role": "Child", "text": "Night!",},
        {"role": "Narrator", "text": "The Moon is too sleepy to shine! Star Team wants to shine brighter, but Night Owl suggests finding Moon a wake-up snack. How else could we help?",},
        {"role": "Child", "text": "What wakes up the Moon?",},
        {"role": "Narrator", "text": "Maybe moon cookies, starlight juice, or cloud pillows! We could also sing wake-up songs. What should we try?",},
        {"role": "Child", "text": "Let's make moon cookies!",},
        {"role": "Narrator", "text": "Should we shape them like stars or crescents? Or make both?",},
        {"role": "Child", "text": "Both kinds!",},
        {"role": "Narrator", "text": "Moon loves the cookies and wakes up bright! Start a moon bakery or share cookie recipes?",},
    ],

    # Story 209: The Dancing Plants
    [
        {"role": "Child", "text": "Moving!",},
        {"role": "Narrator", "text": "The garden plants want to dance but can't move their roots! Wise Wind offers to help them sway, while Music Maker suggests special growing songs. Any other ideas?",},
        {"role": "Child", "text": "How can plants dance?",},
        {"role": "Narrator", "text": "We could make leaf instruments, teach them wave moves, or create a breeze ballet! Which first?",},
        {"role": "Child", "text": "Leaf instruments!",},
        {"role": "Narrator", "text": "Should we make leaf drums or petal whistles? Maybe both?",},
        {"role": "Child", "text": "Let's use all the leaves!",},
        {"role": "Narrator", "text": "The garden becomes a dancing orchestra! Have a plant party or teach other gardens?",},
    ],

    # Story 210: The Paint Mixer
    [
        {"role": "Child", "text": "Colors!",},
        {"role": "Narrator", "text": "The Paint Mixer is stuck making only brown! Color Wizard wants to use magic, but Artist Ant thinks we can fix it with teamwork. What should we do?",},
        {"role": "Child", "text": "How do we fix colors?",},
        {"role": "Narrator", "text": "We could wash it with rainbow water, add happy thoughts, or collect new colors from nature. Which way sounds best?",},
        {"role": "Child", "text": "Where do we find colors?",},
        {"role": "Narrator", "text": "Flowers have colors, sunsets make colors, and rainbows drop colors! Where should we look first?",},
        {"role": "Child", "text": "Let's check the flowers!",},
        {"role": "Narrator", "text": "The Paint Mixer works again! Create a color recipe book or start a paint mixing school?",},
    ],
]

## Number of stories

In [6]:
str(len(story_conversations))

'200'

In [7]:
def split_into_substories(story_conversations):
    """
    Split story conversations into progressively longer substories.
    Returns a single flat list of all substories.
    Each substory contains complete child-narrator interaction pairs.
    """
    all_substories = []

    for story in story_conversations:
        # Group conversations into child-narrator pairs
        pairs = []
        current_pair = []

        for message in story:
            current_pair.append(message)
            if len(current_pair) == 2:
                pairs.append(current_pair)
                current_pair = []

        # Create progressively longer substories
        for i in range(1, len(pairs) + 1):
            substory = []
            for pair in pairs[:i]:
                substory.extend(pair)
            all_substories.append(substory)

    return all_substories

# Split stories and print example output
substories = split_into_substories(story_conversations)

In [8]:
# Print the total number of substories
print(f"\nTotal number of substories: {len(substories)}")


Total number of substories: 925


In [9]:
substories[0]

[{'role': 'Child', 'text': "What's a butterfly?"},
 {'role': 'Narrator',
  'text': "Great question! A butterfly is a beautiful insect with colorful wings. They start as caterpillars and then change into butterflies. Would you like to hear more about butterflies or continue with Mei's story?"}]

In [ ]:
substories[1]

[{'role': 'Child', 'text': "What's a butterfly?"},
 {'role': 'Narrator',
  'text': "Great question! A butterfly is a beautiful insect with colorful wings. They start as caterpillars and then change into butterflies. Would you like to hear more about butterflies or continue with Mei's story?"},
 {'role': 'Child', 'text': 'Tell me more about butterflies!'},
 {'role': 'Narrator',
  'text': "Butterflies have four wings covered in tiny scales. They use their long tongues, called proboscises, to drink nectar from flowers. Butterflies are important because they help pollinate plants. Now, shall we return to Mei's story?"}]

In [ ]:
story_conversations = substories

## Vertex AI Experiment (failed)

In [ ]:
def convert_and_save_data_for_vertexai(conversations, output_file='vertexai_training_data.jsonl'):
    line_count = 0
    with open(output_file, 'w') as f:
        for conversation in conversations:
            context = ""
            for i in range(0, len(conversation), 2):
                child_message = conversation[i]['text']
                narrator_message = conversation[i+1]['text'] if i+1 < len(conversation) else None

                if narrator_message:
                    formatted_data = {
                        "contents": [
                            {
                                "role": "user",
                                "parts": [{"text": context + "Child: " + child_message}]
                            },
                            {
                                "role": "model",
                                "parts": [{"text": narrator_message}]
                            }
                        ]
                    }

                    f.write(json.dumps(formatted_data) + '\n')
                    line_count += 1

                    context += f"Child: {child_message}\nNarrator: {narrator_message}\n"

    print(f"Data has been saved to {output_file}")
    print(f"Total number of lines in the JSONL file: {line_count}")
    return line_count

# Usage (assuming you have your conversations data)
total_lines = convert_and_save_data_for_vertexai(story_conversations)

# Verify the output
def print_first_few_lines(file_path, num_lines=3):
    with open(file_path, 'r') as f:
        for _ in range(num_lines):
            print(f.readline().strip())

print("\nFirst few lines of the JSONL file:")
print_first_few_lines('vertexai_training_data.jsonl')

# Double-check the line count
with open('vertexai_training_data.jsonl', 'r') as f:
    line_count = sum(1 for _ in f)
print(f"\nVerified number of lines in the JSONL file: {line_count}")
assert line_count == total_lines, "Line count mismatch!"

Data has been saved to vertexai_training_data.jsonl
Total number of lines in the JSONL file: 483

First few lines of the JSONL file:
{"contents": [{"role": "user", "parts": [{"text": "Child: What's a butterfly?"}]}, {"role": "model", "parts": [{"text": "Great question! A butterfly is a beautiful insect with colorful wings. They start as caterpillars and then change into butterflies. Would you like to hear more about butterflies or continue with Mei's story?"}]}]}
{"contents": [{"role": "user", "parts": [{"text": "Child: What's a butterfly?\nNarrator: Great question! A butterfly is a beautiful insect with colorful wings. They start as caterpillars and then change into butterflies. Would you like to hear more about butterflies or continue with Mei's story?\nChild: Tell me more about butterflies!"}]}, {"role": "model", "parts": [{"text": "Butterflies have four wings covered in tiny scales. They use their long tongues, called proboscises, to drink nectar from flowers. Butterflies are impor

### Notes:
1. 113 stories gets transformed into 483 training examples for the LLM

## to disk

In [10]:
flattened_data = []
for story_id, conversation in enumerate(story_conversations):
    for turn in conversation:
        flattened_data.append({
            "story_id": story_id,
            "role": turn["role"],
            "text": turn["text"]
        })

df = pd.DataFrame(flattened_data)
dataset = Dataset.from_pandas(df)
dataset.save_to_disk("children_book_conversations")

Saving the dataset (0/1 shards):   0%|          | 0/1850 [00:00<?, ? examples/s]

In [12]:
!pwd
!ls

/content
children_book_conversations  drive  sample_data


# Next cell takes 1 minute (2b) to load on paid colab L4 or A100 (T4 doesn't have enough memory)

# 9b & 27b don't work - run out of memory

In [13]:
# Part 2: Load the model and tokenizer

# Set your Hugging Face token
os.environ["HUGGINGFACE_TOKEN"] = userdata.get('huggingface_api_token_2') # george -'HF_KEY')

# Login to Hugging Face
login(token=os.environ["HUGGINGFACE_TOKEN"])

model_name = "google/gemma-2-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Adjust tokenizer padding settings for longer sequences
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [31]:
import traceback

# Part 3: Define LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)

# Part 4: Apply LoRA to the model
peft_model = get_peft_model(model, lora_config)

# Part 5: Load and preprocess the dataset
dataset = load_from_disk("children_book_conversations")

def preprocess_function(examples):
    print("Preprocess")
    # Combine all turns in a story into one text
    stories = {}
    for story_id, role, text in zip(examples['story_id'], examples['role'], examples['text']):
        if story_id not in stories:
            stories[story_id] = []
        stories[story_id].append(f"{role}: {text}")

    # Process each story
    input_ids_list = []
    attention_mask_list = []
    labels_list = []

    for story in stories.values():
        full_text = " ".join(story)
        encoded = tokenizer(full_text, truncation=True, padding="max_length", max_length=recommended_max_length)
        input_ids_list.append(encoded['input_ids'])
        print("full_text=", len(encoded['input_ids']), "--", full_text)
        attention_mask_list.append(encoded['attention_mask'])
        labels_list.append(encoded['input_ids'].copy())  # For causal language modeling, labels are the same as input_ids

    try:
      print("input_ids_list=", len(input_ids_list))
    except:
      traceback.print_exc()

    return {
        'input_ids': input_ids_list,
        'attention_mask': attention_mask_list,
        'labels': labels_list
    }

# Apply the preprocessing function
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)
#print("tokenized_dataset=", len(tokenized_dataset))
#print("sample=", len(tokenized_dataset[0]['input_ids']))
# Set the format of the dataset to PyTorch tensors
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# # Verify the dataset structure
# print(tokenized_dataset[0])
# print(tokenized_dataset[0]['input_ids'].shape)
# print(tokenized_dataset[0]['attention_mask'].shape)
# print(tokenized_dataset[0]['labels'].shape)

Map:   0%|          | 0/1850 [00:00<?, ? examples/s]

Preprocess
full_text= 1006 -- Child: What's a butterfly? Narrator: Great question! A butterfly is a beautiful insect with colorful wings. They start as caterpillars and then change into butterflies. Would you like to hear more about butterflies or continue with Mei's story? Child: Tell me more about butterflies! Narrator: Butterflies have four wings covered in tiny scales. They use their long tongues, called proboscises, to drink nectar from flowers. Butterflies are important because they help pollinate plants. Now, shall we return to Mei's story? Child: Yes, let's continue the story. Narrator: Mei flew to a flower. She saw two paths: one led to a pond, the other to a field of flowers. Where should Mei go? Child: Let's go to the pond! Narrator: Mei fluttered to the pond. There, she met Froggy the frog. 'Hello!' said Froggy. 'Want to play leap frog?' Should Mei play with Froggy or keep exploring? Child: Mei should play with Froggy! Narrator: Mei and Froggy had a wonderful time playing l

In [15]:
def analyze_story_lengths(story_conversations, model_name=model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    story_lengths = []
    for story in story_conversations:
        full_story = " ".join([f"{turn['role']}: {turn['text']}" for turn in story])
        length = len(tokenizer.encode(full_story))
        story_lengths.append(length)

    max_length = max(story_lengths)
    p95_length = np.percentile(story_lengths, 95)
    median_length = np.median(story_lengths)

    print(f"Maximum story length: {max_length} tokens")
    print(f"95th percentile length: {p95_length:.0f} tokens")
    print(f"Median story length: {median_length:.0f} tokens")

    return max_length, p95_length, median_length

# Analyze your story conversations
max_length, p95_length, median_length = analyze_story_lengths(story_conversations)

# Recommend a max_length based on the analysis
recommended_max_length = min(8192, max(256, int(p95_length * 1.3)))
print(f"\nRecommended max_length: {recommended_max_length}")

Maximum story length: 1006 tokens
95th percentile length: 343 tokens
Median story length: 156 tokens

Recommended max_length: 446


In [16]:
recommended_max_length = 1006

# Training takes about 10 minutes for ~1000 stories

In [32]:
# Part 6: Set up Trainer
num_train_epochs = 3

training_args = TrainingArguments(
    output_dir="./results",  # Directory where the model checkpoints and logs will be saved
    num_train_epochs=num_train_epochs,  # Total number of training cycles. 3 is usually sufficient with 1000 examples; 10 for 100 examples
    per_device_train_batch_size=1,  # Number of samples processed on each device per batch. Reduced for larger sequences
    gradient_accumulation_steps=4,  # Number of steps to accumulate gradients before updating. Compensates for small batch size
    warmup_steps=500,  # Number of steps for learning rate warmup. Helps stabilize training in the beginning
    weight_decay=0.01,  # L2 regularization factor. Helps prevent overfitting
    logging_dir="./logs",  # Directory where training logs will be stored
    fp16=True,  # Enables mixed precision training. Can speed up training and reduce memory usage
    learning_rate=2e-4,  # Initial learning rate for the optimizer. Adjust based on your specific model and task
    save_steps=100,  # Save a checkpoint every 100 steps
    eval_steps=100,  # Perform evaluation every 100 steps
    report_to=None # don't integrate with WANDB
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Part 7: Train the model
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


TrainOutput(global_step=150, training_loss=0.8154279073079427, metrics={'train_runtime': 116.3727, 'train_samples_per_second': 5.182, 'train_steps_per_second': 1.289, 'total_flos': 7337779271884800.0, 'train_loss': 0.8154279073079427, 'epoch': 2.9850746268656714})

In [33]:
# Part 8a: Generate text using the fine-tuned model
input_text = "Tell me a story about a goblin"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(peft_model.device)

with torch.no_grad():
    output = peft_model.generate(input_ids, max_length=recommended_max_length, num_return_sequences=1, do_sample=True, temperature=0.7)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated text:")
print(generated_text)

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Generated text:
Tell me a story about a goblin who dreams of being an architect.

Filbert was no ordinary goblin. While his kin preferred digging dirt and hoarding shiny trinkets, Filbert was captivated by towering castles and intricate bridges. He dreamed of designing amazing structures, not digging them in the ground.

One day, a travelling gnome, Professor Tinkerbell, arrived in Filbert's village. Filbert, bursting with excitement, approached him, proposing a dream collaboration: designing a goblin city, complete with workshops and communal gardens. Professor Tinkerbell was amused, but his heart was touched by Filbert's passion. He agreed to help, offering to teach him about architecture and building techniques.

Over the next few years, Filbert learned everything he could. He studied ancient cave drawings, experimented with different materials, and built scale models of his ideas. The village's other goblins were initially skeptical, but Filbert's enthusiasm was contagious. Some ev

In [34]:
# Part 8b: Generate text using the fine-tuned model
input_text = "Banana"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(peft_model.device)

with torch.no_grad():
    output = peft_model.generate(input_ids, max_length=recommended_max_length, num_return_sequences=1, do_sample=True, temperature=0.7)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated text:")
print(generated_text)

Generated text:
Banana: A tropical fruit that is both healthy and delicious. The banana is a good source of potassium, which is important for maintaining electrolyte balance. Bananas can also be used to make smoothies, bread, and other baked goods.

The Banana Plant: The banana plant is a type of herbaceous flowering plant that produces edible bananas. The banana plant is a tropical plant that grows in warm climates. It is typically found in clusters of banana trees. The banana plant can grow up to 20 feet tall.

Banana Benefits: The banana is a nutritious fruit with a number of health benefits. These include:
* Potassium: Potassium is an electrolyte that helps to regulate muscle contractions, nerve function, and fluid balance.
* Fiber: Banana provides a good source of dietary fiber, which is essential for digestive health.
* Vitamin B6: Vitamin B6 is important for immune function, brain development, and energy production.
* Magnesium: Magnesium is needed for over 300 processes in the 

# Connect to google drive

In [ ]:
model_type = model_name.replace("google/", "huggingface_")

# Saved model and weights
lora_save_path = "/content/drive/My Drive/NEW_Kaggle_X_George_and_Victor_2024/Weights/" + model_type + "_" + str(num_train_epochs) + "_epochs_" + str(len(story_conversations)) + "_stories_lora_weights"

lora_save_path

# Save lora adapter weights google drive

In [ ]:
# Create directory if it doesn't exist
os.makedirs(os.path.dirname(lora_save_path), exist_ok=True)

peft_model.save_pretrained(lora_save_path)

# Load Model

In [ ]:
def load_lora_model(base_model_name, lora_weights_path):
    # Load the base model
    base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

    # Load the LoRA configuration
    peft_config = PeftConfig.from_pretrained(lora_weights_path)

    # Load the LoRA model
    model = PeftModel.from_pretrained(base_model, lora_weights_path)

    # Merge LoRA weights with base model
    model = model.merge_and_unload()

    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)

    return model, tokenizer

# Usage
# Set your Hugging Face token
os.environ["HUGGINGFACE_TOKEN"] = userdata.get('HF_KEY')

# Login to Hugging Face
login(token=os.environ["HUGGINGFACE_TOKEN"])

model, tokenizer = load_lora_model(model_name, lora_save_path)

In [ ]:
# Now you can use the model for inference
input_text = "Tell me a story about a dragon"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

with torch.no_grad():
    output = model.generate(input_ids, max_length=200, num_return_sequences=1, do_sample=True, temperature=0.7)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated text:")
print(generated_text)

# Conversational Bot

In [ ]:
# Ensure model and all its parameters are on CUDA
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = model.to(device)
for param in model.parameters():
    param.data = param.data.to(device)

# Create a TextGenerationPipeline
pipe = TextGenerationPipeline(
    model=model,
    tokenizer=tokenizer,
    device=device,
    max_length=500,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    no_repeat_ngram_size=3,
    repetition_penalty=1.2,
    truncation=True
)

# Define a prompt template
template = """You are a storytelling chatbot that engages in interactive storytelling with children.
Your answers should be one sentence responses between 3 to 50 words depending on what you were asked.
After the user has interacted with you 3 times, you should end the story.
Current conversation:
{history}
Child: {input}
Narrator:"""

prompt = PromptTemplate(input_variables=["history", "input"], template=template)

# Set up the conversation memory
memory = ConversationBufferMemory(human_prefix="Child", ai_prefix="Narrator")

# Function to generate response
def generate_response(input_text):
    history = memory.load_memory_variables({})['history']
    full_prompt = prompt.format(history=history, input=input_text)
    response = pipe(full_prompt, max_length=len(full_prompt) + 100)[0]['generated_text']
    response = response.split("Narrator:")[-1].strip()
    memory.save_context({"input": input_text}, {"output": response})
    return response

# Function to interact with the chatbot
def chat_with_bot():
    print("Welcome to the interactive storytelling chatbot! Type 'quit' to exit.")
    while True:
        user_input = input("Child: ")
        if user_input.lower() == 'quit':
            break
        try:
            response = generate_response(user_input)
            print("Narrator:", response)
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Let's try again.")

In [ ]:
# Start the chat
chat_with_bot()

In [ ]:
# Seems to ignore the system prompt to end the story after 3 turns.
# "After the user has interacted with you 3 times, you should end the story."
chat_with_bot()

In [ ]:
# The chatbot seems capricious - sometimes it ignores the system prompt like above
# and sometimes the system prompt can break the bot
# (here I said: "Your answers should be one sentence responses between 3 to 5 words")
chat_with_bot()